# Εισαγωγή

Στην παρούσα εργασία υλοποιήθηκε βήμα–βήμα ένα σύστημα συστάσεων ταινιών, ακολουθώντας την εκφώνηση της άσκησης. Η ανάλυση βασίστηκε στο dataset **MovieLens 10M**, το οποίο περιλαμβάνει αξιολογήσεις χρηστών σε ταινίες, καθώς και βασικές πληροφορίες για κάθε ταινία (τίτλο, είδη).

Η ροή της εργασίας οργανώθηκε ως εξής:

1. **Φόρτωση και προεπεξεργασία δεδομένων**  
   - Διαβάσαμε τα αρχεία `ratings.dat` και `movies.dat`.  
   - Ενοποιήσαμε τα δεδομένα, δημιουργήσαμε στήλη έτους και καθαρίσαμε τα genres.  

2. **Ανάλυση ανά είδος (genres)**  
   - Εξετάσαμε πώς εξελίχθηκε η μέση βαθμολογία των ταινιών ανά είδος.  
   - Υπολογίσαμε ποια είδη παρουσίασαν τη μεγαλύτερη μείωση στη μέση βαθμολογία.  
   - Ελέγξαμε αν ο αριθμός αξιολογήσεων επηρεάζει τα αποτελέσματα.

3. **Εκπαίδευση recommendation system**  
   - Χωρίσαμε τα δεδομένα σε train, validation και test set με βάση το έτος.  
   - Υλοποιήσαμε μοντέλο **Matrix Factorization** με gradient descent, και χρησιμοποιήσαμε **early stopping** για πιο σταθερή εκπαίδευση.  
   - Αξιολογήσαμε το μοντέλο σε γνωστές (known).

4. **Απαντήσεις στα θεωρητικά ερωτήματα**  
   - Συζητήθηκε πώς θα μπορούσαμε να προτείνουμε ταινίες σε έναν νέο χρήστη χωρίς ιστορικό.  
   - Εξετάστηκε το σενάριο όπου ο χρήστης έδωσε 3 αγαπημένες ταινίες.

---

### Επέκταση (εκτός εκφώνησης)

Για λόγους εκπαίδευσης και πειραματισμού προχώρησα ένα βήμα παραπέρα:  
- Κατέβασα από το **The Movie Database (TMDB)** τις περιγραφές (overviews) ταινιών.  
- Δημιούργησα **embeddings** των περιγραφών με το μοντέλο *Sentence-BERT (all-MiniLM-L6-v2)*.  
- Για ταινίες χωρίς overview, χρησιμοποίησα embeddings από τα genres.  
- Ενσωμάτωσα αυτά τα embeddings στο σύστημα ώστε να μπορεί να κάνει προτάσεις και σε cold-start ταινίες.  

Στο **ερώτημα 6**, όπου ο νέος χρήστης δίνει ως αγαπημένες τις *Iron Man*, *300* και *Transformers*, δοκίμασα 3 διαφορετικές μεθόδους συστάσεων:  
1. Με βάση το περιεχόμενο (content-only).  
2. Με βάση την ομοιότητα στα Q του MF (item–item).  
3. Υβριδική προσέγγιση (MF + content embeddings).  

Έτσι φάνηκε στην πράξη πώς διαφορετικές τεχνικές μπορούν να επηρεάσουν τις προτεινόμενες ταινίες.  

---

### Δομή του Notebook

Στην αρχή παρουσιάζονται οι βιβλιοθήκες και οι βασικές συναρτήσεις που χρησιμοποιήθηκαν.  
Οι συναρτήσεις αυτές δημιουργήθηκαν σε παλαιότερες εργασίες και σε κάθε νέα εργασία, αν χρειαστεί, επεκτείνονται σε λειτουργικότητα και επαναχρησιμοποιούνται.  
Με αυτόν τον τρόπο υπάρχει συνέπεια στη ροή της δουλειάς και μειώνεται ο χρόνος υλοποίησης.

Έπειτα ακολουθούν τα βήματα που ζητά η εκφώνηση της εργασίας.  
Στο τέλος παρουσιάζεται μια επέκταση, όπου κατεβάσαμε περιγραφές ταινιών (overviews) από το TMDB, δημιουργήσαμε embeddings και δείξαμε προτάσεις για το Ερώτημα 6 με τρεις διαφορετικές μεθόδους.

In [1]:
# --- Βασικές βιβλιοθήκες ---

# Διαχείριση αιτημάτων HTTP / λήψη δεδομένων από URL
import urllib.request  
# Αποσυμπίεση και συμπίεση αρχείων ZIP
import zipfile          
# Λειτουργίες συστήματος αρχείων (paths, περιβάλλον κλπ.)
import os               
# Επεξεργασία και ανάλυση δεδομένων σε πίνακες (DataFrames)
import pandas as pd     
# Αριθμητικοί υπολογισμοί, πίνακες και μαθηματικές πράξεις
import numpy as np      
# Δημιουργία/χρήση αραιών μητρώων (sparse matrices)
from scipy.sparse import coo_matrix  
# Αποστολή αιτημάτων HTTP (πιο εύκολα από urllib)
import requests         
# Φόρτωση μεταβλητών περιβάλλοντος από αρχείο .env
from dotenv import load_dotenv  
# Διαχείριση χρόνου (καθυστερήσεις, timestamps)
import time             
# Μοντέλα embeddings για NLP (sentence transformers)
from sentence_transformers import SentenceTransformer  
# Κανονικές εκφράσεις για επεξεργασία κειμένου
import re  

# --- Ρυθμίσεις εμφάνισης pandas ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

### Συνάρτηση: `download_movielens`

**Σκοπός:**  
Κατεβάζει το MovieLens dataset (ή άλλο zip αρχείο) από URL, το αποθηκεύει τοπικά και το αποσυμπιέζει.  

**Τι κάνει:**  
- Δημιουργεί τον φάκελο προορισμού αν δεν υπάρχει  
- Ελέγχει αν υπάρχει ήδη το zip τοπικά, αλλιώς το κατεβάζει  
- Αποσυμπιέζει το zip αρχείο στον φάκελο προορισμού  
- Επιστρέφει το path του φακέλου προορισμού  

**Parameters:**  
- `url : str` → το URL του zip αρχείου  
- `target_dir : str` → φάκελος προορισμού  
- `zip_name : str` → όνομα για το αποθηκευμένο zip  

**Returns:**  
- `target_dir : str` → διαδρομή φακέλου με τα αποσυμπιεσμένα δεδομένα  

In [2]:
def download_movielens(url, target_dir, zip_name):
    """
    Κατέβασμα και αποσυμπίεση MovieLens dataset (ή άλλου zip).

    Parameters
    ----------
    url : str
        Το URL από το οποίο θα κατέβει το zip αρχείο.
    target_dir : str
        Ο φάκελος προορισμού για το zip και τα αποσυμπιεσμένα αρχεία.
    zip_name : str
        Το όνομα του zip αρχείου που θα αποθηκευτεί τοπικά.

    Returns
    -------
    str
        Το path του φακέλου όπου βρίσκονται τα αποσυμπιεσμένα δεδομένα.
    """
    # Δημιουργία φακέλου προορισμού αν δεν υπάρχει
    os.makedirs(target_dir, exist_ok=True)

    # Ορισμός πλήρους διαδρομής για το zip
    zip_path = os.path.join(target_dir, zip_name)

    # Κατέβασμα zip αν δεν υπάρχει ήδη
    if not os.path.exists(zip_path):
        print(f"Κατέβασμα από {url}...")
        urllib.request.urlretrieve(url, zip_path)
        print("ok yes ok yes")
    else:
        print("Το αρχείο zip υπάρχει ήδη")

    # Αποσυμπίεση zip στον φάκελο προορισμού
    with zipfile.ZipFile(zip_path, 'r') as z:
        print(f"Αποσυμπίεση στο {target_dir}...")
        z.extractall(target_dir)
        print("Ok!!")

    return target_dir

### Συνάρτηση: `load_data`

**Σκοπός:**  
Φορτώνει δεδομένα από αρχείο (CSV/Excel/.dat) ή από ενσωματωμένο dataset του `sklearn`.

**Τι κάνει:**  
- Αν `source='file'`:  
  - Υποστηρίζει `.csv`, `.xls/.xlsx`, `.dat`  
  - Εμφανίζει διαθέσιμα φύλλα για Excel αν δεν δοθεί `sheet_name`  
  - Στα `.dat` ζητάει από τον χρήστη encoding και separator  
- Αν `source='sklearn'`: φορτώνει dataset από `sklearn.datasets` και επιστρέφει `(df, target)`  
- Εκτυπώνει πληροφορίες για το σχήμα των δεδομένων  
- Σε αποτυχία, εμφανίζει μήνυμα σφάλματος και επιστρέφει `(None, None)`  

**Parameters:**  
- `source : str` → `'file'` (default) ή `'sklearn'`  
- `filepath : str or None` → διαδρομή αρχείου (για source='file')  
- `dataset_func : callable or None` → π.χ. `load_iris` (για source='sklearn')  
- `sheet_name : str or None` → όνομα φύλλου Excel  

**Returns:**  
- `df : pd.DataFrame` ή `None` → τα δεδομένα  
- `target : np.ndarray` ή `None` → ετικέτες στόχου (μόνο για sklearn datasets)  

In [3]:
def load_data(source='file', filepath=None,  dataset_func=None, sheet_name=None):
    """
    Φόρτωση δεδομένων από αρχείο (CSV/Excel/.dat) ή από ενσωματωμένο dataset του sklearn.

    Parameters
    ----------
    source : str, optional
        'file' για τοπικό αρχείο (default) ή 'sklearn' για dataset του sklearn
    filepath : str or None
        Διαδρομή αρχείου όταν source='file'. Υποστηρίζει .csv, .xls, .xlsx, .dat
    dataset_func : callable or None
        Συνάρτηση από sklearn.datasets (π.χ. load_iris) όταν source='sklearn'
    sheet_name : str or None
        Όνομα φύλλου Excel (χρειάζεται μόνο για .xls/.xlsx)

    Returns
    -------
    tuple
        (df, target) όπου:
        df : pd.DataFrame με τα δεδομένα ή None
        target : np.ndarray ή None με ετικέτες στόχου (για sklearn datasets)
    """
    if source == 'file':
        if not filepath:
            print("\nΠαρακαλώ δώσε filepath για CSV.")
            return None, None
        try:
            if filepath.endswith('.csv'):
                try:
                    df = pd.read_csv(filepath, encoding="utf-8")
                    print("\nDataset φορτώθηκε από CSV αρχείο:", df.shape)
                except UnicodeDecodeError:
                    df = pd.read_csv(filepath, encoding="utf-8-sig")
                    print("\nDataset φορτώθηκε από CSV αρχείο:", df.shape)
                return df, None
                
            elif filepath.endswith(('.xls', '.xlsx')):
                if sheet_name is None:
                    # εμφάνιση διαθέσιμων φύλλων (sheets) για επιλογή
                    xls = pd.ExcelFile(filepath)
                    print("\nΔιαθέσιμα φύλλα εργασίας (sheets):", xls.sheet_names)
                    print("Χρησιμοποίησε το όρισμα sheet_name για να διαλέξεις φύλλο.")
                    return None, None
                else:
                    df = pd.read_excel(filepath, sheet_name=sheet_name)
                    print("\n Dataset φορτώθηκε από XLS/XLSX αρχείο:", df.shape)
                    return df, None
                    
            elif filepath.endswith('.dat'):
                try:
                    encoding = input("Δώσε encoding (π.χ. utf-8 ή latin-1): ")
                    separator = input("Δώσε διαχωριστικό (π.χ. ',' ή '::' ή '\\t'): ")
                    
                    # Αν ο χρήστης δεν δώσει τίποτα, βάλε default
                    if encoding.strip() == "":
                        encoding = "utf-8"
                    if separator.strip() == "":
                        separator = ","
                
                    df = pd.read_csv(filepath, sep=separator, encoding=encoding, engine="python", header=None)
                    print("\nDataset φορτώθηκε από .dat αρχείο:", df.shape)
                    return df, None
                
                except UnicodeDecodeError:
                    print("\n Σφάλμα: Το encoding που δόθηκε δεν ταιριάζει με το αρχείο.")
                    return None, None
        
                except pd.errors.ParserError:
                    print("\n Σφάλμα: Το διαχωριστικό που δόθηκε μάλλον δεν ταιριάζει.")
                    return None, None
        
        except Exception as e:
            print("\nΣφάλμα κατά το διάβασμα του αρχείου:", e)
            return None, None

    elif source == 'sklearn':
        if not dataset_func:
            print("\nΠαρακαλώ δώσε συνάρτηση π.χ. load_iris για φόρτωση sklearn dataset.")
            return None, None
        try:
            dataset = dataset_func()
            df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
            target = dataset.target
            print(f"\nDataset φορτώθηκε από sklearn ({dataset_func.__name__}):", df.shape)
            return df, target
        except Exception as e:
            print(f"\nΣφάλμα κατά το φόρτωμα του Dataset:", e)
            return None, None

    else:
        print("\nΜη έγκυρη επιλογή source. Δοκίμασε: 'file' ή 'sklearn'.")
        return None, None

### Συνάρτηση: `inspect_data`

**Σκοπός:**  
Γρήγορη επισκόπηση ενός `DataFrame`.

**Τι κάνει:**  
- Εμφανίζει σχήμα (γραμμές, στήλες)  
- Τύπους δεδομένων (`info()`)  
- Πρώτες γραμμές (`head()`)  
- Περιγραφικά στατιστικά (`describe()`)  
- Έλεγχο για NaN τιμές  
- Αν δοθεί `target_column`: εμφανίζει κατανομή τιμών (counts + ποσοστά)  

**Parameters:**  
- `df : pd.DataFrame` → το DataFrame προς ανάλυση  
- `target_column : str, optional` → στήλη με τις ετικέτες στόχου, αν υπάρχει  

**Returns:**  
- Δεν επιστρέφει κάτι, απλά τυπώνει πληροφορίες  

In [4]:
def inspect_data(df, target_column=None):
    """
    Γρήγορη επισκόπηση ενός DataFrame.

    Εμφανίζει:
    - Σχήμα (γραμμές, στήλες)
    - Τύπους δεδομένων και πλήθος NaN
    - Πρώτες γραμμές (head)
    - Περιγραφικά στατιστικά (describe)
    - Αν δοθεί target_column: κατανομή τιμών (counts και ποσοστά)

    Parameters
    df : pd.DataFrame
        Το DataFrame προς ανάλυση.
    target_column : str, optional
        Στήλη στόχου (π.χ. για ταξινόμηση). Αν υπάρχει, εμφανίζεται η κατανομή της.
    """
    print(f"\nΣχήμα DataFrame: {df.shape}")
    print("\nΠληροφορίες DataFrame:")
    df.info()
    print("\nΠρώτες 5 γραμμές:")
    print(df.head())
    print("\nΠεριγραφικά στατιστικά:")
    print(df.describe())
    print("\nΈλεγχος για NaN:")
    print(df.isna().sum())

    if target_column is not None and target_column in df.columns:
        print(f"\nΚατανομή Target labels ({target_column}):")
        print(df[target_column].value_counts())
        print("\nΚατανομή σε ποσοστά:")
        print(df[target_column].value_counts(normalize=True) * 100)

### Συνάρτηση: `make_maps`

**Σκοπός:**  
Δημιουργεί λεξικά αντιστοίχισης για κατηγορίες μιας στήλης.

**Τι κάνει:**  
- Παίρνει τις μοναδικές τιμές μιας στήλης  
- Δημιουργεί ένα λεξικό που αντιστοιχίζει κατηγορία → ακέραιο index  
- Δημιουργεί και το αντίστροφο λεξικό (index → κατηγορία)  
- Επιστρέφει και τα δύο λεξικά  

**Parameters:**  
- `df : pd.DataFrame` → ο πίνακας δεδομένων  
- `col : str` → όνομα στήλης για την οποία θα φτιαχτούν οι αντιστοιχίσεις  

**Returns:**  
- `val_map : dict` → αντιστοίχιση `τιμή → index`  
- `val_imap : dict` → αντιστοίχιση `index → τιμή` 

In [5]:
def make_maps(df, col):
    """
    Δημιουργία λεξικών αντιστοίχισης για τις τιμές μιας στήλης.

    Parameters
    ----------
    df : pd.DataFrame
        Ο πίνακας δεδομένων.
    col : str
        Το όνομα της στήλης για την οποία θα δημιουργηθούν οι αντιστοιχίσεις.

    Returns
    -------
    tuple
        val_map : dict
            Λεξικό με αντιστοίχιση τιμή -> index.
        val_imap : dict
            Λεξικό με αντιστοίχιση index -> τιμή.
    """
    # Εύρεση μοναδικών τιμών της στήλης
    unique_vals = df[col].unique()
    
    # Δημιουργία λεξικού: τιμή -> index
    val_map  = {val: idx for idx, val in enumerate(unique_vals)}
    
    # Δημιουργία αντίστροφου λεξικού: index -> τιμή
    val_imap = {idx: val for idx, val in enumerate(unique_vals)}
    
    return val_map, val_imap

## Σημείωση για τις συναρτήσεις εκπαίδευσης

Ακολουθούν παρόμοιες συναρτήσεις εκπαίδευσης (**train**, **rmse**) με μικρές διαφοροποιήσεις:

- Η βασική διαφορά στις εκδόσεις `train_mf_sgd` και `train_mf_sgd_2` είναι ότι στη δεύτερη υπάρχει δυνατότητα εισαγωγής **content embeddings** και διαφορετικός τρόπος εκπαίδευσης όταν αυτά υπάρχουν.  
- Επίσης, οι συναρτήσεις `train` και `rmse` στην αρχική τους μορφή δεν είχαν **typecasting** σε συγκεκριμένους αριθμητικούς τύπους. Στις εκδόσεις με κατάληξη `_2` προστέθηκε χρήση `np.int32` / `np.float32` ώστε να μειωθεί η μνήμη και να επιταχυνθεί η εκπαίδευση, λόγω περιορισμένης υπολογιστικής ισχύος (CPU) και μεγάλης διάρκειας training.  
- Οι εκδόσεις με κατάληξη `_2` δεν είναι απαραίτητες για τα ερωτήματα της εργασίας· υλοποιήθηκαν καθαρά ως **συμπληρωματικά βήματα** που ήθελα να δοκιμάσω.  

Λόγω περιορισμένου χρόνου, δεν μπόρεσα να ενοποιήσω τις εκδοχές αυτές σε μία γενικότερη συνάρτηση με επιπλέον ορίσματα που θα ενεργοποιούσαν την αντίστοιχη λειτουργικότητα (embeddings, typecasting κ.λπ.), γι’ αυτό και υπάρχουν ξεχωριστές εκδόσεις.

### Συνάρτηση: `train_mf_sgd`

**Σκοπός:**  
Εκπαίδευση Matrix Factorization με Stochastic Gradient Descent (SGD) πάνω σε αραιό πίνακα αξιολογήσεων.

**Τι κάνει:**  
- Αρχικοποιεί τυχαία τους παραγοντικούς πίνακες `P` (users × K) και `Q` (items × K)  
- Κάνει **shuffle** των δεικτών του train και περνάει από mini-batches  
- Εφαρμόζει SGD update για κάθε `(u, i, r)` με **L2 regularization**  
- Προαιρετικά: υπολογίζει **validation RMSE** μέσω `rmse_known` και χρησιμοποιεί **early stopping** με `EarlyStopPQ`  
- Αν γίνει early stop, επαναφέρει τα **best weights** (P, Q) που πέτυχαν το χαμηλότερο val RMSE  
- Επιστρέφει τους πίνακες `P` και `Q`

**Parameters:**  
- `R_coo : scipy.sparse.coo_matrix` → sparse COO πίνακας TRAIN (users × items) με ratings  
- `n_users : int` → πλήθος μοναδικών χρηστών στο TRAIN  
- `n_items : int` → πλήθος μοναδικών items/ταινιών στο TRAIN  
- `K : int, optional` → διαστασιμότητα των latent factors (default=20)  
- `epochs : int, optional` → αριθμός εποχών εκπαίδευσης (default=100)  
- `batch_size : int, optional` → μέγεθος mini-batch σε αριθμό ratings (default=100_000)  
- `lr : float, optional` → learning rate (η) (default=0.01)  
- `reg : float, optional` → L2 regularization (λ) (default=0.05)  
- `seed : int, optional` → seed για reproducibility (default=42)  
- `verbose : bool, optional` → αν θα τυπώνει πρόοδο (default=True)  
- `val_known : any or None` → δομή/δεδομένα για υπολογισμό validation RMSE (αν δοθεί)  
- `patience : int, optional` → υπομονή για early stopping (default=5)  
- `min_delta : float, optional` → ελάχιστη βελτίωση για early stopping (default=1e-4)

**Returns:**  
- `P : np.ndarray` → πίνακας χρηστών (n_users × K)  
- `Q : np.ndarray` → πίνακας items (n_items × K)

In [6]:
def train_mf_sgd(
    R_coo,                 # sparse COO του TRAIN (users x items) με τα ratings
    n_users, n_items,      # πλήθος μοναδικών χρηστών/ταινιών στο TRAIN
    K=20,                  # μήκος διανύσματος (latent factors)
    epochs=100,            # πόσες περασιές
    batch_size=100_000,    # πόσα τυχαία ratings ανά epoch
    lr=0.01,               # learning rate (η)
    reg=0.05,              # L2 regularization (λ)
    seed=42,               # seed για reproducibility
    verbose=True,          # αν θα τυπώνει πρόοδο
    val_known=None,        # (προαιρετικό) validation data/δομή για υπολογισμό RMSE σε κάθε epoch   
    patience=5,            # early stopping: πόσα συνεχόμενα epochs «χωρίς ουσιαστική βελτίωση» θα ανεχτούμε
    min_delta=1e-4         # early stopping: ελάχιστη απαιτούμενη βελτίωση στο val RMSE για να θεωρηθεί πρόοδος
):
    """
    Matrix Factorization με SGD (χωρίς biases),
    Επιστρέφει τους πίνακες παραγόντων P (users x K) και Q (items x K).
    """

    stopper = EarlyStopPQ(patience=patience, min_delta=min_delta)

    rng = np.random.default_rng(seed)

    # --- Αρχικοποίηση παραγόντων (μικρές τυχαίες τιμές γύρω από 0)
    P = 0.1 * rng.standard_normal((n_users, K))
    Q = 0.1 * rng.standard_normal((n_items, K))

    # --- Παίρνουμε τις μη-μηδενικές θέσεις του TRAIN (πιο αποδοτικά από COO)
    rows  = R_coo.row          # UserIdx για κάθε rating στο train
    cols  = R_coo.col          # MovieIdx για κάθε rating στο train
    data  = R_coo.data.astype(float, copy=False)  # πραγματικά ratings (float)

    n = len(data)              # συνολικός αριθμός train ratings
    if n == 0:
        if verbose:
            print("WARN: Empty training set. Returning initial P, Q.")
        return P, Q

    for epoch in range(1, epochs + 1):
        # 1) shuffle όλων των δεικτών του train
        idx_all = np.arange(n)
        rng.shuffle(idx_all)

        # 2) πέρασμα σε mini-batches ώστε να καλύψεις όλο το train
        num_batches = int(np.ceil(n / batch_size))
        if verbose:
            print(f"Epoch {epoch}/{epochs} — {num_batches} batches των ~{batch_size:,} ratings")

        
       # πέρασμα όλων των mini-batches
        for b in range(num_batches):
            start = b * batch_size
            end   = min(start + batch_size, n)
            idx   = idx_all[start:end]

            u_batch = rows[idx]
            i_batch = cols[idx]
            r_batch = data[idx]

            take = end - start 
        
            # --- πυρήνας SGD: ένα update ανά (u,i,r)
            for k in range(take):
                u = u_batch[k]
                i = i_batch[k]
                r = r_batch[k]

                # πρόβλεψη με dot product
                pred = float(np.dot(P[u], Q[i]))

                # σφάλμα
                err = r - pred

                # κρατάμε παλιό P[u] για να ενημερώσουμε σωστά το Q[i]
                Pu_old = P[u].copy()

                # ενημερώσεις με L2 regularization
                P[u] += lr * (err * Q[i]   - reg * P[u])
                Q[i] += lr * (err * Pu_old - reg * Q[i])
            
        if val_known is not None:
            val_rmse = rmse_known(P, Q, val_known)
            if verbose:
                print(f"[val] epoch {epoch} RMSE={val_rmse:.4f}")

            stopper(val_rmse, P=P, Q=Q, epoch=epoch)
            if stopper.early_stop:
                if verbose:
                    print("Early stopping triggered.")
                break
        else:
            if verbose:
                # αν δεν έχεις δώσει validation, απλώς εκτύπωσε πρόοδο
                print(f"Epoch {epoch}/{epochs} — processed {take:,} ratings")
            
        
    # --- μετά το loop ---
    if (stopper.best_P is not None) and (stopper.best_Q is not None):
        P, Q, best_rmse, best_epoch = stopper.restore()
        print(f"Restored best weights from epoch {best_epoch} "
              f"(best val rmse: {best_rmse:.4f}).")

    return P, Q

### Συνάρτηση: `rmse_known`

**Σκοπός:**  
Υπολογισμός του Root Mean Squared Error (RMSE) μόνο για ratings όπου είναι γνωστά και τα δύο indices (`UserIdx`, `MovieIdx`).  

**Τι κάνει:**  
- Παίρνει από `df_known` τους δείκτες χρηστών, ταινιών και τις πραγματικές βαθμολογίες  
- Υπολογίζει τις προβλέψεις ως dot product των παραγόντων χρηστών και ταινιών (`P[u] · Q[i]`)  
- Υπολογίζει το **Mean Squared Error (MSE)** και τη ρίζα του (RMSE)  
- Επιστρέφει την τιμή RMSE  

**Parameters:**  
- `P : np.ndarray` → παράγοντες χρηστών (n_users × K)  
- `Q : np.ndarray` → παράγοντες ταινιών (n_items × K)  
- `df_known : pd.DataFrame` → DataFrame με στήλες `UserIdx`, `MovieIdx`, `rating` (χωρίς NaN)  

**Returns:**  
- `rmse : float` → τιμή Root Mean Squared Error  

In [7]:
def rmse_known(P, Q, df_known):
    """
    Υπολογίζει RMSE μόνο για γραμμές όπου υπάρχουν UserIdx και MovieIdx (known-known).

    Parameters
    ----------
    P : np.ndarray
        Factors χρηστών (πίνακας n_users x K).
    Q : np.ndarray
        Factors ταινιών (πίνακας n_items x K).
    df_known : pd.DataFrame
        DataFrame με στήλες UserIdx, MovieIdx, rating (χωρίς NaN στους δύο δείκτες).

    Returns
    -------
    float
        Η τιμή RMSE πάνω στις γνωστές τριάδες (u,i,r).
    """
    # Μετατροπή των στηλών του DataFrame σε numpy arrays
    u = df_known['UserIdx'].to_numpy(int)    # indices χρηστών
    i = df_known['MovieIdx'].to_numpy(int)   # indices ταινιών
    y = df_known['rating'].to_numpy(float)   # πραγματικές βαθμολογίες

    # Υπολογισμός προβλέψεων:
    # για κάθε (u,i) παίρνουμε το dot product των διανυσμάτων P[u] και Q[i]
    yhat = (P[u] * Q[i]).sum(axis=1)

    # Υπολογισμός Mean Squared Error
    mse = ((y - yhat) ** 2).mean()

    # Τετραγωνική ρίζα του MSE -> RMSE
    rmse = mse ** 0.5

    return rmse

### Συνάρτηση: `train_mf_sgd_2`

**Σκοπός:**  
Εκπαίδευση Matrix Factorization με Stochastic Gradient Descent (SGD), με δυνατότητα **content regularization** μέσω εξωτερικών embeddings `E` και γραμμικού χάρτη `W`.

**Τι κάνει:**  
- Αρχικοποιεί τυχαία τους πίνακες παραγόντων `P` (users × K) και `Q` (items × K)  
- Εκτελεί SGD σε mini-batches με **L2 regularization**  
- (Προαιρετικά) Αν δοθεί `E` και `reg_content>0`:  
  - Αρχικοποιεί και μαθαίνει έναν γραμμικό χάρτη `W` (διάστασης K × d)  
  - Προσθέτει όρο regularization ώστε τα item factors `Q[i]` να πλησιάζουν το `W @ E[i]`  
  - Κάνει update και στο `W` (με ρυθμό μάθησης `lr_w`, αν δοθεί αλλιώς `lr`)  
- (Προαιρετικά) Υπολογίζει **validation RMSE** με `rmse_known_2` και χρησιμοποιεί **early stopping** (`EarlyStopPQ`)  
- Στο τέλος, αν έγινε early stop, επαναφέρει τα **best weights** (P, Q)  
- Επιστρέφει `P`, `Q`, και (αν χρησιμοποιήθηκε) `W`

**Parameters:**  
- `R_coo : scipy.sparse.coo_matrix` → TRAIN ratings σε COO μορφή (users × items)  
- `n_users : int` → πλήθος μοναδικών χρηστών στο TRAIN  
- `n_items : int` → πλήθος μοναδικών items/ταινιών στο TRAIN  
- `K : int, optional` → διαστασιμότητα latent factors (default=20)  
- `epochs : int, optional` → αριθμός εποχών (default=100)  
- `batch_size : int, optional` → μέγεθος mini-batch σε πλήθος ratings (default=100_000)  
- `lr : float, optional` → learning rate για P, Q (default=0.01)  
- `reg : float, optional` → L2 regularization strength (default=0.05)  
- `seed : int, optional` → τυχαιότητα για reproducibility (default=42)  
- `verbose : bool, optional` → εκτύπωση προόδου (default=True)  
- `val_known : any or None` → δομή για validation RMSE με `rmse_known_2` (αν δοθεί)  
- `patience : int, optional` → υπομονή για early stopping (default=5)  
- `min_delta : float, optional` → ελάχιστη βελτίωση για early stopping (default=1e-4)  
- `E : np.ndarray or None` → content embeddings για τα items (σχήμα `n_items × d`)  
- `reg_content : float, optional` → ένταση content regularization (default=0.0)  
- `lr_w : float or None` → learning rate για τον χάρτη `W` (αν None, χρησιμοποιεί `lr`)

**Returns:**  
- `P : np.ndarray` → παράγοντες χρηστών (n_users × K)  
- `Q : np.ndarray` → παράγοντες items (n_items × K)  
- `W : np.ndarray or None` → γραμμικός χάρτης (K × d) αν χρησιμοποιήθηκε content regularization, αλλιώς `None`

In [8]:
def train_mf_sgd_2(
    R_coo,                 # sparse COO του TRAIN (users x items) με τα ratings
    n_users, n_items,      # πλήθος μοναδικών χρηστών/ταινιών στο TRAIN
    K=20,                  # μήκος διανύσματος (latent factors)
    epochs=100,            # πόσες περασιές
    batch_size=100_000,    # πόσα τυχαία ratings ανά epoch
    lr=0.01,               # learning rate (η)
    reg=0.05,              # L2 regularization (λ)
    seed=42,               # seed για reproducibility
    verbose=True,          # αν θα τυπώνει πρόοδο
    val_known=None,        # (προαιρετικό) validation data/δομή για υπολογισμό RMSE σε κάθε epoch
    patience=5,            # early stopping: πόσα συνεχόμενα epochs «χωρίς ουσιαστική βελτίωση» θα ανεχτούμε
    min_delta=1e-4,        # early stopping: ελάχιστη απαιτούμενη βελτίωση στο val RMSE για να θεωρηθεί πρόοδος
    E=None,                # content embeddings για items (προαιρετικό)
    reg_content=0.0,       # ένταση regularization προς content embeddings
    lr_w=None              # learning rate για τον χάρτη W
):
    """
    Matrix Factorization με SGD (χωρίς biases).
    Επιστρέφει τους πίνακες παραγόντων P (users x K), Q (items x K), 
    και προαιρετικά τον χάρτη W (K x d) αν χρησιμοποιείται content regularization.
    """

    # Early stopping μηχανισμός:
    # - patience: επιτρέπει έως 'patience' διαδοχικά epochs χωρίς βελτίωση > min_delta
    # - min_delta: ελάχιστη μείωση στο val RMSE για να θεωρηθεί «βελτίωση»
    stopper = EarlyStopPQ(patience=patience, min_delta=min_delta)

    # Τυχαίος αριθμοπαραγωγός με συγκεκριμένο seed
    rng = np.random.default_rng(seed)
    
    # --- Αν δόθηκαν embeddings E και ενεργοποιηθεί το content regularization
    if (E is not None) and (reg_content > 0):
        d = E.shape[1]   # διάσταση των content embeddings (π.χ. 384)
        # Αρχικοποίηση του W με μικρές τυχαίες τιμές
        W = 0.1 * rng.standard_normal((K, d)).astype(np.float32)
        # Αν δεν δοθεί ξεχωριστό lr για W, βάζουμε ίδιο με το lr
        if lr_w is None:
            lr_w = lr
    else:
        W = None

    # --- Αρχικοποίηση παραγόντων χρηστών (P) και items (Q) με μικρές τυχαίες τιμές
    P = (0.1 * rng.standard_normal((n_users, K))).astype(np.float32)
    Q = (0.1 * rng.standard_normal((n_items, K))).astype(np.float32)

    # --- Από τον COO παίρνουμε τις θέσεις (γραμμές=users, στήλες=items, δεδομένα=ratings)
    rows = R_coo.row
    cols = R_coo.col
    data = R_coo.data.astype(np.float32, copy=False)

    n = len(data)  # πλήθος ratings στο TRAIN
    if n == 0:
        if verbose:
            print("WARN: Empty training set. Returning initial P, Q.")
        return P, Q

    # --- Loop εκπαίδευσης
    for epoch in range(1, epochs + 1):
        # 1) κάνουμε shuffle όλα τα indices (για τυχαία σειρά δειγμάτων)
        idx_all = np.arange(n)
        rng.shuffle(idx_all)

        # 2) χωρίζουμε σε mini-batches
        num_batches = int(np.ceil(n / batch_size))
        if verbose:
            print(f"Epoch {epoch}/{epochs} — {num_batches} batches των ~{batch_size:,} ratings")

        # --- πέρασμα όλων των mini-batches
        for b in range(num_batches):
            start = b * batch_size
            end   = min(start + batch_size, n)
            idx   = idx_all[start:end]

            # batch χρηστών, items και ratings
            u_batch = rows[idx]
            i_batch = cols[idx]
            r_batch = data[idx]

            take = end - start  # μέγεθος batch

            # --- update για κάθε (u,i,r) στο batch
            for k in range(take):
                u = u_batch[k]  # index χρήστη
                i = i_batch[k]  # index item
                r = r_batch[k]  # πραγματικό rating

                # πρόβλεψη = εσωτερικό γινόμενο P[u] · Q[i]
                pred = float(np.dot(P[u], Q[i]))

                # σφάλμα πρόβλεψης
                err = r - pred

                # αντιγράφουμε το P[u] πριν το update, για χρήση στην ενημέρωση του Q[i]
                Pu_old = P[u].copy()

                # αν έχουμε content mapping: υπολογίζουμε WE_i = W @ E[i]
                WEi = None
                if (W is not None):
                    Ei  = E[i]      # embedding του item i (διάστασης d)
                    WEi = W @ Ei    # προβολή embedding στον χώρο latent (διάστασης K)

                # --- ενημέρωση P[u] με όρο σφάλματος + L2 regularization
                P[u] += lr * (err * Q[i] - reg * P[u])

                # --- ενημέρωση Q[i]
                if WEi is None:
                    # κλασικό update χωρίς content regularization
                    Q[i] += lr * (err * Pu_old - reg * Q[i])
                else:
                    # update με content regularization ώστε Q[i] να πλησιάζει WEi
                    Q[i] += lr * (err * Pu_old - reg * Q[i] - reg_content * (Q[i] - WEi))

                # --- ενημέρωση W (αν υπάρχει content regularization)
                if WEi is not None:
                    diff = (Q[i] - WEi)  # διαφορά ανάμεσα σε Q[i] και content προβολή WEi
                    # outer product diff·Ei για update του W
                    W += ( (lr_w if lr_w is not None else lr) * reg_content ) * np.outer(diff, Ei)

        # --- Validation RMSE και early stopping (αν υπάρχει validation set)
        if val_known is not None:
            # Υπολογισμός val RMSE. Αν δεν πέσει τουλάχιστον κατά min_delta σε σχέση με το καλύτερο ως τώρα,
            # ο μετρητής 'υπομονής' του stopper αυξάνεται. Όταν φτάσει το 'patience', σταματάμε.
            val_rmse = rmse_known_2(P, Q, val_known)
            if verbose:
                print(f"[val] epoch {epoch} RMSE={val_rmse:.4f}")

            stopper(val_rmse, P=P, Q=Q, epoch=epoch)  # ενημέρωση early stopper με το νέο score
            if stopper.early_stop:
                if verbose:
                    print("Early stopping triggered.")  # ενεργοποιήθηκε γιατί εξαντλήθηκε το patience
                break
        else:
            if verbose:
                print(f"Epoch {epoch}/{epochs} — processed {take:,} ratings")
            
    # --- Μετά το loop: αν υπάρχει καλύτερο P,Q από early stopping, τα επαναφέρουμε
    if (stopper.best_P is not None) and (stopper.best_Q is not None):
        P, Q, best_rmse, best_epoch = stopper.restore()
        print(f"Restored best weights from epoch {best_epoch} "
              f"(best val rmse: {best_rmse:.4f}).")

    return P, Q, W

### Συνάρτηση: `rmse_known_2`

**Σκοπός:**  
Υπολογισμός του Root Mean Squared Error (RMSE) για ratings όπου είναι γνωστοί και οι δύο δείκτες (`UserIdx`, `MovieIdx`), χρησιμοποιώντας τύπους δεδομένων `np.int32` και `np.float32` για μεγαλύτερη αποδοτικότητα.  

**Τι κάνει:**  
- Παίρνει από `df_known` τους δείκτες χρηστών, ταινιών και τις πραγματικές βαθμολογίες  
- Υπολογίζει τις προβλέψεις ως dot product των παραγόντων χρηστών και ταινιών (`P[u] · Q[i]`)  
- Υπολογίζει το **Mean Squared Error (MSE)** και από αυτό το RMSE  
- Επιστρέφει την τιμή RMSE  

**Parameters:**  
- `P : np.ndarray` → παράγοντες χρηστών (n_users × K)  
- `Q : np.ndarray` → παράγοντες ταινιών (n_items × K)  
- `df_known : pd.DataFrame` → DataFrame με στήλες `UserIdx`, `MovieIdx`, `rating`  

**Returns:**  
- `rmse : float` → τιμή Root Mean Squared Error  

In [9]:
def rmse_known_2(P, Q, df_known):
    """
    Υπολογίζει RMSE μόνο για γραμμές όπου υπάρχουν UserIdx και MovieIdx (known-known).

    Parameters
    ----------
    P : np.ndarray
        Factors χρηστών (πίνακας n_users x K).
    Q : np.ndarray
        Factors ταινιών (πίνακας n_items x K).
    df_known : pd.DataFrame
        DataFrame με στήλες UserIdx, MovieIdx, rating (χωρίς NaN στους δύο δείκτες).

    Returns
    -------
    float
        Η τιμή RMSE πάνω στις γνωστές τριάδες (u,i,r).
    """
    # Μετατροπή των στηλών σε numpy arrays με συγκεκριμένους τύπους για αποδοτικότητα
    u = df_known['UserIdx'].to_numpy(dtype=np.int32)    # indices χρηστών
    i = df_known['MovieIdx'].to_numpy(dtype=np.int32)   # indices ταινιών
    y = df_known['rating'].to_numpy(dtype=np.float32)   # πραγματικές βαθμολογίες

    # Υπολογισμός προβλέψεων:
    # για κάθε (u,i) παίρνουμε το dot product P[u] · Q[i]
    yhat = (P[u] * Q[i]).sum(axis=1)

    # Υπολογισμός Mean Squared Error (MSE)
    mse = ((y - yhat) ** 2).mean()

    # Root Mean Squared Error
    rmse = mse ** 0.5

    return rmse

### Κλάση: `EarlyStopPQ`

**Σκοπός:**  
Υλοποίηση μηχανισμού **Early Stopping** για εκπαίδευση Matrix Factorization, ώστε να σταματά η εκπαίδευση αν δεν υπάρχει ουσιαστική βελτίωση στο validation RMSE μετά από συγκεκριμένο αριθμό εποχών.  

**Τι κάνει:**  
- Παρακολουθεί την καλύτερη τιμή RMSE που έχει εμφανιστεί μέχρι τώρα  
- Αν το validation RMSE βελτιωθεί κατά περισσότερο από `min_delta`, αποθηκεύει τα αντίστοιχα βάρη (P, Q)  
- Αν δεν υπάρξει βελτίωση για `patience` συνεχόμενα epochs, ενεργοποιεί το flag `early_stop = True`  
- Παρέχει μέθοδο `restore()` για επαναφορά των βαρών (P, Q) που πέτυχαν το καλύτερο RMSE  

**Parameters:**  
- `patience : int` → αριθμός διαδοχικών epochs χωρίς βελτίωση που θα ανεχτούμε πριν το stop  
- `min_delta : float` → ελάχιστη μείωση του RMSE για να θεωρηθεί βελτίωση  

**Attributes:**  
- `best_rmse : float or None` → η καλύτερη τιμή RMSE που έχει βρεθεί  
- `best_P, best_Q : np.ndarray or None` → snapshots των παραγόντων χρηστών και items  
- `best_epoch : int or None` → epoch στο οποίο επιτεύχθηκε η καλύτερη τιμή  
- `early_stop : bool` → αν έχει ενεργοποιηθεί το early stopping  

**Methods:**  
- `__call__(val_rmse, P, Q, epoch)` → ενημερώνει την κατάσταση με το νέο RMSE και (αν είναι καλύτερο) αποθηκεύει snapshots  
- `restore()` → επιστρέφει `(best_P, best_Q, best_rmse, best_epoch)`

In [10]:
class EarlyStopPQ:
    def __init__(self, patience=5, min_delta=0.0):
        # Παράμετροι ελέγχου
        self.patience  = patience       # πόσα consecutive epochs χωρίς βελτίωση επιτρέπονται
        self.min_delta = min_delta      # ελάχιστη απαιτούμενη βελτίωση στο RMSE

        # Αρχικές τιμές
        self.best_rmse = None           # καλύτερο RMSE που έχουμε δει
        self.bad_count = 0              # μετρητής epochs χωρίς βελτίωση
        self.early_stop = False         # flag που δείχνει αν πρέπει να σταματήσουμε
        self.best_epoch = None          # epoch με το καλύτερο RMSE

        # Snapshots βαρών (P,Q) όταν βρεθεί καλύτερο RMSE
        self.best_P = None
        self.best_Q = None
        

    def __call__(self, val_rmse, P=None, Q=None, epoch=None):
        """
        Ενημερώνει την κατάσταση με το τρέχον val_rmse και (πιθανά) αποθηκεύει καλύτερα P,Q.

        Parameters
        ----------
        val_rmse : float
            Η νέα τιμή RMSE από το validation set.
        P, Q : np.ndarray or None
            Παράγοντες χρηστών και items για αποθήκευση snapshot (αν βελτιωθεί το RMSE).
        epoch : int or None
            Ο αριθμός του τρέχοντος epoch.

        Returns
        -------
        None
            Θέτει το flag early_stop = True αν ξεπεραστεί το patience.
        """
        # Αν δεν έχουμε ακόμη "καλύτερο RMSE" ή βελτιώθηκε κατά > min_delta
        if (self.best_rmse is None) or (self.best_rmse - val_rmse > self.min_delta):
            # Βελτίωση
            self.best_rmse = float(val_rmse)   # αποθήκευση καλύτερου RMSE
            self.bad_count = 0                 # μηδενισμός μετρητή "κακών" epochs
            self.early_stop = False            # συνεχίζουμε εκπαίδευση

            # Αν δόθηκαν P και Q, αποθηκεύουμε snapshot
            if (P is not None) and (Q is not None):
                self.best_P = P.copy()
                self.best_Q = Q.copy()

            # Αποθήκευση epoch όπου εμφανίστηκε η βελτίωση
            if epoch is not None:
                self.best_epoch = epoch
        else:
            # Δεν υπήρξε βελτίωση
            self.bad_count += 1
            # Αν ξεπεράσαμε το patience -> ενεργοποίηση early stopping
            if self.bad_count >= self.patience:
                self.early_stop = True

    def restore(self):
        """
        Επιστρέφει τα καλύτερα αποθηκευμένα βάρη (P, Q) και την καλύτερη τιμή RMSE.

        Returns
        -------
        tuple
            (best_P, best_Q, best_rmse, best_epoch)
        """
        if (self.best_P is None) or (self.best_Q is None):
            raise RuntimeError("No snapshot saved — έλεγξε αν η EarlyStopping κλήθηκε με P,Q.")
        return self.best_P, self.best_Q, self.best_rmse, self.best_epoch

### Συνάρτηση: `mean_l2`

**Σκοπός:**  
Υπολογίζει το μέσο διάνυσμα (mean vector) από μια λίστα embeddings και το κανονικοποιεί σε L2-norm.  

**Τι κάνει:**  
- Στοιχίζει όλα τα embeddings σε έναν πίνακα (`np.vstack`)  
- Υπολογίζει το μέσο όρο ανά διάσταση  
- Επιστρέφει το κανονικοποιημένο διάνυσμα (με L2 κανονικοποίηση ώστε να έχει μήκος 1)  

**Parameters:**  
- `vectors : list of np.ndarray` → λίστα από διανύσματα (embeddings) ίδιων διαστάσεων  

**Returns:**  
- `m_norm : np.ndarray` → το κανονικοποιημένο μέσο embedding (μονάδα ως προς L2) 

In [11]:
def mean_l2(vectors):
    """
    Υπολογίζει το μέσο όρο μιας λίστας από embeddings και επιστρέφει 
    το κανονικοποιημένο (L2) διάνυσμα.

    Parameters
    ----------
    vectors : list of np.ndarray
        Λίστα από διανύσματα (embeddings) με την ίδια διάσταση.

    Returns
    -------
    np.ndarray
        Το μέσο διάνυσμα κανονικοποιημένο ώστε ||m||_2 = 1.
    """
    # Στοιχίζουμε όλα τα embeddings σε έναν πίνακα 2D (n_vectors x dim)
    mat = np.vstack(vectors)

    # Υπολογίζουμε τον μέσο όρο κατά μήκος του άξονα 0 (ανά διάσταση)
    m = mat.mean(axis=0)

    # Κανονικοποιούμε το μέσο διάνυσμα ώστε να έχει L2 norm = 1
    return m / np.linalg.norm(m)

### Συνάρτηση: `movie_proto_embedding`

**Σκοπός:**  
Δημιουργεί το embedding μιας ταινίας, υπολογίζοντας τον μέσο όρο των **prototype embeddings** των genres που ανήκει η ταινία.  

**Τι κάνει:**  
- Για κάθε genre της ταινίας, παίρνει το αντίστοιχο prototype embedding από τον πίνακα `protos`  
- Αγνοεί genres που δεν υπάρχουν στο index του `protos`  
- Υπολογίζει το μέσο L2-normalized embedding μέσω της συνάρτησης `mean_l2`  
- Επιστρέφει το τελικό διάνυσμα embedding της ταινίας  

**Parameters:**  
- `genres : list of str` → λίστα με τα genres της ταινίας  

**Returns:**  
- `embedding : np.ndarray` → κανονικοποιημένο embedding της ταινίας  

In [12]:
# φτιάχνει embedding για μια ταινία: μέσος όρος των prototype embeddings των genres της
def movie_proto_embedding(genres, protos):
    """
    Υπολογίζει το embedding μιας ταινίας με βάση τα prototype embeddings των genres της.

    Parameters
    ----------
    genres : list of str
        Λίστα με genres στα οποία ανήκει η ταινία.

    protos : pandas.Series or pandas.DataFrame
        Αντιστοίχιση των ονομάτων των genres με τα αντίστοιχα διανύσματα
        πρωτοτύπων (prototype embeddings).

    Returns
    -------
    np.ndarray
        Το τελικό κανονικοποιημένο embedding της ταινίας.
    """
    # Συλλέγουμε τα prototype embeddings για κάθε genre της ταινίας
    # (αν το genre υπάρχει στο index του DataFrame 'protos')
    vecs = [protos[g] for g in genres if g in protos.index]

    # Υπολογίζουμε τον μέσο όρο και κανονικοποιούμε με L2 (unit vector)
    return mean_l2(vecs)

### Συνάρτηση: `normalize_title`

**Σκοπός:**  
Κανονικοποιεί τον τίτλο μιας ταινίας για πιο εύκολη αναζήτηση και σύγκριση.

**Τι κάνει:**  
- Μετατρέπει τον τίτλο σε πεζά.  
- Κρατά μόνο γράμματα, αριθμούς και κενά.  
- Αφαιρεί πολλαπλά κενά.  
- Επιστρέφει τον καθαρό τίτλο.  

**Parameters:**  
- `title : str` → αρχικός τίτλος της ταινίας  

**Returns:**  
- `norm_title : str` → κανονικοποιημένος τίτλος

In [13]:
def normalize_title(s):
    """Απλή κανονικοποίηση τίτλου για αναζήτηση (πεζοποίηση, αφαίρεση συμβόλων)."""
    s = str(s).lower()                              # πεζά
    s = re.sub(r'[^a-z0-9 ]+', ' ', s)              # μόνο γράμματα/αριθμοί/κενά
    s = re.sub(r'\s+', ' ', s).strip()              # συμπίεση πολλαπλών κενών
    return s

### Συνάρτηση: `extract_year_from_title`

**Σκοπός:**  
Εξάγει το έτος παραγωγής μιας ταινίας από τον τίτλο της (π.χ. `"Movie (2008)"`).  

**Τι κάνει:**  
- Αναζητά τετραψήφιο αριθμό μέσα σε παρενθέσεις.  
- Αν βρεθεί, επιστρέφεται ως ακέραιος.  
- Αν δεν υπάρχει, επιστρέφει `None`.  

**Parameters:**  
- `title : str` → τίτλος ταινίας  

**Returns:**  
- `year : int | None` → το έτος, αν βρέθηκε

In [14]:
def extract_year_from_title(title):
    """Προσπάθεια εξαγωγής έτους από τίτλο τύπου 'Movie (2008)'. Αν δεν υπάρχει, επιστρέφει None."""
    m = re.search(r'\((\d{4})\)', str(title))
    return int(m.group(1)) if m else None

### Συνάρτηση: `l2_norm_rows`

**Σκοπός:**  
Κανονικοποιεί κάθε διάνυσμα ενός πίνακα σε μήκος 1 (L2-normalization).  

**Τι κάνει:**  
- Υπολογίζει το L2 μήκος (norm) κάθε γραμμής.  
- Διαιρεί κάθε στοιχείο της γραμμής με το αντίστοιχο norm.  
- Αν το norm είναι πολύ μικρό, χρησιμοποιεί μια μικρή σταθερά για σταθερότητα.  

**Parameters:**  
- `X : np.ndarray` → πίνακας διανυσμάτων (N × d)  

**Returns:**  
- `X_normed : np.ndarray` → πίνακας με normalized διανύσματα

In [15]:
def l2_norm_rows(X, eps = 1e-8):
    """L2 κανονικοποίηση κάθε γραμμής (κάθε διάνυσμα να έχει μήκος ~1)."""
    norms = np.linalg.norm(X, axis=1, keepdims=True)         # μήκος κάθε διανύσματος
    return X / np.clip(norms, eps, None)                     # διαίρεση με προσοχή σε πολύ μικρά

### Συνάρτηση: `mean_cosine_to_seeds`

**Σκοπός:**  
Υπολογίζει τη μέση ομοιότητα (cosine similarity) κάθε item προς μια ομάδα seed διανυσμάτων.  

**Τι κάνει:**  
- Κανονικοποιεί τα διανύσματα items και seeds.  
- Υπολογίζει όλα τα dot products (cosine similarities).  
- Για κάθε item παίρνει τον μέσο όρο των ομοιοτήτων προς όλα τα seeds.  

**Parameters:**  
- `items : np.ndarray` → πίνακας διανυσμάτων items (N × d)  
- `seeds : np.ndarray` → πίνακας διανυσμάτων seeds (M × d)  

**Returns:**  
- `scores : np.ndarray` → μονοδιάστατος πίνακας με τις μέσες ομοιότητες για κάθε item

In [16]:
def mean_cosine_to_seeds(items, seeds):
    """
    Υπολογίζει τη ΜΕΣΗ cosine ομοιότητα κάθε item προς ΟΛΑ τα seed διανύσματα.
    - items: πίνακας (N, D) με διανύσματα προς βαθμολόγηση
    - seeds: πίνακας (S, D) με seed διανύσματα
    Επιστρέφει: διάνυσμα (N,) με τον μέσο όρο των cosine similarities.
    """
    A = l2_norm_rows(items)           # κανονικοποίηση items
    B = l2_norm_rows(seeds)           # κανονικοποίηση seeds
    sims = A @ B.T                    # (N, S): όλα τα dot products
    return sims.mean(axis=1)          # (N,): μέσος όρος ανά item

### Λήψη MovieLens 10M dataset

Χρησιμοποιούμε τη βοηθητική συνάρτηση `download_movielens` για να κατεβάσουμε και να αποσυμπιέσουμε το αρχείο:
- URL: https://files.grouplens.org/datasets/movielens/ml-10m.zip  

In [17]:
url = "https://files.grouplens.org/datasets/movielens/ml-10m.zip"
dataset_dir = download_movielens(url, target_dir=r"C:\Users\giorg\Desktop\dataset_ml", zip_name="ml-10m.zip")

Το αρχείο zip υπάρχει ήδη
Αποσυμπίεση στο C:\Users\giorg\Desktop\dataset_ml...
Ok!!


In [18]:
print(os.listdir(dataset_dir))

['ml-10m.zip', 'ml-10M100K', 'ml-latest', 'ml-latest.zip']


### Φόρτωση αρχείου `ratings.dat`

Χρησιμοποιούμε τη συνάρτηση `load_data` για να φορτώσουμε το αρχείο με τις αξιολογήσεις:
- `source='file'` για διάβασμα από τοπικό αρχείο
- `filepath` προς το `ratings.dat`
Το αποτέλεσμα είναι ένα DataFrame με τις αξιολογήσεις.

In [19]:
# --- Load ratings.dat ---
# ΣΗΜΕΙΩΣΗ: Όταν το load_data() ζητήσει ρυθμίσεις, βεβαιωθείτε ότι επιλέγετε:
#           - Κωδικοποίηση (Encoding): utf-8
#           - Διαχωριστικό (Separator): ::
# Αυτές οι ρυθμίσεις είναι απαραίτητες για τη σωστή ανάγνωση των αρχείων .dat του MovieLens.
# Φόρτωση των αξιολογήσεων (ratings) από το αρχείο ratings.dat
ratings_df, _ = load_data(
    source='file',
    filepath=r"C:\Users\giorg\Desktop\dataset_ml\ml-10M100K\ratings.dat"
)

# Το αντικείμενο ratings_df είναι pandas DataFrame που περιέχει τις στήλες:
# userId :: movieId :: rating :: timestamp

Δώσε encoding (π.χ. utf-8 ή latin-1):  utf-8
Δώσε διαχωριστικό (π.χ. ',' ή '::' ή '\t'):  ::



Dataset φορτώθηκε από .dat αρχείο: (10000054, 4)


In [20]:
print("\n--- Πρώτες γραμμές από ratings ---")
print(ratings_df.head())


--- Πρώτες γραμμές από ratings ---
   0    1    2          3
0  1  122  5.0  838985046
1  1  185  5.0  838983525
2  1  231  5.0  838983392
3  1  292  5.0  838983421
4  1  316  5.0  838983392


In [21]:
# Ονοματοδοσία των στηλών στο DataFrame με τις αξιολογήσεις
ratings_df.columns = ["userId", "movieId", "rating", "timestamp"]

# Εμφάνιση των πρώτων γραμμών για έλεγχο
print("\n--- Πρώτες γραμμές από το DataFrame ratings ---")
print(ratings_df.head())


--- Πρώτες γραμμές από το DataFrame ratings ---
   userId  movieId  rating  timestamp
0       1      122     5.0  838985046
1       1      185     5.0  838983525
2       1      231     5.0  838983392
3       1      292     5.0  838983421
4       1      316     5.0  838983392


In [22]:
# --- Load movies.dat ---
# ΣΗΜΕΙΩΣΗ: Όταν το load_data() ζητήσει ρυθμίσεις, βεβαιωθείτε ότι επιλέγετε:
#           - Κωδικοποίηση (Encoding): utf-8
#           - Διαχωριστικό (Separator): ::
# Αυτές οι ρυθμίσεις είναι απαραίτητες για τη σωστή ανάγνωση των αρχείων .dat του MovieLens.
# Φόρτωση των ταινιών (movies) από το αρχείο movies.dat
movies_df, _ = load_data(
    source='file',
    filepath=r"C:\Users\giorg\Desktop\dataset_ml\ml-10M100K\movies.dat"
)

# Το αντικείμενο movies_df είναι pandas DataFrame που περιέχει τις στήλες:
# movieId :: title :: genres

Δώσε encoding (π.χ. utf-8 ή latin-1):  utf-8
Δώσε διαχωριστικό (π.χ. ',' ή '::' ή '\t'):  ::



Dataset φορτώθηκε από .dat αρχείο: (10681, 3)


In [23]:
print("\n--- Πρώτες γραμμές από movies ---")
print(movies_df.head())


--- Πρώτες γραμμές από movies ---
   0                                   1                                            2
0  1                    Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1  2                      Jumanji (1995)                   Adventure|Children|Fantasy
2  3             Grumpier Old Men (1995)                               Comedy|Romance
3  4            Waiting to Exhale (1995)                         Comedy|Drama|Romance
4  5  Father of the Bride Part II (1995)                                       Comedy


In [24]:
# Ονοματοδοσία των στηλών στο DataFrame με τις ταινίες
movies_df.columns = ["movieId", "title", "genres"]

# Εμφάνιση των πρώτων γραμμών για έλεγχο
print("\n--- Πρώτες γραμμές από το DataFrame movies ---")
print(movies_df.head())


--- Πρώτες γραμμές από το DataFrame movies ---
   movieId                               title                                       genres
0        1                    Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1        2                      Jumanji (1995)                   Adventure|Children|Fantasy
2        3             Grumpier Old Men (1995)                               Comedy|Romance
3        4            Waiting to Exhale (1995)                         Comedy|Drama|Romance
4        5  Father of the Bride Part II (1995)                                       Comedy


In [25]:
inspect_data(ratings_df)


Σχήμα DataFrame: (10000054, 4)

Πληροφορίες DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000054 entries, 0 to 10000053
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 305.2 MB

Πρώτες 5 γραμμές:
   userId  movieId  rating  timestamp
0       1      122     5.0  838985046
1       1      185     5.0  838983525
2       1      231     5.0  838983392
3       1      292     5.0  838983421
4       1      316     5.0  838983392

Περιγραφικά στατιστικά:
             userId       movieId        rating     timestamp
count  1.000005e+07  1.000005e+07  1.000005e+07  1.000005e+07
mean   3.586986e+04  4.120291e+03  3.512422e+00  1.032606e+09
std    2.058534e+04  8.938402e+03  1.060418e+00  1.159640e+08
min    1.000000e+00  1.000000e+00  5.000000e-01  7.896520e+08
25%    1.812300e+04  6.480000e+02  3.000000e+00  9.4

### Εξερεύνηση του DataFrame `ratings_df`

Μετά τη φόρτωση του αρχείου **ratings.dat** παρατηρούμε τα εξής:

- **Σχήμα:** 10.000.054 γραμμές και 4 στήλες  
- **Στήλες:** `userId`, `movieId`, `rating`, `timestamp`  
- **Τύποι δεδομένων:** `userId` (int64), `movieId` (int64), `rating` (float64), `timestamp` (int64)  
- **Μνήμη:** ~305 MB  
- **Στατιστικά:**  
  - Οι `userId` εκτείνονται από 1 έως 71.567  
  - Οι `movieId` εκτείνονται από 1 έως 65.133  
- **Έλεγχος NaN:** δεν υπάρχουν ελλιπείς τιμές

Το dataset είναι καθαρό και έτοιμο για περαιτέρω επεξεργασία.

In [26]:
inspect_data(movies_df)


Σχήμα DataFrame: (10681, 3)

Πληροφορίες DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10681 entries, 0 to 10680
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10681 non-null  int64 
 1   title    10681 non-null  object
 2   genres   10681 non-null  object
dtypes: int64(1), object(2)
memory usage: 250.5+ KB

Πρώτες 5 γραμμές:
   movieId                               title                                       genres
0        1                    Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1        2                      Jumanji (1995)                   Adventure|Children|Fantasy
2        3             Grumpier Old Men (1995)                               Comedy|Romance
3        4            Waiting to Exhale (1995)                         Comedy|Drama|Romance
4        5  Father of the Bride Part II (1995)                                       Comedy

Περιγραφικά στατιστικά:
      

### Εξερεύνηση του DataFrame `movies_df`

Μετά τη φόρτωση του αρχείου **movies.dat** παρατηρούμε τα εξής:

- **Σχήμα:** 10.681 γραμμές και 3 στήλες  
- **Στήλες:** `movieId`, `title`, `genres`  
- **Τύποι δεδομένων:** `movieId` (int64), `title` (object), `genres` (object)  
- **Μνήμη:** ~250 KB  
- **Παραδείγματα:** ταινίες από το 1995 όπως *Toy Story*, *Jumanji*, *Grumpier Old Men*  
- **Στατιστικά:**  
  - Τα `movieId` εκτείνονται από 1 έως 65.133  
  - Ο μέσος όρος του `movieId` είναι ~13.120  
- **Έλεγχος NaN:** δεν υπάρχουν ελλιπείς τιμές

Το dataset με τις ταινίες είναι καθαρό και περιέχει πληροφορίες τίτλου και ειδών (genres) για κάθε ταινία.

In [27]:
# Έλεγχος για διπλότυπες αξιολογήσεις (ίδιος χρήστης να έχει βαθμολογήσει την ίδια ταινία δύο φορές)
dup_ratings = ratings_df.duplicated(subset=["userId", "movieId"])
print("Διπλότυπα ratings:", dup_ratings.sum())  # Αποτέλεσμα: 0

# Έλεγχος για διπλότυπες εγγραφές ταινιών (ίδιο movieId, τίτλος και είδη)
dup_movies = movies_df.duplicated()
print("Διπλότυπα movies:", dup_movies.sum())  # Αποτέλεσμα: 0

Διπλότυπα ratings: 0
Διπλότυπα movies: 0


## Ερώτημα 1 — Δημιουργία στήλης έτους

Ξεκινάμε το πρώτο ερώτημα της εργασίας δημιουργώντας μια νέα στήλη `year` στο `ratings_df`.  
Η στήλη αυτή προκύπτει από τη μετατροπή του `timestamp` (Unix time σε δευτερόλεπτα) σε ημερομηνία και 
την εξαγωγή μόνο του έτους.  

In [28]:
# Δημιουργία νέας στήλης "year" από το timestamp
# Μετατροπή του Unix timestamp (σε δευτερόλεπτα) σε datetime και εξαγωγή μόνο του έτους
ratings_df["year"] = pd.to_datetime(ratings_df["timestamp"], unit="s").dt.year

In [29]:
ratings_df.head()

,userId,movieId,rating,timestamp,year
0,1,122,5.0,838985046,1996
1,1,185,5.0,838983525,1996
2,1,231,5.0,838983392,1996
3,1,292,5.0,838983421,1996
4,1,316,5.0,838983392,1996


In [30]:
print(f"Μικρότερο year: {ratings_df['year'].min()}")

Μικρότερο year: 1995


In [31]:
print(f"Μεγαλύτερο year: {ratings_df['year'].max()}")

Μεγαλύτερο year: 2009


### Συγχώνευση των DataFrames

Στο επόμενο βήμα ενώνουμε τα δύο DataFrames:
- `ratings_df` (αξιολογήσεις)  
- `movies_df` (πληροφορίες ταινιών)  

Η συγχώνευση γίνεται με βάση το `movieId`.  
Επιλέγουμε **left join** ώστε να διατηρηθούν όλες οι εγγραφές από το `ratings_df`, ακόμα κι αν κάποια ταινία δεν βρεθεί στο `movies_df`.

In [32]:
# Συγχώνευση των δύο DataFrames (ratings και movies) με βάση το movieId
# Χρησιμοποιούμε left join ώστε να κρατήσουμε όλες τις αξιολογήσεις,
# ακόμα κι αν κάποια ταινία δεν βρεθεί στο movies_df
df = ratings_df.merge(movies_df, on="movieId", how="left")

Σημείωση:  
Χρησιμοποιούμε το `ratings_df` ως βασικό πίνακα γιατί περιέχει τις αξιολογήσεις, οι οποίες είναι απαραίτητες για την εκπαίδευση του recommendation system.  
Το `movies_df` προστίθεται μόνο για συμπληρωματικές πληροφορίες (τίτλοι, είδη).  
Αν ξεκινούσαμε από το `movies_df`, θα κρατούσαμε και ταινίες χωρίς καμία αξιολόγηση, κάτι που δεν έχει πρακτική χρησιμότητα στην ανάλυση.

In [33]:
inspect_data(df)


Σχήμα DataFrame: (10000054, 7)

Πληροφορίες DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000054 entries, 0 to 10000053
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
 4   year       int32  
 5   title      object 
 6   genres     object 
dtypes: float64(1), int32(1), int64(3), object(2)
memory usage: 495.9+ MB

Πρώτες 5 γραμμές:
   userId  movieId  rating  timestamp  year                 title                        genres
0       1      122     5.0  838985046  1996      Boomerang (1992)                Comedy|Romance
1       1      185     5.0  838983525  1996       Net, The (1995)         Action|Crime|Thriller
2       1      231     5.0  838983392  1996  Dumb & Dumber (1994)                        Comedy
3       1      292     5.0  838983421  1996       Outbreak (1995)  Action|Drama|Sci-Fi|Thriller
4       1      316     5.0  838983392  19

### Εξερεύνηση του συγχωνευμένου DataFrame `df`

Μετά τη συγχώνευση `ratings_df` και `movies_df`:

- **Σχήμα:** 10.000.054 γραμμές και 7 στήλες  
- **Στήλες:** `userId`, `movieId`, `rating`, `timestamp`, `year`, `title`, `genres`  
- **Μνήμη:** ~496 MB  
- **Χρονικό εύρος:** από το 1995 έως το 2009  
- **Βαθμολογίες:** από 0.5 έως 5.0, με μέσο όρο ~3.51  
- **Έλεγχος NaN:** δεν υπάρχουν ελλιπείς τιμές 

In [34]:
# Αφαίρεση της στήλης "timestamp" επειδή δεν χρειάζεται πλέον,
# καθώς έχουμε ήδη εξάγει το έτος (year) από αυτήν
df = df.drop(columns=["timestamp"])

## Ερώτημα 2 — Ανάλυση κατά είδος και έτος

Από αυτό το σημείο ξεκινά η απάντηση στο δεύτερο ερώτημα της εργασίας.  
Στόχος: να βρούμε τα 5 κορυφαία είδη (`genres`) που είχαν τη μεγαλύτερη μείωση στη μέση ετήσια βαθμολογία από το πρώτο έτος που υπάρχουν δεδομένα μέχρι το τελευταίο.

Παρόλο που το σπάσιμο των genres δεν ήταν αυστηρά απαραίτητο — αφού θα μπορούσαμε να θεωρήσουμε κάθε συνδυασμό ειδών (π.χ. *Action|Comedy*) ως ένα ξεχωριστό «είδος» — προτίμησα να τα διαχωρίσω.  
Ο λόγος είναι ότι αυτός ο χειρισμός θα είναι πιο χρήσιμος στο επόμενο μέρος της εργασίας, όπου χρειάστηκε να δουλέψω μεμονωμένα πάνω σε κάθε είδος.

In [35]:
df_simple = df.copy()

### Δημιουργία λίστας ειδών (genres)

Προσθέτουμε μια νέα στήλη `genres_list` όπου τα είδη κάθε ταινίας 
χωρίζονται σε λίστα, με βάση τον διαχωριστή `"|"`.  
Έτσι μπορούμε πιο εύκολα να αναλύσουμε τα δεδομένα ανά είδος.

In [36]:
df_simple["genres_list"] = df_simple["genres"].fillna("").apply(lambda x: x.split("|"))
df_simple.head()

,userId,movieId,rating,year,title,genres,genres_list
0,1,122,5.0,1996,Boomerang (1992),Comedy|Romance,"[Comedy, Romance]"
1,1,185,5.0,1996,"Net, The (1995)",Action|Crime|Thriller,"[Action, Crime, Thriller]"
2,1,231,5.0,1996,Dumb & Dumber (1994),Comedy,[Comedy]
3,1,292,5.0,1996,Outbreak (1995),Action|Drama|Sci-Fi|Thriller,"[Action, Drama, Sci-Fi, Thriller]"
4,1,316,5.0,1996,Stargate (1994),Action|Adventure|Sci-Fi,"[Action, Adventure, Sci-Fi]"


### Ανάπτυξη των ειδών (explode)

Χρησιμοποιούμε τη μέθοδο `explode` στη στήλη `genres_list` έτσι ώστε κάθε είδος 
να εμφανίζεται σε ξεχωριστή γραμμή.  
Με αυτό τον τρόπο, μια ταινία που ανήκει σε πολλά είδη συνδέεται με κάθε είδος ξεχωριστά.

In [37]:
df_genres = df_simple.explode("genres_list")

In [38]:
df_genres.head()

,userId,movieId,rating,year,title,genres,genres_list
0,1,122,5.0,1996,Boomerang (1992),Comedy|Romance,Comedy
0,1,122,5.0,1996,Boomerang (1992),Comedy|Romance,Romance
1,1,185,5.0,1996,"Net, The (1995)",Action|Crime|Thriller,Action
1,1,185,5.0,1996,"Net, The (1995)",Action|Crime|Thriller,Crime
1,1,185,5.0,1996,"Net, The (1995)",Action|Crime|Thriller,Thriller


In [39]:
print("Γραμμές πριν:", len(df), "— μετά το explode:", len(df_genres))

Γραμμές πριν: 10000054 — μετά το explode: 25967194


In [40]:
# Εμφάνιση όλων των μοναδικών ειδών (genres) που υπάρχουν στο DataFrame
# Χρησιμοποιούμε sorted() για ταξινόμηση αλφαβητικά
print("Μοναδικά genres:", sorted(df_genres["genres_list"].unique()))

Μοναδικά genres: ['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


### Καθαρισμός λίστας ειδών

Στα μοναδικά είδη που προέκυψαν εμφανίζονται και οι κατηγορίες:
- `(no genres listed)`  
- `IMAX`  

Οι κατηγορίες αυτές δεν μας ενδιαφέρουν για την ανάλυση, οπότε θα τις αφαιρέσουμε.

In [41]:
# Δημιουργία μάσκας για τις εγγραφές με είδος "(no genres listed)"
mask = df_genres["genres_list"] == "(no genres listed)"

# Έλεγχος πλήθους ταινιών χωρίς δηλωμένο είδος
print("Πλήθος χωρίς είδος:", mask.sum())

# Εμφάνιση παραδειγμάτων από αυτές τις εγγραφές
print(df_genres[mask].head())

Πλήθος χωρίς είδος: 7
         userId  movieId  rating  year                 title              genres         genres_list
1025054    7701     8606     5.0  2007  Pull My Daisy (1958)  (no genres listed)  (no genres listed)
1453344   10680     8606     4.5  2007  Pull My Daisy (1958)  (no genres listed)  (no genres listed)
4066834   29097     8606     2.0  2004  Pull My Daisy (1958)  (no genres listed)  (no genres listed)
6456905   46142     8606     3.5  2008  Pull My Daisy (1958)  (no genres listed)  (no genres listed)
8046610   57696     8606     4.5  2008  Pull My Daisy (1958)  (no genres listed)  (no genres listed)


In [42]:
# Αφαίρεση εγγραφών χωρίς είδος ("(no genres listed)")
df_genres = df_genres[df_genres["genres_list"] != "(no genres listed)"]

# Αφαίρεση εγγραφών με είδος "IMAX", καθώς δεν μας ενδιαφέρει στην ανάλυση
df_genres = df_genres[df_genres["genres_list"] != "IMAX"]

In [43]:
print("Μοναδικά genres:", sorted(df_genres["genres_list"].unique()))

Μοναδικά genres: ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


### Υπολογισμός μέσης βαθμολογίας ανά είδος και έτος

Ομαδοποιούμε τα δεδομένα με βάση τα πεδία `genres_list` και `year`  
και υπολογίζουμε τη μέση τιμή της βαθμολογίας (`rating`).  
Το αποτέλεσμα είναι ένας πίνακας με μέση βαθμολογία για κάθε είδος σε κάθε έτος.

In [44]:
mean_per_year = df_genres.groupby(["genres_list","year"])["rating"].mean().reset_index()
mean_per_year.head()

,genres_list,year,rating
0,Action,1995,3.000000
1,Action,1996,3.440509
2,Action,1997,3.539915
3,Action,1998,3.440092
4,Action,1999,3.447994


### Εύρεση πρώτου και τελευταίου έτους ανά είδος

Χρησιμοποιούμε τον πίνακα `mean_per_year` για να εντοπίσουμε:  
- το πρώτο έτος (`first_year`) στο οποίο υπάρχει βαθμολογία για κάθε είδος,  
- το τελευταίο έτος (`last_year`) στο οποίο υπάρχει βαθμολογία για κάθε είδος.  
Αυτές οι πληροφορίες θα χρειαστούν για να υπολογίσουμε τη μεταβολή στη μέση βαθμολογία.

In [45]:
# 1) Έχουμε: mean_per_year με στήλες ['genres_list','year','rating']

g = mean_per_year.copy()

# 2) Πρώτο και τελευταίο έτος ανά genre
first_year = g.groupby("genres_list")["year"].min().rename("first_year").reset_index()
last_year  = g.groupby("genres_list")["year"].max().rename("last_year").reset_index()

In [46]:
first_year.head()

,genres_list,first_year
0,Action,1995
1,Adventure,1996
2,Animation,1996
3,Children,1996
4,Comedy,1995


In [47]:
last_year.head()

,genres_list,last_year
0,Action,2009
1,Adventure,2009
2,Animation,2009
3,Children,2009
4,Comedy,2009


### Μέση βαθμολογία στο πρώτο έτος ανά είδος

Με βάση τον πίνακα `first_year` κρατάμε μόνο τις γραμμές που αντιστοιχούν στο πρώτο έτος κάθε είδους,  
ώστε να έχουμε τη μέση βαθμολογία στην αρχή της χρονοσειράς.

In [48]:
# 3) Εύρεση μέσης βαθμολογίας στο πρώτο έτος για κάθε είδος
# Κάνουμε merge με τον πίνακα first_year ώστε να γνωρίζουμε το πρώτο έτος κάθε είδους
g_first = g.merge(first_year, on=["genres_list"], how="inner")

# Φιλτράρουμε ώστε να κρατήσουμε μόνο τις γραμμές όπου το έτος είναι το πρώτο έτος του κάθε είδους
g_first = g_first[g_first["year"] == g_first["first_year"]]

In [49]:
g_first.head()

,genres_list,year,rating,first_year
0,Action,1995,3.000000,1995
15,Adventure,1996,3.516534,1996
29,Animation,1996,3.688559,1996
43,Children,1996,3.531240,1996
57,Comedy,1995,3.000000,1995


### Διαμόρφωση πίνακα πρώτου έτους

Κρατάμε μόνο τις στήλες που χρειαζόμαστε:  
- `genres_list` (είδος)  
- `first_year` (πρώτο έτος)  
- `rating` (μέση βαθμολογία στο πρώτο έτος)  

Μετονομάζουμε τη στήλη `rating` σε `mean_first` ώστε να είναι πιο περιγραφική.

In [50]:
g_first = g_first[["genres_list", "first_year", "rating"]].rename(columns={"rating": "mean_first"})

In [51]:
g_first.head()

,genres_list,first_year,mean_first
0,Action,1995,3.000000
15,Adventure,1996,3.516534
29,Animation,1996,3.688559
43,Children,1996,3.531240
57,Comedy,1995,3.000000


### Διαμόρφωση πίνακα τελευταίου έτους

Με παρόμοιο τρόπο βρίσκουμε τη μέση βαθμολογία για το τελευταίο έτος κάθε είδους:  
1. Κάνουμε merge με τον πίνακα `last_year`.  
2. Φιλτράρουμε ώστε να κρατήσουμε μόνο τις γραμμές που αντιστοιχούν στο τελευταίο έτος κάθε είδους.  
3. Κρατάμε μόνο τις στήλες `genres_list`, `last_year`, `rating`.  
4. Μετονομάζουμε τη στήλη `rating` σε `mean_last` για σαφήνεια.

In [52]:
g_last = g.merge(last_year, on=["genres_list"], how="inner")
g_last = g_last[g_last["year"] == g_last["last_year"]]
g_last = g_last[["genres_list","last_year","rating"]].rename(columns={"rating":"mean_last"})
g_last.head()

,genres_list,last_year,mean_last
14,Action,2009,3.434257
28,Adventure,2009,3.481291
42,Animation,2009,3.572374
56,Children,2009,3.383498
71,Comedy,2009,3.324105


### Υπολογισμός μεταβολής μέσης βαθμολογίας

Συνδυάζουμε τους δύο πίνακες (`g_first` και `g_last`) με βάση το `genres_list`  
και δημιουργούμε μια νέα στήλη `delta`, η οποία υπολογίζεται ως:

Η μεταβολή: **delta = mean_first − mean_last**.

Δηλαδή, η μείωση (ή αύξηση) της μέσης βαθμολογίας από το πρώτο στο τελευταίο έτος για κάθε είδος.

In [53]:
results = g_first.merge(g_last, on="genres_list", how="inner").assign(delta=lambda x: x["mean_first"]-x["mean_last"])
results.head()

,genres_list,first_year,mean_first,last_year,mean_last,delta
0,Action,1995,3.000000,2009,3.434257,-0.434257
1,Adventure,1996,3.516534,2009,3.481291,0.035244
2,Animation,1996,3.688559,2009,3.572374,0.116184
3,Children,1996,3.531240,2009,3.383498,0.147742
4,Comedy,1995,3.000000,2009,3.324105,-0.324105


### Τα 5 είδη με τη μεγαλύτερη μείωση

Ταξινομούμε τα αποτελέσματα με βάση τη στήλη `delta` σε φθίνουσα σειρά  
και κρατάμε τα 5 πρώτα είδη, δηλαδή αυτά που παρουσίασαν τη μεγαλύτερη μείωση 
στη μέση βαθμολογία από το πρώτο στο τελευταίο έτος.

In [54]:
worst5 = results.sort_values("delta", ascending=False).head(5)
print(worst5)

   genres_list  first_year  mean_first  last_year  mean_last     delta
10      Horror        1995    5.000000       2009   3.310458  1.689542
15    Thriller        1995    5.000000       2009   3.472470  1.527530
12     Mystery        1995    5.000000       2009   3.659091  1.340909
5        Crime        1995    4.000000       2009   3.619338  0.380662
16         War        1996    3.943008       2009   3.723529  0.219478


## Ερώτημα 3 — Προσαρμογή ως προς τον αριθμό αξιολογήσεων

Στο τρίτο ερώτημα εξετάζουμε κατά πόσο η μείωση της μέσης βαθμολογίας ανά είδος 
επηρεάζεται από το πλήθος των αξιολογήσεων.  
Θα λάβουμε υπόψη τον αριθμό των ratings για να αποφύγουμε παραπλανητικά αποτελέσματα 

### Πλήθος αξιολογήσεων ανά είδος και έτος

Ομαδοποιούμε τα δεδομένα με βάση τα πεδία `genres_list` και `year`  
και υπολογίζουμε τον αριθμό των αξιολογήσεων (`count`) για κάθε συνδυασμό.  
Αυτό μας επιτρέπει να δούμε πόσο αξιόπιστη είναι η μέση βαθμολογία κάθε είδους σε κάθε έτος

In [55]:
count_per_year = (
    df_genres.groupby(["genres_list","year"])
    .size()
    .reset_index(name="count")
)
print(count_per_year.sort_values("count").head(30))

     genres_list  year  count
0         Action  1995      1
216     Thriller  1995      1
173      Mystery  1995      1
144       Horror  1995      1
101        Drama  1995      1
72         Crime  1995      2
57        Comedy  1995      2
143    Film-Noir  2009    155
100  Documentary  2009    195
258      Western  2009    220
172      Musical  2009    680
244          War  2009    765
187      Mystery  2009    990
42     Animation  2009   1133
89   Documentary  1998   1220
158       Horror  2009   1224
56      Children  2009   1515
129      Fantasy  2009   1902
215       Sci-Fi  2009   2289
86         Crime  2009   2296
88   Documentary  1997   2750
201      Romance  2009   2759
132    Film-Noir  1998   3182
28     Adventure  2009   3394
130    Film-Noir  1996   3446
230     Thriller  2009   3814
247      Western  1998   3993
14        Action  2009   4221
131    Film-Noir  1997   4620
87   Documentary  1996   4736


### Παρατήρηση αποτελεσμάτων πλήθους

Βλέπουμε ότι σε ορισμένα είδη και έτη το πλήθος αξιολογήσεων είναι **πολύ μικρό**  
(π.χ. `Action` το 1995 με μόνο 1 αξιολόγηση).  
Αυτό σημαίνει ότι η μέση βαθμολογία σε αυτές τις περιπτώσεις δεν είναι αξιόπιστη.  

Αντίθετα, σε πιο πρόσφατα έτη (π.χ. το 2009) υπάρχουν χιλιάδες αξιολογήσεις ανά είδος,  
οπότε η μέση βαθμολογία είναι πολύ πιο αντιπροσωπευτική.  

Γι’ αυτό θα χρειαστεί να λάβουμε υπόψη τον αριθμό αξιολογήσεων  
ώστε να έχουμε πιο έγκυρα συμπεράσματα για τη μεταβολή της δημοτικότητας.

### Ορισμός κατωφλιού αξιολογήσεων

Θέτουμε ένα κατώφλι `N = 1500` ώστε να κρατήσουμε μόνο τις περιπτώσεις 
(είδος–έτος) όπου υπάρχουν τουλάχιστον 1500 αξιολογήσεις.  
Με αυτόν τον τρόπο αποφεύγουμε να βασίσουμε τα συμπεράσματα σε πολύ μικρά δείγματα

In [56]:
# Ορίζουμε κατώφλι πλήθους αξιολογήσεων
N = 1500

# Κρατάμε μόνο τα ζεύγη (είδος–έτος) που έχουν τουλάχιστον N αξιολογήσεις
valid_years = count_per_year[count_per_year["count"] >= N]

# Φιλτράρουμε το αρχικό df_genres ώστε να περιλαμβάνει μόνο τα έτη/είδη
# που πέρασαν το κατώφλι αξιολογήσεων
df_valid = df_genres.merge(valid_years[["genres_list", "year"]], on=["genres_list", "year"])

# Εμφάνιση των πρώτων γραμμών για έλεγχο
df_valid.head()

,userId,movieId,rating,year,title,genres,genres_list
0,1,122,5.0,1996,Boomerang (1992),Comedy|Romance,Comedy
1,1,122,5.0,1996,Boomerang (1992),Comedy|Romance,Romance
2,1,185,5.0,1996,"Net, The (1995)",Action|Crime|Thriller,Action
3,1,185,5.0,1996,"Net, The (1995)",Action|Crime|Thriller,Crime
4,1,185,5.0,1996,"Net, The (1995)",Action|Crime|Thriller,Thriller


In [57]:
# Υπολογισμός μέσης βαθμολογίας ανά είδος και έτος (μετά το φιλτράρισμα ως προς το κατώφλι N)
mean_per_year_2 = df_valid.groupby(["genres_list", "year"])["rating"].mean().reset_index()

# Αντιγραφή του DataFrame για συνέχιση επεξεργασίας
g_2 = mean_per_year_2.copy()

# Εντοπισμός πρώτου έτους αξιολογήσεων για κάθε είδος
first_year_2 = (
    g_2.groupby("genres_list")["year"]
    .min()
    .rename("first_year")
    .reset_index()
)

# Εντοπισμός τελευταίου έτους αξιολογήσεων για κάθε είδος
last_year_2 = (
    g_2.groupby("genres_list")["year"]
    .max()
    .rename("last_year")
    .reset_index()
)

# Εμφάνιση των πρώτων 30 γραμμών του πίνακα first_year_2 για έλεγχο
first_year_2.head(30)

,genres_list,first_year
0,Action,1996
1,Adventure,1996
2,Animation,1996
3,Children,1996
4,Comedy,1996
5,Crime,1996
6,Documentary,1996
7,Drama,1996
8,Fantasy,1996
9,Film-Noir,1996


### Εφαρμογή ίδιων βημάτων με το Ερώτημα 2

Ακολουθούμε την ίδια διαδικασία όπως και πριν:  
- Εντοπίζουμε το πρώτο και τελευταίο έτος για κάθε είδος.  
- Υπολογίζουμε τη μέση βαθμολογία στο πρώτο (`mean_first`) και στο τελευταίο έτος (`mean_last`).  

Η διαφορά είναι ότι τώρα δουλεύουμε μόνο με τα φιλτραρισμένα δεδομένα (`df_valid`),  
δηλαδή με είδη–έτη που έχουν τουλάχιστον 1500 αξιολογήσεις.

In [58]:
# 3) Εύρεση μέσης βαθμολογίας στο πρώτο έτος για κάθε είδος
g_first_2 = g_2.merge(first_year_2, on=["genres_list"], how="inner")
# Κρατάμε μόνο τις γραμμές όπου το έτος αντιστοιχεί στο πρώτο έτος του είδους
g_first_2 = g_first_2[g_first_2["year"] == g_first_2["first_year"]]
# Κρατάμε τις απαραίτητες στήλες και μετονομάζουμε τη rating σε mean_first
g_first_2 = g_first_2[["genres_list", "first_year", "rating"]].rename(columns={"rating": "mean_first"})

# Αντίστοιχα, εύρεση μέσης βαθμολογίας στο τελευταίο έτος για κάθε είδος
g_last_2 = g_2.merge(last_year_2, on=["genres_list"], how="inner")
# Κρατάμε μόνο τις γραμμές όπου το έτος αντιστοιχεί στο τελευταίο έτος του είδους
g_last_2 = g_last_2[g_last_2["year"] == g_last_2["last_year"]]
# Κρατάμε τις απαραίτητες στήλες και μετονομάζουμε τη rating σε mean_last
g_last_2 = g_last_2[["genres_list", "last_year", "rating"]].rename(columns={"rating": "mean_last"})

# Εμφάνιση των πρώτων γραμμών του g_last_2 για έλεγχο
g_last_2.head()

,genres_list,last_year,mean_last
13,Action,2009,3.434257
27,Adventure,2009,3.481291
40,Animation,2008,3.587039
54,Children,2009,3.383498
68,Comedy,2009,3.324105


### Υπολογισμός μεταβολής μετά το φιλτράρισμα

Συνδυάζουμε τους πίνακες `g_first_2` και `g_last_2` και υπολογίζουμε ξανά τη μεταβολή 
της μέσης βαθμολογίας (`delta = mean_first − mean_last`) για κάθε είδος.

In [59]:
results_2 = g_first_2.merge(g_last_2, on="genres_list", how="inner").assign(delta=lambda x: x["mean_first"]-x["mean_last"])
results_2.head()

,genres_list,first_year,mean_first,last_year,mean_last,delta
0,Action,1996,3.440509,2009,3.434257,0.006252
1,Adventure,1996,3.516534,2009,3.481291,0.035244
2,Animation,1996,3.688559,2008,3.587039,0.101520
3,Children,1996,3.531240,2009,3.383498,0.147742
4,Comedy,1996,3.434176,2009,3.324105,0.110071


In [60]:
worst5_2 = results_2.sort_values("delta", ascending=False).head(5)
print(worst5_2)

    genres_list  first_year  mean_first  last_year  mean_last     delta
10       Horror        1996    3.599110       2008   3.348849  0.250261
16          War        1996    3.943008       2008   3.737926  0.205081
6   Documentary        1996    3.867188       2008   3.693026  0.174161
13      Romance        1996    3.617932       2009   3.443820  0.174112
3      Children        1996    3.531240       2009   3.383498  0.147742


### Σύγκριση αποτελεσμάτων πριν και μετά το φιλτράρισμα

Σε σχέση με τα αρχικά αποτελέσματα, η εικόνα αλλάζει:  
μετά την εφαρμογή κατωφλιού αξιολογήσεων (≥1500), βλέπουμε διαφορετικά είδη να εμφανίζονται στη λίστα με τη μεγαλύτερη μείωση.  

Αυτό δείχνει ότι ο αριθμός των αξιολογήσεων επηρεάζει τα συμπεράσματα και ότι η ανάλυση γίνεται πιο αξιόπιστη όταν αγνοούμε είδη–έτη με πολύ λίγα δεδομένα.

## Ερώτημα 4 — Εκπαίδευση Recommendation System

Στο τέταρτο ερώτημα θα περάσουμε από την ανάλυση δεδομένων στην εκπαίδευση ενός 
μοντέλου συστάσεων.  

### Στόχοι
1. Διαχωρισμός του dataset σε:
   - **Train set**: αξιολογήσεις πριν από το 2008  
   - **Test set**: αξιολογήσεις για τα έτη 2008 και 2009  
2. Εκπαίδευση recommendation system με τη χρήση **gradient descent**.  
3. Εκτίμηση βαθμολογιών για το test set και υπολογισμός του **μέσου τετραγωνικού σφάλματος (RMSE)**.  

### Πρόσθετες βελτιώσεις (εκτός των ζητούμενων)
- Δημιουργήθηκε μια συνάρτηση **early_stopping** η οποία:  
  - Αποθηκεύει τα καλύτερα βάρη κατά την εκπαίδευση.  
  - Σταματάει πρόωρα την εκπαίδευση αν για αρκετές εποχές δεν υπάρχει πρόοδος ή η βελτίωση είναι πολύ μικρή.  
  - Επαναφέρει τα καλύτερα βάρη που έχουν αποθηκευτεί.  
  - Ο έλεγχος γίνεται με χρήση ενός μικρού **validation set**, στο οποίο περιλαμβάνονται μόνο χρήστες και ταινίες που υπάρχουν και στο train set (ώστε να μην παρουσιαστούν προβλήματα με άγνωστους χρήστες ή ταινίες.).  

- Η συνάρτηση **train** περνάει σε κάθε εποχή **όλο το train set** αντί να επιλέγει τυχαία κομμάτια (mini-batches).  
  Έτσι διασφαλίζεται ότι σε κάθε εποχή όλα τα δεδομένα θα χρησιμοποιηθούν για εκπαίδευση, 
  επιτρέποντας στο μοντέλο να αξιοποιήσει πλήρως το dataset και να ανταποκριθεί καλύτερα.

### Διαχωρισμός σε Train, Validation και Test sets

Διαχωρίζουμε τα δεδομένα σε τρία σύνολα:
- **Train**: αξιολογήσεις πριν από το 2008 (χρησιμοποιείται για εκπαίδευση).  
- **Validation**: μικρό υποσύνολο (10%) από το Train, για έλεγχο βελτίωσης και early stopping.  
- **Test**: αξιολογήσεις των ετών 2008–2009, για τελική αξιολόγηση του μοντέλου.  

Η συνάρτηση `sample(frac=...)` επιτρέπει να πάρουμε ένα ποσοστό τυχαίων γραμμών από το DataFrame,  
ενώ το `drop` αφαιρεί τις ίδιες γραμμές για να φτιάξουμε το validation set.

In [61]:
# Κάνουμε ένα αντίγραφο του DataFrame για ασφάλεια
df_2 = df.copy()

# Train set: όλες οι αξιολογήσεις πριν το 2008
train = df_2[df["year"] < 2008]

# Test set: όλες οι αξιολογήσεις για το 2008 και μετά
test = df_2[df["year"] >= 2008]

# Ποσοστό validation σε σχέση με το train
val_frac = 0.1

# Δημιουργία training subset (train_fit): παίρνουμε το 90% των δεδομένων του train
# Η παράμετρος frac ορίζει το ποσοστό, το random_state εξασφαλίζει επαναληψιμότητα
train_fit = train.sample(frac=1 - val_frac, random_state=42)

# Validation subset (val_raw): τα υπόλοιπα 10% των δεδομένων που έμειναν έξω
val_raw = train.drop(train_fit.index)

# Γρήγορη περίληψη: μέγεθος και εύρος ετών για κάθε σύνολο
print(f"Train size: {len(train_fit):,} | years: {int(train_fit['year'].min())}–{int(train_fit['year'].max())}")
print(f"val size: {len(val_raw):,} | years: {int(val_raw['year'].min())}–{int(val_raw['year'].max())}")
print(f"Test  size: {len(test):,}  | years: {int(test['year'].min())}–{int(test['year'].max())}")

Train size: 8,290,646 | years: 1995–2007
val size: 921,183 | years: 1995–2007
Test  size: 788,225  | years: 2008–2009


In [62]:
#Οπτικός έλεγχος 2-3 γραμμών
display(train_fit.sample(3, random_state=0))
display(val_raw.sample(3, random_state=0))
display(test.sample(3, random_state=0))

,userId,movieId,rating,year,title,genres
4349757,31027,3359,4.0,2004,Breaking Away (1979),Comedy|Drama
1693563,12401,2550,3.0,1999,"Haunting, The (1963)",Horror
6197138,44266,1275,3.0,1996,Highlander (1986),Action|Adventure|Fantasy


,userId,movieId,rating,year,title,genres
9375955,67193,2628,3.0,2005,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi
5732897,40945,500,4.0,1996,Mrs. Doubtfire (1993),Comedy|Drama
5141687,36771,4369,4.0,2001,"Fast and the Furious, The (2001)",Action|Crime|Thriller


,userId,movieId,rating,year,title,genres
9000155,64480,24,3.0,2008,Powder (1995),Drama|Sci-Fi
5392353,38517,6377,3.5,2008,Finding Nemo (2003),Adventure|Animation|Children|Comedy
6242652,44603,3615,3.5,2008,Dinosaur (2000),Animation|Children


### Δημιουργία χαρτών για χρήστες και ταινίες

Για να εκπαιδεύσουμε το recommendation model, χρειάζεται να μετατρέψουμε τα αναγνωριστικά 
χρηστών και ταινιών (`userId`, `movieId`) σε συνεχόμενους ακέραιους δείκτες.  

Βήματα:  
1. Με τη συνάρτηση `make_maps` δημιουργούμε δύο λεξικά για κάθε πεδίο:  
   - `map`: από το πραγματικό ID → σε νέο δείκτη (0, 1, 2, …).  
   - `imap`: από τον δείκτη → πίσω στο πραγματικό ID.  
2. Χρησιμοποιούμε τη μέθοδο `map` της pandas για να προσθέσουμε στο DataFrame δύο νέες στήλες:  
   - `UserIdx`: τον δείκτη του χρήστη.  
   - `MovieIdx`: τον δείκτη της ταινίας.  
3. Έτσι τα δεδομένα γίνονται πιο εύχρηστα για αριθμητικούς αλγορίθμους και για την αναπαράσταση με πίνακες.

In [63]:
# Δημιουργία λεξικών για mapping από userId σε συνεχόμενο ακέραιο δείκτη
# Επιστρέφονται δύο λεξικά: user_map (ID -> δείκτης), user_imap (δείκτης -> ID)
user_map,  user_imap  = make_maps(train_fit, "userId")

# Δημιουργία λεξικών για mapping από movieId σε συνεχόμενο ακέραιο δείκτη
movie_map, movie_imap = make_maps(train_fit, "movieId")

# Δημιουργούμε ένα αντίγραφο του train_fit για ασφάλεια
train_fit = train_fit.copy()

# Προσθέτουμε στο DataFrame μια νέα στήλη με τους δείκτες των χρηστών
train_fit['UserIdx']  = train_fit['userId'].map(user_map)

# Προσθέτουμε στο DataFrame μια νέα στήλη με τους δείκτες των ταινιών
train_fit['MovieIdx'] = train_fit['movieId'].map(movie_map)

# Εμφάνιση των πρώτων γραμμών για έλεγχο
train_fit.head()

,userId,movieId,rating,year,title,genres,UserIdx,MovieIdx
9381785,67242,1380,4.0,2001,Grease (1978),Comedy|Musical|Romance,0,0
1323219,9705,5630,3.5,2005,Red Dragon (2002),Crime|Mystery|Thriller,1,1
3276287,23599,316,3.0,1996,Stargate (1994),Action|Adventure|Sci-Fi,2,2
5728956,40924,858,4.0,1997,"Godfather, The (1972)",Crime|Drama,3,3
9157875,65685,6567,2.0,2005,Buffalo Soldiers (2001),Comedy|Crime|Drama|War,4,4


### Προετοιμασία validation set

Εφαρμόζουμε τους ίδιους χάρτες (`user_map`, `movie_map`) και στο validation set (`val_raw`),  
ώστε να χρησιμοποιεί τους ίδιους συνεχόμενους δείκτες για χρήστες και ταινίες.  
Με αυτό τον τρόπο διασφαλίζουμε ότι οι δείκτες είναι συνεπείς μεταξύ train και validation.

In [64]:
# Δημιουργούμε αντίγραφο του validation set για να μην τροποποιήσουμε το αρχικό
val = val_raw.copy()

# Χαρτογράφηση των userId σε συνεχόμενους δείκτες με χρήση του user_map
val['UserIdx'] = val['userId'].map(user_map)

# Χαρτογράφηση των movieId σε συνεχόμενους δείκτες με χρήση του movie_map
val['MovieIdx'] = val['movieId'].map(movie_map)

In [65]:
val.head()

,userId,movieId,rating,year,title,genres,UserIdx,MovieIdx
4,1,316,5.0,1996,Stargate (1994),Action|Adventure|Sci-Fi,20738,2.0
21,1,616,5.0,1996,"Aristocats, The (1970)",Animation|Children,20738,427.0
35,2,858,2.0,1997,"Godfather, The (1972)",Crime|Drama,59593,3.0
37,2,1073,3.0,1997,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical,59593,776.0
45,3,590,3.5,2006,Dances with Wolves (1990),Adventure|Drama|Western,48357,273.0


### Χωρισμός validation set σε γνωστά και άγνωστα IDs

Στο validation set μπορεί να υπάρχουν χρήστες ή ταινίες που δεν εμφανίζονται στο train set.  
Για να αποφύγουμε σφάλματα κατά την εκπαίδευση/αξιολόγηση:  
- Κρατάμε μόνο τις εγγραφές με γνωστούς δείκτες (`val_known`).  
- Αποθηκεύουμε ξεχωριστά τις εγγραφές με άγνωστους δείκτες (`val_oov`) για πιθανή μελλοντική χρήση.

In [66]:
# Μάσκα που ελέγχει ποιες γραμμές έχουν γνωστούς δείκτες χρήστη και ταινίας (δηλαδή δεν είναι NaN)
val_mask_known = val['UserIdx'].notna() & val['MovieIdx'].notna()

# Validation set με γνωστούς χρήστες/ταινίες (χρησιμοποιείται στον έλεγχο)
val_known = val[val_mask_known].copy()

# Validation set με άγνωστους χρήστες/ταινίες (εκτός εκπαίδευσης/ελέγχου)
val_oov = val[~val_mask_known].copy()

### Προετοιμασία test set

Όπως και με το validation set, εφαρμόζουμε τους χάρτες `user_map` και `movie_map` στο test set.  
Στη συνέχεια το χωρίζουμε σε δύο μέρη:  
- `test_known`: εγγραφές με χρήστες και ταινίες που υπάρχουν ήδη στο train set.  
- `test_oov`: εγγραφές με άγνωστους χρήστες ή ταινίες (out-of-vocabulary), που δεν μπορούν να αξιολογηθούν από το μοντέλο.

In [67]:
# Δημιουργούμε αντίγραφο του test set
test = test.copy()

# Χαρτογράφηση userId σε συνεχόμενους δείκτες
test['UserIdx'] = test['userId'].map(user_map)

# Χαρτογράφηση movieId σε συνεχόμενους δείκτες
test['MovieIdx'] = test['movieId'].map(movie_map)

# Μάσκα για γραμμές με γνωστούς χρήστες και ταινίες
mask_known = test['UserIdx'].notna() & test['MovieIdx'].notna()

# Test set με γνωστούς χρήστες και ταινίες (χρησιμοποιείται για αξιολόγηση)
test_known = test[mask_known].copy()

# Test set με άγνωστους χρήστες ή ταινίες (δεν μπορεί να χρησιμοποιηθεί για αξιολόγηση)
test_oov = test[~mask_known].copy()

In [68]:
n_users = len(user_map)
n_items = len(movie_map)

# Πλήθος μοναδικών χρηστών και ταινιών στο train set
print(f"Users in train_fit (unique): {n_users:,}")
print(f"Movies in train_fit (unique): {n_items:,}")

# Validation set: σύνολο και κατανομή σε γνωστά και OOV
print(f"Validation rows total:   {len(val):,}")
print(f"  Known IDs:             {len(val_known):,}")
print(f"  OOV (cold-start):      {len(val_oov):,}")

# Test set: σύνολο και κατανομή σε γνωστά και OOV
print(f"Test rows total:         {len(test):,}")
print(f"  Known IDs:             {len(test_known):,}")
print(f"  OOV (cold-start):      {len(test_oov):,}")

Users in train_fit (unique): 65,514
Movies in train_fit (unique): 9,701
Validation rows total:   921,183
  Known IDs:             921,182
  OOV (cold-start):      1
Test rows total:         788,225
  Known IDs:             140,728
  OOV (cold-start):      647,497


### Μετατροπή δεδομένων σε NumPy πίνακες

Για να περάσουμε τα δεδομένα στο recommendation μοντέλο, χρειάζεται να τα έχουμε 
σε αριθμητική μορφή (`NumPy arrays`) αντί για DataFrame.  

Βήματα:  
- `u`: πίνακας με τα indices των χρηστών (`UserIdx`).  
- `i`: πίνακας με τα indices των ταινιών (`MovieIdx`).  
- `r`: πίνακας με τις βαθμολογίες (`rating`).  

Αυτή η μορφή είναι πιο αποδοτική υπολογιστικά και επιτρέπει να ταΐσουμε απευθείας 
τα δεδομένα στον αλγόριθμο εκπαίδευσης (π.χ. gradient descent).

In [69]:
# Παίρνουμε τα indices των χρηστών σε numpy array (ακέραιοι)
u = train_fit['UserIdx'].to_numpy(int)

# Παίρνουμε τα indices των ταινιών σε numpy array (ακέραιοι)
i = train_fit['MovieIdx'].to_numpy(int)

# Παίρνουμε τις βαθμολογίες σε numpy array (float)
r = train_fit['rating'].to_numpy(float)

### Δημιουργία COO sparse matrix

Από τα τριπλέτα `(UserIdx, MovieIdx, rating)` φτιάχνουμε έναν **COO sparse πίνακα**:  

- **Sparse matrix** σημαίνει αραιός πίνακας: αποθηκεύει μόνο τις μη μηδενικές τιμές (εδώ τις βαθμολογίες),  
  χωρίς να κρατά όλα τα υπόλοιπα κελιά που θα ήταν μηδέν.  
- Η αναπαράσταση **COO (Coordinate format)** αποθηκεύει τις τιμές μαζί με τις συντεταγμένες `(row, col)`.  
- Είναι ιδανική μορφή για datasets με πολλές κενές τιμές, όπως οι πίνακες χρηστών–ταινιών,  
  όπου το μεγαλύτερο μέρος των κελιών δεν έχει καμία βαθμολογία.  

Χρειαζόμαστε αυτή τη μορφή για αποδοτική αποθήκευση και γρήγορη αριθμητική επεξεργασία κατά την εκπαίδευση.

In [70]:
# Φτιάχνουμε έναν COO sparse matrix από τα τριπλέτα (u, i, r)
# u = χρήστης (row index), i = ταινία (col index), r = rating (data)
# Το shape είναι (n_users, n_items), δηλαδή πίνακας με όλους τους χρήστες και όλες τις ταινίες
R_coo = coo_matrix((r, (u, i)), shape=(n_users, n_items))

### Στατιστικά σπανιότητας πίνακα

Ο πίνακας χρηστών–ταινιών είναι εξαιρετικά αραιός:  
- Υπολογίζουμε πόσα ratings υπάρχουν (μη μηδενικά στοιχεία).  
- Συγκρίνουμε αυτό τον αριθμό με το σύνολο των πιθανών κελιών (users × items).  
- Ο δείκτης **sparsity** δείχνει το ποσοστό των άδειων κελιών.  

Αυτό τονίζει γιατί είναι σημαντικό να χρησιμοποιούμε sparse αναπαραστάσεις αντί για κανονικούς πίνακες.

In [71]:
# Πλήθος μη μηδενικών στοιχείων = συνολικός αριθμός ratings
num_nonzero = R_coo.nnz             

# Συνολικός αριθμός κελιών στον πίνακα (users × items)
total_cells = n_users * n_items

# Υπολογισμός ποσοστού σπανιότητας = τι ποσοστό κελιών είναι άδεια
sparsity = 1 - (num_nonzero / total_cells)

# Εκτύπωση στατιστικών
print(f"Train ratings (non-zeros): {num_nonzero:,}")
print(f"Matrix shape: {n_users:,} users × {n_items:,} items")
print(f"Sparsity: {sparsity:.6f} (δηλ. {100*sparsity:.4f}% κελιά άδεια)")

Train ratings (non-zeros): 8,290,646
Matrix shape: 65,514 users × 9,701 items
Sparsity: 0.986955 (δηλ. 98.6955% κελιά άδεια)


### Εκπαίδευση μοντέλου με SGD (Matrix Factorization)

Καλούμε τη συνάρτηση `train_mf_sgd` για να εκπαιδεύσουμε το recommendation system.  

Σημειώσεις για τις υπερπαραμέτρους:  
- **lr (learning rate)**: ρυθμός εκμάθησης. Όσο μεγαλύτερος, τόσο πιο γρήγορη η εκπαίδευση αλλά με κίνδυνο αστάθειας.  
- **reg (regularization)**: συντελεστής κανονικοποίησης που βοηθά να αποφευχθεί το overfitting.  
- **min_delta**: το ελάχιστο κατώφλι βελτίωσης στο validation set για να θεωρηθεί ότι υπάρχει πρόοδος.  
- **patience**: πόσες εποχές περιμένουμε χωρίς ουσιαστική βελτίωση πριν σταματήσει το early stopping.  

Οι τιμές που χρησιμοποιήθηκαν εδώ (π.χ. `lr=0.03`, `reg=0.03`, `min_delta=8e-4`) δεν είναι απαραίτητα οι βέλτιστες,  
αλλά επελέγησαν ώστε η εκπαίδευση να είναι πιο γρήγορη και πρακτική μέσα στα χρονικά όρια της άσκησης.

In [72]:
P, Q = train_mf_sgd(
    R_coo,
    n_users=n_users,
    n_items=n_items,
    K=40,
    epochs=15,
    batch_size=300_000,
    lr=0.03,
    reg=0.03,
    seed=42,
    verbose=True,
    val_known=val_known,
    patience=3,
    min_delta=8e-4
)

Epoch 1/15 — 28 batches των ~300,000 ratings
[val] epoch 1 RMSE=0.8904
Epoch 2/15 — 28 batches των ~300,000 ratings
[val] epoch 2 RMSE=0.8559
Epoch 3/15 — 28 batches των ~300,000 ratings
[val] epoch 3 RMSE=0.8423
Epoch 4/15 — 28 batches των ~300,000 ratings
[val] epoch 4 RMSE=0.8341
Epoch 5/15 — 28 batches των ~300,000 ratings
[val] epoch 5 RMSE=0.8288
Epoch 6/15 — 28 batches των ~300,000 ratings
[val] epoch 6 RMSE=0.8252
Epoch 7/15 — 28 batches των ~300,000 ratings
[val] epoch 7 RMSE=0.8243
Epoch 8/15 — 28 batches των ~300,000 ratings
[val] epoch 8 RMSE=0.8230
Epoch 9/15 — 28 batches των ~300,000 ratings
[val] epoch 9 RMSE=0.8213
Epoch 10/15 — 28 batches των ~300,000 ratings
[val] epoch 10 RMSE=0.8221
Epoch 11/15 — 28 batches των ~300,000 ratings
[val] epoch 11 RMSE=0.8217
Epoch 12/15 — 28 batches των ~300,000 ratings
[val] epoch 12 RMSE=0.8207
Early stopping triggered.
Restored best weights from epoch 9 (best val rmse: 0.8213).


### Αποτελέσματα εκπαίδευσης & early stopping

Η εκπαίδευση σταμάτησε πρόωρα με **early stopping**.  
- Το καλύτερο αποτέλεσμα παρατηρήθηκε στο **epoch 9** με RMSE = 0.8213.  
- Παρόλο που στο epoch 12 το RMSE βελτιώθηκε ελαφρώς (0.8207), η βελτίωση δεν ξεπέρασε το κατώφλι `min_delta=8e-4`.  
- Για τον λόγο αυτό το early stopping δεν το θεώρησε σημαντική πρόοδο και επανέφερε τα βάρη από το epoch 9.  

Συνεπώς, το μοντέλο αξιολογείται με τα βάρη που αντιστοιχούν στο **epoch 9**, το οποίο θεωρείται το καλύτερο σύμφωνα με τα κριτήρια που ορίσαμε.

### Αξιολόγηση στο test set

Χρησιμοποιούμε το `test_known` (δηλαδή μόνο εγγραφές με γνωστούς χρήστες και ταινίες) για να αξιολογήσουμε το μοντέλο.  

Βήματα:  
1. Εξάγουμε τους δείκτες χρηστών, ταινιών και τις πραγματικές βαθμολογίες.  
2. Υπολογίζουμε τις προβλέψεις με εσωτερικό γινόμενο (dot product) των πινάκων χρηστών–ταινιών `P` και `Q`.  
3. Υπολογίζουμε τα σφάλματα:  
   - **MSE (Mean Squared Error)** = μέσο τετραγωνικό σφάλμα.  
   - **RMSE (Root Mean Squared Error)** = τετραγωνική ρίζα του MSE.

In [73]:
# 1) Παίρνουμε indices χρηστών/ταινιών και τις πραγματικές βαθμολογίες από το test_known
u_test = test_known['UserIdx'].to_numpy(int)
i_test = test_known['MovieIdx'].to_numpy(int)
y_test = test_known['rating'].to_numpy(float)

# 2) Υπολογίζουμε προβλέψεις: εσωτερικό γινόμενο (dot product) για κάθε (χρήστης, ταινία)
yhat = (P[u_test] * Q[i_test]).sum(axis=1)

# 3) Υπολογίζουμε MSE και RMSE για τις προβλέψεις
mse_known  = ((y_test - yhat) ** 2).mean()
rmse_known = mse_known ** 0.5

# Εκτύπωση αποτελεσμάτων με μέγεθος δείγματος και σφάλματα
print(f"Test_known: N={len(y_test):,} | MSE={mse_known:.4f} | RMSE={rmse_known:.4f}")

Test_known: N=140,728 | MSE=0.6683 | RMSE=0.8175


### Αποτελέσματα στο test set

Στο σύνολο **test_known** (δηλαδή μόνο με χρήστες και ταινίες που υπήρχαν στο train set) 
πήραμε τα εξής αποτελέσματα:  

- **Πλήθος αξιολογήσεων:** 140,728  
- **MSE:** 0.6683  
- **RMSE:** 0.8175  

Η τιμή του RMSE είναι σχετικά χαμηλή, γεγονός που δείχνει ότι το μοντέλο 
καταφέρνει να προσεγγίσει αρκετά καλά τις πραγματικές βαθμολογίες, 
αν και υπάρχει ακόμη περιθώριο βελτίωσης με πιο προσεκτικό tuning (learning rate, regularization, bias terms).

## Ερώτημα 5 — Προτάσεις για νέο χρήστη

Υποθέτουμε ότι πρέπει να προτείνουμε **10 ταινίες** σε έναν νέο χρήστη χωρίς καμία προηγούμενη αξιολόγηση (cold start).

Το πρόβλημα με τον νέο χρήστη είναι ότι δεν έχουμε πληροφορίες για τις προτιμήσεις του, άρα δεν μπορούμε να του προτείνουμε άμεσα κάτι «προσωπικό».  
Υπάρχουν όμως διάφορες στρατηγικές για να αντιμετωπιστεί το "ξεκίνημα" ενός νέου χρήστη, όπως:

- Να προτείνουμε ταινίες που είναι αυτή την περίοδο **trending**.  
- Να προτείνουμε ταινίες που θεωρούνται **κλασικές** και παραμένουν διαχρονικά δημοφιλείς.  
- Να αξιοποιήσουμε στοιχεία όπως **εθνικότητα/τοποθεσία** (από όπου γίνεται login) ώστε να προτείνουμε ταινίες που είναι δημοφιλείς στην περιοχή αυτή.  
- Να ζητήσουμε από τον χρήστη να επιλέξει **αγαπημένα είδη** και να του προτείνουμε τις πιο δημοφιλείς ταινίες αυτών των κατηγοριών.  
- Να ζητήσουμε από τον χρήστη να δηλώσει μερικές **αγαπημένες ταινίες** ώστε να δημιουργηθεί ένα αρχικό προφίλ (π.χ. θεωρώντας ότι τις βαθμολογεί πάνω από τον μέσο όρο), και έτσι να αρχίσει ο αλγόριθμος να παράγει πιο στοχευμένες προτάσεις.  
- Ένας **συνδυασμός** από τις παραπάνω προσεγγίσεις. 

## Ερώτημα 6 — Προτάσεις με βάση αγαπημένες ταινίες νέου χρήστη

Στο τελευταίο ερώτημα υποθέτουμε ότι ο νέος χρήστης μας έδωσε 3 αγαπημένες ταινίες:  
**"Iron Man"**, **"300"** και **"Transformers"**.  

Στόχος είναι να προτείνουμε 10 ταινίες που πιθανόν να του αρέσουν, εξηγώντας το σκεπτικό μας.

Υπάρχουν διάφορες τεχνικές που μπορούμε να ακολουθήσουμε για να δημιουργήσουμε προτάσεις με βάση το δείγμα που μας έδωσε:

---

#### 1) Collaborative filtering
Μπορούμε να χρησιμοποιήσουμε τα διανύσματα **Q** των τριών αγαπημένων ταινιών.  

- Μία προσέγγιση είναι να υπολογίσουμε την **cosine similarity** κάθε μίας από τις τρεις ταινίες με όλες τις υπόλοιπες και να προτείνουμε τις κορυφαίες ταινίες από κάθε λίστα.  
- Εναλλακτικά, μπορούμε να πάρουμε τον **μέσο όρο των τριών διανυσμάτων** (δημιουργώντας έτσι ένα «προφίλ χρήστη») και να υπολογίσουμε την cosine similarity αυτού του μέσου διανύσματος με όλα τα υπόλοιπα. Οι δέκα ταινίες με τη μεγαλύτερη ομοιότητα μπορούν να προταθούν.

---

#### 2) Content-based
Εδώ χρησιμοποιούμε **metadata** (π.χ. genres, σκηνοθέτης, σεναριογράφος, περιγραφή).  

- Με ένα one-hot encoding των genres (και πιθανόν άλλων χαρακτηριστικών) φτιάχνουμε διανύσματα περιεχομένου.  
- Υπολογίζουμε την cosine similarity μεταξύ των διανυσμάτων των αγαπημένων ταινιών και των υπόλοιπων.  
- Οι ταινίες με τα υψηλότερα σκορ θεωρούνται πιο κοντινές στο γούστο που εκφράζουν οι επιλογές του χρήστη.

---

#### 3) Υβριδικό approach (συνδυασμός collaborative + content)
Συνδυάζουμε τα δύο προηγούμενα:  

- Υπολογίζουμε collaborative similarity (από τα embeddings Q).  
- Υπολογίζουμε content-based similarity (από τα genres/μεταδεδομένα).  
- Κανονικοποιούμε τα δύο σκορ στην ίδια κλίμακα και τα συνδυάζουμε (π.χ. με μέσο όρο ή βάρη).  
- Οι ταινίες με τα υψηλότερα τελικά σκορ προτείνονται στον χρήστη.

---

Με τον τρόπο αυτό μπορούμε να προτείνουμε 10 ταινίες που σχετίζονται με τις αρχικές επιλογές του χρήστη, είτε βάσει ομοιότητας στα «κρυφά χαρακτηριστικά» του μοντέλου, είτε βάσει περιεχομένου, είτε με συνδυασμό των δύο.

## Επέκταση με χρήση περιγραφών ταινιών (context embeddings)

Στη συνέχεια θα προχωρήσω σε επιπλέον βήματα ώστε να ενσωματώσω περισσότερο context στις προτάσεις:  

1. **Λήψη περιγραφών (overviews):**  
   - Θα κατεβάσω τα δεδομένα από το **The Movie Database (TMDB)**, ώστε να έχω πρόσβαση στις περιγραφές των ταινιών.  
   - Στο αρχείο **MovieLens 10M** δεν υπάρχουν αυτά τα IDs, οπότε θα χρησιμοποιήσω το μεγαλύτερο αρχείο (MovieLens 33M) για να κατεβάσω το αντίστοιχο `links` αρχείο, το οποίο περιέχει τα TMDB IDs.  

2. **Δημιουργία embeddings:**  
   - Για τις περιγραφές θα δημιουργήσω embeddings χρησιμοποιώντας ένα ελαφρύ sentence-BERT μοντέλο.  
   - Έτσι θα έχουμε για κάθε ταινία ένα διάνυσμα που εκφράζει το νόημα (context) της περιγραφής της.  

3. **Συνδυασμός με collaborative filtering:**  
   - Όπως και πριν, θα δουλέψω με τους πίνακες χρηστών και ταινιών (P και Q).  
   - Επιπλέον, θα δημιουργήσω έναν πίνακα βαρών **W**, ο οποίος θα περιέχει τα embeddings από τις περιγραφές των ταινιών.  
   - Με αυτόν τον τρόπο, το μοντέλο θα αξιοποιεί και πληροφορία περιεχομένου, πράγμα που βοηθά ειδικά σε περιπτώσεις ταινιών που δεν υπάρχουν στο training set.  

4. **Αντιμετώπιση ταινιών χωρίς overview:**  
   - Σε ταινίες που δεν διαθέτουν περιγραφή, θα χρησιμοποιηθούν τα **genres**.  
   - Για κάθε genre θα υπολογιστεί ένα embedding (μέσος όρος των διανυσμάτων ταινιών με overview).  
   - Αν μια ταινία δεν έχει overview, θα πάρει ως embedding τον μέσο όρο των embeddings των genres στα οποία ανήκει.  

### Σημείωση
- Με αυτό τον τρόπο μπορούμε να δώσουμε προτάσεις ακόμη και για ταινίες που είναι **άγνωστες στο training set** (cold start για items).  
- Ωστόσο, για νέους χρήστες (χωρίς καμία αξιολόγηση), το πρόβλημα παραμένει και δεν μπορούμε να κάνουμε αξιόπιστη αξιολόγηση.

### Κατέβασμα MovieLens-latest dataset

Επειδή στο MovieLens 10M δεν υπάρχουν TMDB IDs, κατεβάζουμε το 
**MovieLens-latest** dataset, το οποίο περιλαμβάνει το αρχείο `links.csv` με 
τα αντίστοιχα αναγνωριστικά ταινιών στο TMDB.  
Αυτό θα μας επιτρέψει να συνδέσουμε τις ταινίες με τα δεδομένα από το TMDB 
(π.χ. περιγραφές/overviews).

In [74]:
# URL του MovieLens-latest dataset
url = "https://files.grouplens.org/datasets/movielens/ml-latest.zip"

# Λήψη και αποσυμπίεση του dataset στον φάκελο target_dir
dataset_dir = download_movielens(
    url,
    target_dir=r"C:\Users\giorg\Desktop\dataset_ml",
    zip_name="ml-latest.zip"
)

Το αρχείο zip υπάρχει ήδη
Αποσυμπίεση στο C:\Users\giorg\Desktop\dataset_ml...
Ok!!


In [75]:
print(os.listdir(dataset_dir))

['ml-10m.zip', 'ml-10M100K', 'ml-latest', 'ml-latest.zip']


### Φόρτωση του αρχείου links.csv

Από το MovieLens-latest dataset διαβάζουμε το αρχείο `links.csv`.  
Αυτό περιέχει για κάθε `movieId` τα αντίστοιχα IDs από το **IMDb** και το **TMDB**,  
τα οποία θα χρειαστούμε για να συνδέσουμε τις ταινίες με εξωτερικά δεδομένα (π.χ. περιγραφές από το TMDB).

In [76]:
# Διαβάζουμε το αρχείο links.csv που περιέχει τις αντιστοιχίσεις movieId → imdbId / tmdbId
links_df, _ = load_data(
    source='file',
    filepath=r"C:\Users\giorg\Desktop\dataset_ml\ml-latest/links.csv"
)


Dataset φορτώθηκε από CSV αρχείο: (86537, 3)


In [77]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### Συγχώνευση των ταινιών με τα εξωτερικά IDs

Κάνουμε merge του `movies_df` (πληροφορίες ταινιών από MovieLens) με το `links_df` 
(που περιέχει τα **IMDb** και **TMDB IDs**).  

Με αυτόν τον τρόπο, για κάθε ταινία έχουμε επιπλέον τα IDs που χρειάζονται για να συνδέσουμε 
το dataset με εξωτερικές πηγές (π.χ. The Movie Database).

In [78]:
# Ενώνουμε τα δεδομένα ταινιών με τα links ώστε να αποκτήσουμε imdbId και tmdbId
# Χρησιμοποιούμε validate='one_to_one' για να βεβαιωθούμε ότι κάθε movieId ταιριάζει σε μία εγγραφή
movies_links = movies_df.merge(links_df, on='movieId', how='left', validate='one_to_one')
movies_links.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709.0,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497.0,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228.0,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885.0,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041.0,11862.0


### Σημαντική σημείωση για τα IDs

Κατά τη φόρτωση παρατηρούμε ότι οι στήλες `imdbId` και `tmdbId` εμφανίζονται ως **δεκαδικοί αριθμοί (float)**.  
Αυτό συμβαίνει επειδή κάποιες εγγραφές έχουν **NaN τιμές**, και η pandas μετατρέπει αυτόματα όλη τη στήλη σε `float64`.  

Για να διατηρήσουμε τα IDs ως ακέραιους αλλά να επιτρέπουμε και NaN,  
μετατρέπουμε τις στήλες σε τύπο **Int64** (nullable integer type της pandas).

In [79]:
movies_links['imdbId'] = movies_links['imdbId'].astype('Int64')
movies_links['tmdbId'] = movies_links['tmdbId'].astype('Int64')

In [80]:
movies_links.head(1)

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862


In [81]:
# Συνολικός αριθμός ταινιών
total = len(movies_links)

# Πλήθος ταινιών χωρίς imdbId
missing_imdb = movies_links['imdbId'].isna().sum()

# Πλήθος ταινιών χωρίς tmdbId
missing_tmdb = movies_links['tmdbId'].isna().sum()

# Εκτύπωση αποτελεσμάτων με ποσοστά
print(f"Σύνολο ταινιών: {total}")
print(f"Missing imdbId: {missing_imdb}  ({missing_imdb/total:.2%})")
print(f"Missing tmdbId: {missing_tmdb}  ({missing_tmdb/total:.2%})")

Σύνολο ταινιών: 10681
Missing imdbId: 93  (0.87%)
Missing tmdbId: 118  (1.10%)


### Προετοιμασία πίνακα ταινιών για TMDB

Διατηρούμε μόνο τα απαραίτητα πεδία (`movieId`, `title`, `genres`, `tmdbId`)  
και δημιουργούμε μία βοηθητική στήλη `has_tmdb` που δείχνει αν υπάρχει διαθέσιμο `tmdbId`.  
Αυτό θα μας χρειαστεί για να ξέρουμε ποιες ταινίες μπορούν να συνδεθούν με δεδομένα από το TMDB.

In [82]:
movies_links = movies_links.copy()

# Κρατάμε μόνο τις βασικές στήλες που μας ενδιαφέρουν
movies_links = movies_links[['movieId', 'title', 'genres', 'tmdbId']]

# Προσθέτουμε flag που δείχνει αν η ταινία έχει διαθέσιμο tmdbId
movies_links['has_tmdb'] = movies_links['tmdbId'].notna()

# Εμφάνιση κατανομής: πόσες ταινίες έχουν / δεν έχουν tmdbId
print(movies_links['has_tmdb'].value_counts())

# Εμφάνιση πρώτων γραμμών για έλεγχο
movies_links.head()

has_tmdb
True     10563
False      118
Name: count, dtype: int64


,movieId,title,genres,tmdbId,has_tmdb
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862,True
1,2,Jumanji (1995),Adventure|Children|Fantasy,8844,True
2,3,Grumpier Old Men (1995),Comedy|Romance,15602,True
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,31357,True
4,5,Father of the Bride Part II (1995),Comedy,11862,True


### Δοκιμαστικό αίτημα στο TMDB API

Για να ελέγξουμε ότι η σύνδεση με το **The Movie Database (TMDB)** λειτουργεί σωστά:  
- Παίρνουμε ένα διαθέσιμο `tmdbId` από το `movies_links`.  
- Κάνουμε κλήση στο endpoint `/movie/{tmdbId}` με το API key.  
- Επιστρέφουμε τον τίτλο και την αρχή της περίληψης (overview) της ταινίας.

In [83]:
# Ανάκτηση του API key από το περιβάλλον
API_KEY = os.getenv("TMDB_API_KEY")

# Επιλογή ενός διαθέσιμου tmdbId (παίρνουμε το πρώτο που βρήκαμε)
tmdb_id = int(movies_links.loc[movies_links['has_tmdb'], 'tmdbId'].iloc[0])

# Δημιουργία URL για το API call
url = f"https://api.themoviedb.org/3/movie/{tmdb_id}"
params = {"api_key": API_KEY, "language": "en"}

# Εκτέλεση του HTTP request
r = requests.get(url, params=params, timeout=10)

# Έλεγχος αποτελέσματος
print("HTTP", r.status_code)

# Αν είναι επιτυχές, εκτύπωση τίτλου και των πρώτων 200 χαρακτήρων από το overview
data = r.json() if r.ok else {}
print("TMDB title:", data.get("title"))
print("Overview:", (data.get("overview") or "")[:200], "…")

HTTP 200
TMDB title: Toy Story
Overview: Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstan …


### Δοκιμαστική ανάκτηση περιγραφών από το TMDB

Για να επιβεβαιώσουμε ότι μπορούμε να τραβήξουμε σωστά δεδομένα από το TMDB API,  
παίρνουμε ένα τυχαίο δείγμα 3 ταινιών που έχουν διαθέσιμο `tmdbId`.  

Για κάθε ταινία ζητάμε:  
- τον τίτλο που έχουμε τοπικά (από MovieLens),  
- τον τίτλο όπως επιστρέφεται από το TMDB,  
- και τα πρώτα 200 χαρακτήρες της περίληψης (overview). 

In [84]:
# Επιλέγουμε τυχαία 3 ταινίες με διαθέσιμο tmdbId
sample = movies_links.loc[movies_links['has_tmdb'], ['movieId','title','tmdbId']].sample(3, random_state=42)

rows = []
for _, row in sample.iterrows():
    tmdb_id = int(row.tmdbId)
    url = f"https://api.themoviedb.org/3/movie/{tmdb_id}"
    params = {"api_key": API_KEY, "language": "en"}
    
    # Κλήση στο TMDB API
    resp = requests.get(url, params=params, timeout=10)
    d = resp.json() if resp.ok else {}
    
    # Αποθήκευση αποτελεσμάτων για σύγκριση
    rows.append({
        "movieId": row.movieId,
        "local_title": row.title,               # τίτλος από MovieLens
        "tmdb_title": d.get("title"),           # τίτλος από TMDB
        "overview_200": (d.get("overview") or "")[:200]  # οι πρώτοι 200 χαρακτήρες του overview
    })

# Δημιουργία DataFrame με τα αποτελέσματα
pd.DataFrame(rows)

,movieId,local_title,tmdb_title,overview_200
0,3395,Nadine (1987),Nadine,"Hairdresser Nadine Hightower wants to retrieve the risqué photos she once posed for, but when she visits the photographer at his office, he's murdered by an intruder. Nadine talks her estranged husban"
1,5453,Lost in Yonkers (1993),Lost in Yonkers,"In the summer of 1942 two young boys are sent to stay with their stern grandmother Kurnitz and their childlike aunt Bella in Yonkers, New York."
2,4734,Jay and Silent Bob Strike Back (2001),Jay and Silent Bob Strike Back,"When Jay and Silent Bob learn that their comic-book alter egos, Bluntman and Chronic, have been sold to Hollywood as part of a big-screen movie that leaves them out of any royalties, the pair travels"


### Φιλτράρισμα ταινιών με διαθέσιμο TMDB ID

Κρατάμε μόνο τις ταινίες που έχουν τιμή στο `tmdbId`.  
Αυτές είναι οι ταινίες για τις οποίες μπορούμε να κατεβάσουμε περιγραφές (overviews) από το TMDB.

In [85]:
# Φτιάχνουμε ένα DataFrame μόνο με τις ταινίες που έχουν tmdbId
df_with_tmdb = movies_links[movies_links['has_tmdb']].copy()

# Εμφάνιση του συνολικού πλήθους
print("Σύνολο ταινιών με tmdbId:", len(df_with_tmdb))

Σύνολο ταινιών με tmdbId: 10563


### Δοκιμαστική λήψη περιγραφών για τις πρώτες ταινίες

Για έλεγχο, κατεβάζουμε από το TMDB τις περιγραφές (overviews) για τις 10 πρώτες ταινίες με διαθέσιμο `tmdbId`.  
Συγκρίνουμε τον τοπικό τίτλο (από το MovieLens) με τον τίτλο που επιστρέφει το TMDB και εμφανίζουμε ένα preview 120 χαρακτήρων της περίληψης.

In [86]:
results = []

# Παίρνουμε τις 10 πρώτες ταινίες με διαθέσιμο tmdbId
for _, row in df_with_tmdb.head(10).iterrows():
    tmdb_id = int(row["tmdbId"])
    local_title = row["title"]
    
    # Κλήση στο TMDB API
    url = f"https://api.themoviedb.org/3/movie/{tmdb_id}"
    params = {"api_key": API_KEY, "language": "en"}
    r = requests.get(url, params=params, timeout=10)
    data = r.json()
    
    # Αποθήκευση βασικών στοιχείων για έλεγχο
    results.append({
        "movieId": row['movieId'],
        "local_title": local_title,                   # τίτλος από MovieLens
        "tmdb_title": data.get("title"),              # τίτλος από TMDB
        "overview": (data.get("overview") or "")[:120] + "…"  # preview πρώτων 120 χαρακτήρων
    })

# Δημιουργία DataFrame με τα αποτελέσματα
pd.DataFrame(results)

,movieId,local_title,tmdb_title,overview
0,1,Toy Story (1995),Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of…"
1,2,Jumanji (1995),Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly i…"
2,3,Grumpier Old Men (1995),Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a s…"
3,4,Waiting to Exhale (1995),Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a…"
4,5,Father of the Bride Part II (1995),Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that …"
5,6,Heat (1995),Heat,Obsessive master thief Neil McCauley leads a top-notch crew on various daring heists throughout Los Angeles while determ…
6,7,Sabrina (1995),Sabrina,"Sabrina Fairchild, a chauffeur's daughter, grew up at the Long Island estate of the wealthy Larrabee family enchanted wi…"
7,8,Tom and Huck (1995),Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses a murder by the deadly Injun Joe. Tom becomes friends with Huckleberry Fi…"
8,9,Sudden Death (1995),Sudden Death,When a man's daughter is suddenly taken during a championship hockey game – with the captors demanding a billion dollars…
9,10,GoldenEye (1995),GoldenEye,"When a powerful satellite system falls into the hands of Alec Trevelyan, AKA Agent 006, a former ally-turned-enemy, only…"


### Προετοιμασία για μαζική λήψη δεδομένων από το TMDB

Επειδή το πλήθος των ταινιών είναι μεγάλο, οργανώνουμε τη λήψη σε **batches**.  
Με αυτόν τον τρόπο μπορούμε να ζητάμε π.χ. 500 ταινίες ανά γύρο, ώστε να μην επιβαρύνουμε το API.  
Δημιουργούμε μια λίστα `results` για να συγκεντρώσουμε τα δεδομένα που θα επιστραφούν (movieId, title, overview).

In [87]:
batch_size = 500               # πόσες ταινίες θα ζητάμε ανά "γύρο"

# Κρατάμε μόνο τις ταινίες με διαθέσιμο tmdbId
df_with_tmdb = movies_links[movies_links['has_tmdb']].copy()

# Λίστα για να αποθηκεύσουμε τα αποτελέσματα (movieId, title, overview)
results = []

# Συνολικός αριθμός ταινιών με tmdbId
n = len(df_with_tmdb)
print("Σύνολο:", n)

Σύνολο: 10563


### Μαζική λήψη περιγραφών από το TMDB API

Χωρίζουμε τις ταινίες σε batches (π.χ. 500 κάθε φορά) και ζητάμε από το TMDB API τις περιγραφές τους.  
Σε κάθε κλήση:  
- Χρησιμοποιούμε το `tmdbId` για το endpoint `/movie/{id}`.  
- Αν προκύψει σφάλμα ή η περιγραφή λείπει, αποθηκεύουμε `None`.  
- Προσθέτουμε μικρό χρονικό διάστημα `sleep` ώστε να μην ξεπερνάμε τα όρια χρήσης του API (rate limiting).  

Συγκεντρώνουμε όλα τα αποτελέσματα σε μια λίστα `results`, με πεδία `movieId`, `title`, `overview`.

In [88]:
for start in range(0, n, batch_size):
    end = min(start + batch_size, n)
    chunk = df_with_tmdb.iloc[start:end]

    print(f"Batch {start}-{end} / {n}")

    # Για κάθε ταινία στο batch ζητάμε overview από το TMDB
    for _, row in chunk.iterrows():
        tmdb_id = int(row['tmdbId'])
        url = f"https://api.themoviedb.org/3/movie/{tmdb_id}"
        params = {"api_key": API_KEY, "language": "en"}

        try:
            r = requests.get(url, params=params, timeout=10)
            
            # Έλεγχος για rate limit (HTTP 429)
            if r.status_code == 429:
                time.sleep(1.0)  # μικρή καθυστέρηση
                r = requests.get(url, params=params, timeout=10)

            if not r.ok:
                overview = None
            else:
                data = r.json()
                overview = (data.get("overview") or "").strip() or None

        except Exception:
            overview = None

        # Αποθήκευση αποτελεσμάτων
        results.append({
            "movieId": row['movieId'],
            "title": row['title'],
            "overview": overview
        })

        # Μικρό throttle για να μην κάνουμε υπερβολικά συχνά αιτήματα
        time.sleep(0.01)

Batch 0-500 / 10563
Batch 500-1000 / 10563
Batch 1000-1500 / 10563
Batch 1500-2000 / 10563
Batch 2000-2500 / 10563
Batch 2500-3000 / 10563
Batch 3000-3500 / 10563
Batch 3500-4000 / 10563
Batch 4000-4500 / 10563
Batch 4500-5000 / 10563
Batch 5000-5500 / 10563
Batch 5500-6000 / 10563
Batch 6000-6500 / 10563
Batch 6500-7000 / 10563
Batch 7000-7500 / 10563
Batch 7500-8000 / 10563
Batch 8000-8500 / 10563
Batch 8500-9000 / 10563
Batch 9000-9500 / 10563
Batch 9500-10000 / 10563
Batch 10000-10500 / 10563
Batch 10500-10563 / 10563


### Ενοποίηση περιγραφών με τις ταινίες

Μετατρέπουμε τα αποτελέσματα των TMDB κλήσεων σε DataFrame (`overviews_df`) και κρατάμε μόνο τα πεδία `movieId` και `overview`.  
Στη συνέχεια τα κάνουμε merge με τον πίνακα `movies_links`, ώστε κάθε ταινία να έχει την αντίστοιχη περίληψη (αν υπάρχει).  

Τέλος, εμφανίζουμε ένα μικρό report κάλυψης για να δούμε σε πόσες ταινίες βρέθηκε overview.

In [89]:
overviews_df = pd.DataFrame(results)  # columns: movieId, title, overview

# Κρατάμε μόνο movieId και overview για το merge
overviews_slim = overviews_df[['movieId', 'overview']].copy()

# Ενοποίηση με τον αρχικό πίνακα ταινιών
movies_with_overview = movies_links.merge(
    overviews_slim, on='movieId', how='left'
)

# Report κάλυψης: πόσες ταινίες έχουν overview
have_text = movies_with_overview['overview'].notna().sum()
print(f"Περιγραφές λήφθηκαν για: {have_text} / {len(movies_with_overview)} "
      f"({have_text/len(movies_with_overview):.2%})")

Περιγραφές λήφθηκαν για: 10548 / 10681 (98.75%)


### Δημιουργία Sentence Embeddings

Χρησιμοποιούμε το προεκπαιδευμένο μοντέλο **SentenceTransformer** με αρχιτεκτονική `"all-MiniLM-L6-v2"`.  

- Είναι μια ελαφριά εκδοχή του BERT, βελτιστοποιημένη για **sentence embeddings**.  
- Παράγει για κάθε κείμενο (π.χ. overview ταινίας) ένα διάνυσμα χαμηλών διαστάσεων που συλλαμβάνει το νοηματικό περιεχόμενο.  
- Επιλέγεται γιατί είναι **γρήγορο** και **αποδοτικό**, κατάλληλο για μεγάλα datasets όπως το MovieLens, χωρίς να απαιτεί πολύ μνήμη ή υπολογιστική ισχύ.

In [90]:
model = SentenceTransformer("all-MiniLM-L6-v2")

### Διαχωρισμός ταινιών με και χωρίς περιγραφή

Για το επόμενο βήμα διαχωρίζουμε τις ταινίες:  
- `df_text`: ταινίες που έχουν διαθέσιμο και μη κενό **overview** (θα χρησιμοποιηθούν για embeddings).  
- `df_no_text`: ταινίες χωρίς overview (θα τις καλύψουμε αργότερα με embeddings από τα genres).  

Δημιουργούμε επίσης μια λίστα `texts` με όλα τα διαθέσιμα overviews, έτοιμη για μετατροπή σε embeddings.

In [91]:
mw = movies_with_overview.copy()

# Μάσκα: ποιες ταινίες έχουν διαθέσιμο και μη κενό overview
has_text_mask = mw['overview'].notna() & (mw['overview'].str.strip() != "")

# Ταινίες με overview
df_text = mw.loc[has_text_mask, ['movieId','title','genres','overview']].copy()

# Ταινίες χωρίς overview
df_no_text = mw.loc[~has_text_mask, ['movieId','title','genres']].copy()

# Λίστα με όλα τα διαθέσιμα overviews
texts = df_text['overview'].tolist()

### Δημιουργία embeddings για τα overviews

1. Χρησιμοποιούμε το sentence transformer μοντέλο για να μετατρέψουμε κάθε περιγραφή (overview) σε ένα διάνυσμα.  
   - Χρησιμοποιούμε batch size 512 για καλύτερη απόδοση.  
   - Ορίζουμε `normalize_embeddings=True` ώστε όλα τα διανύσματα να έχουν μήκος 1 (διευκολύνει μετρήσεις ομοιότητας).  
2. Αποθηκεύουμε τα embeddings στο DataFrame `df_text`, ώστε κάθε ταινία με overview να έχει το δικό της διάνυσμα περιεχομένου.

In [92]:
# 2. Υπολογισμός embeddings για όλα τα overviews
emb = model.encode(
    texts,
    batch_size=512,            # μέγεθος batch για ταχύτερη επεξεργασία
    show_progress_bar=True,    # εμφάνιση progress bar
    convert_to_numpy=True,     # επιστροφή ως numpy array
    normalize_embeddings=True  # κανονικοποίηση διανυσμάτων σε μήκος 1
)

# 3. Επισύναψη των embeddings στο DataFrame
df_text['embedding'] = list(emb)

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [93]:
df_text.head(1)

,movieId,title,genres,overview,embedding
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.","[0.063439, 0.0010269372, 0.09321016, -0.014941696, -0.006400284, 0.015288949, 0.12308541, -0.030445578, -0.037180047, 0.021572258, 0.031388048, 0.0037151107, 0.013458384, 0.008040355, 0.10826041, 0.05762549, -0.035886385, -0.092610225, -0.038548756, -0.03142635, 0.020635925, -0.035120964, 0.07055052, 0.0060065556, -0.0683286, 0.08416882, 0.0018677422, 0.029441223, 0.01173142, 0.007776514, 0.09046826, 0.014040493, -0.0034546931, -0.049409162, -0.08245178, 0.031226631, 0.015459258, -0.007901657, 0.07996006, -0.008406835, -0.045771465, -0.0122955805, -0.023142451, -0.017147437, -0.025055723, -0.018830314, -0.017016739, -0.038838062, 0.104796745, 0.013047272, -0.011202145, -0.02865002, 0.076505914, 0.057782352, 0.07371096, 0.16180626, 0.042045593, -0.019829495, 0.05532267, -0.011506968, -0.03342005, 0.02732795, 0.07759873, -0.072111286, 0.11224795, -0.07189962, -0.027188513, 0.029545017, -0.017497934, -0.042690996, 0.030226028, 0.031031504, -0.04099454, 0.006254584, -0.06825058, 0.0009143695, -0.056303803, 0.009702807, -0.034710914, 0.0019177911, -0.06670405, -0.062479265, -0.01783279, -0.06552035, -0.06862566, 0.033947777, 0.035080537, 0.032017108, -0.10901731, 0.06644949, -0.118026234, -0.09431275, 0.050268345, 0.031968135, 0.012103935, -0.01870459, -0.014339802, -0.031548265, 0.006489694, 0.013806811, ...]"


### Καθαρισμός λίστας ειδών (genres)

Δημιουργούμε μια λίστα `genre_list` για κάθε ταινία, χωρίζοντας το πεδίο `genres` με βάση τον χαρακτήρα `"|"`.  
Στη συνέχεια φιλτράρουμε τη λίστα ώστε να αφαιρέσουμε ανεπιθύμητες τιμές όπως **"IMAX"** και **"(no genres listed)"**, 
διατηρώντας όμως τα υπόλοιπα είδη της ταινίας.  
Με αυτό τον τρόπο έχουμε καθαρότερες λίστες genres για μελλοντική χρήση (π.χ. δημιουργία embeddings).

In [94]:
# Διαχωρίζουμε το πεδίο genres σε λίστα ειδών
df_text['genre_list'] = df_text['genres'].fillna("").str.split('|')

# Καθαρίζουμε κάθε λίστα: αφαιρούμε κενά και genres που δεν μας ενδιαφέρουν
df_text["genre_list"] = df_text["genre_list"].apply(
    lambda lst: [
        g.strip()                            # αφαιρεί τυχόν whitespace
        for g in lst
        if g and g.strip().upper() not in ("IMAX", "(NO GENRES LISTED)")  # φιλτράρει IMAX και "no genres listed"
    ]
)

### Embeddings-πρωτότυπα ανά είδος (genre prototypes)

Στόχος: να δημιουργήσουμε ένα **αντιπροσωπευτικό embedding** για κάθε genre, υπολογίζοντας τον μέσο όρο των embeddings από όλες τις ταινίες που ανήκουν σε αυτό.  
Βήματα:
1. Κρατάμε μόνο ταινίες που διαθέτουν genres και embedding.
2. Κάνουμε `explode` στη λίστα `genre_list` ώστε κάθε γραμμή να αντιστοιχεί σε ένα ζεύγος *(genre, embedding)*.
3. Ομαδοποιούμε ανά `genre` και υπολογίζουμε τον **μέσο όρο** των embeddings.
4. Εφαρμόζουμε **L2 κανονικοποίηση** στο μέσο διάνυσμα, ώστε όλα τα prototypes να έχουν μήκος 1, κάτι που διευκολύνει μετρικές ομοιότητας όπως το cosine (Αν και τα embeddings είχαν κανονικοποιηθεί κατά τη δημιουργία τους, εφαρμόζω ξανά L2 normalisation μετά τον υπολογισμό του μέσου όρου, ώστε να διορθώσω τυχόν αλλοίωση του μήκους που μπορεί να προκύψει από την πράξη του μέσου όρου. Δεν )

In [95]:
# Προσθέτουμε μια boolean στήλη που δείχνει αν η ταινία έχει τουλάχιστον ένα έγκυρο genre
df_text['has_genres'] = df_text['genre_list'].apply(lambda lst: len(lst) > 0)

# Επιλογή μόνο των γραμμών με διαθέσιμα genres και embeddings.
# Κρατάμε τις στήλες genre_list και embedding.
df_g = df_text.loc[df_text['has_genres'], ['genre_list', 'embedding']].explode('genre_list', ignore_index=True)

# Μετονομασία της στήλης για ευκολία
df_g = df_g.rename(columns={'genre_list': 'genre'})

# Ομαδοποίηση ανά genre και υπολογισμός "πρωτοτύπου" ως L2-normalized μέσος όρος embeddings
genre_proto = df_g.groupby('genre')['embedding'].apply(mean_l2)

# Γρήγορη προεπισκόπηση των πρώτων πρωτοτύπων
print(genre_proto)

genre
Action                                              [-0.08927223, 0.06341432, -0.07151144, -0.0043131215, 0.030609867, 0.038231846, 0.10788084, -0.040016763, -0.017653897, 0.00757725, 0.07923681, 0.0019785392, 0.019658258, 0.018373586, -0.054096755, 0.0053332667, -0.0026807515, 0.009851804, -0.0455719, -0.009240636, -0.06524302, -0.026979225, 0.047240704, 0.009097543, -0.09632719, 0.0099304505, 0.03600075, -0.0002096136, -0.07599842, -0.057686277, 0.023495354, 0.0016617626, -0.038507205, 0.06116413, 0.00828976, 0.029952185, 0.06551451, 0.05533259, 0.030607926, -0.009179895, -0.0067557804, -0.04145838, 0.024924286, 0.0272461, -0.033342727, -0.08797171, -0.050812665, -0.035415642, 0.07328361, -0.08383871, -0.03796183, -0.025273321, 0.0031484629, -0.0029716047, 0.07605867, -0.028037319, 0.05626427, -0.023748048, 0.06495572, -0.021644272, 0.0335231, 0.05146062, -0.0062628924, 0.01768252, 0.063148595, -0.0460585, -0.008024243, 0.023440305, -0.007345293, 0.048905905, 0.046033837, -0.03

### Αντιμετώπιση ταινιών χωρίς overview

Για τις ταινίες που δεν έχουν περιγραφή (overview):  
1. Μετατρέπουμε το string `genres` σε λίστα και φιλτράρουμε τυχόν άκυρες τιμές όπως "IMAX" ή "(no genres listed)".  
2. Για κάθε λίστα genres καλούμε τη συνάρτηση `movie_proto_embedding`, η οποία υπολογίζει το embedding της ταινίας ως μέσο όρο των prototype vectors των genres.  

Έτσι εξασφαλίζουμε ότι όλες οι ταινίες χωρίς overview αποκτούν ένα embedding βασισμένο στα genres τους.

In [96]:
# Μετατρέπουμε το string "genres" σε λίστα από είδη
# Χρησιμοποιούμε split με τον χαρακτήρα "|" και αντικαθιστούμε τυχόν NaN με ""
df_no_text["genre_list"] = df_no_text["genres"].fillna("").str.split("|")

# Καθαρίζουμε κάθε λίστα genres:
# - αφαιρούμε τυχόν κενά γύρω από τα ονόματα
# - απορρίπτουμε genres που δεν μας ενδιαφέρουν (IMAX, no genres listed)
df_no_text["genre_list"] = df_no_text["genre_list"].apply(
    lambda lst: [
        g.strip()
        for g in lst
        if g and g.strip().upper() not in ("IMAX", "(NO GENRES LISTED)")
    ]
)

# Προεπισκόπηση πρώτων γραμμών για έλεγχο
df_no_text.head(2)


,movieId,title,genres,genre_list
288,291,Poison Ivy II (1996),Drama|Thriller,"[Drama, Thriller]"
541,545,Harem (1985),Drama,[Drama]


In [97]:
# Για κάθε ταινία χωρίς overview, δημιουργούμε ένα embedding
# χρησιμοποιώντας τη συνάρτηση movie_proto_embedding,
# η οποία παίρνει τον μέσο όρο των prototype embeddings των genres
df_no_text["embedding"] = df_no_text["genre_list"].apply(lambda genres: movie_proto_embedding(genres, genre_proto))

# Προεπισκόπηση των πρώτων γραμμών με τα νέα embeddings
df_no_text.head()

,movieId,title,genres,genre_list,embedding
288,291,Poison Ivy II (1996),Drama|Thriller,"[Drama, Thriller]","[-0.060856435, 0.042294204, -0.047518723, 0.020989243, 0.025522346, 0.05071921, 0.09508387, -0.04454737, 0.010107905, -0.015500768, 0.07068979, 0.008485043, -0.0032269103, 0.007561233, -0.048586525, 0.01095309, -0.023872782, -0.0013527198, -0.050810117, 0.028184665, -0.07737675, -0.037261717, 0.04331363, -0.0074762395, -0.054336175, 0.006997297, 0.06551643, -0.022801993, -0.074704655, -0.01622518, 0.030568346, 0.028113278, -0.038205694, 0.049372625, 0.032832637, 0.03189665, 0.053821895, 0.06961814, 0.034910686, 0.011219209, -0.017032009, -0.059806995, 0.016404469, 0.023106383, -0.06969745, -0.09260401, -0.03634089, -0.025232358, 0.053931404, -0.096672095, -0.08238161, -0.025447324, -0.012875139, -0.027661204, 0.05576996, 0.011534816, 0.06214302, 0.023074277, 0.032219082, -0.010969689, 0.04046005, 0.033472273, -0.012017847, 0.00686186, 0.07549232, -0.017085843, -0.012019522, 0.02488175, 0.004109277, 0.044889953, 0.037485335, -0.02776931, -0.06606767, -0.021288715, -0.032007333, -0.015004327, 0.016843138, -0.0362227, 0.029749287, -0.04029969, 0.021729404, -0.10352865, -0.031728353, 0.02275658, -0.0037617956, -0.023807991, -0.0123867, -0.08316467, 0.06064778, 0.03823794, -0.12369598, -0.079981364, 0.022624992, 0.0017854513, -0.018173812, 0.04140984, -0.025299612, 0.045611758, -0.15391634, 0.11602236, ...]"
541,545,Harem (1985),Drama,[Drama],"[-0.050507367, 0.043687683, -0.0381084, 0.029767592, 0.008306021, 0.06353342, 0.09301296, -0.046095572, -2.776007e-05, -0.027732667, 0.06465954, -0.003470726, -0.005830365, 0.0069828094, -0.035011113, 0.025930986, -0.03308654, -0.006522107, -0.049385887, 0.026247546, -0.085951105, -0.043087326, 0.038489487, 0.0059987116, -0.042978767, 0.0056958064, 0.065445095, -0.016643364, -0.068466984, -0.0037081456, 0.029583143, 0.04189667, -0.034337252, 0.048100304, 0.020780401, 0.04682649, 0.046700485, 0.077955544, 0.03134698, 0.010402007, -0.013032308, -0.0610535, 0.0145748025, 0.0042668884, -0.059491, -0.09299168, -0.021888597, -0.039758194, 0.04979842, -0.07582235, -0.081105925, -0.024647158, -0.004546708, -0.042810153, 0.068128504, 0.037409335, 0.060377195, 0.021695575, 0.037701283, -0.0032927294, 0.027994437, 0.02775328, -0.015158543, 0.005450006, 0.06275428, -0.031092828, -0.010883811, 0.04528124, -0.020758243, 0.04225026, 0.031548742, -0.01933947, -0.07150819, -0.017911848, -0.030516393, -0.020054616, 0.010188074, -0.033440817, 0.025287047, -0.049383946, 0.004738382, -0.09550366, -0.02835114, 0.013610487, -0.023580452, -0.037457973, 0.0028248467, -0.10102096, 0.058457445, 0.036141895, -0.1361724, -0.07620043, 0.011379908, 0.0052205184, -0.022075776, 0.031814367, -0.020820588, 0.033820454, -0.15462583, 0.121855974, ...]"
598,604,Criminals (1996),Documentary,[Documentary],"[-0.041842278, 0.0350647, -0.049132958, 0.011209802, 0.059724517, 0.102008246, 0.047505032, -0.033161122, -0.003819847, -0.033939067, 0.009947301, 0.012471529, 0.0022954128, 0.029115891, -0.06618668, 0.023260714, 0.0014955917, -0.016497014, 0.0039986647, -0.02712488, -0.020685662, 0.0060559465, 0.06992258, 0.05165844, -0.041965667, 0.01004123, 0.017360784, 0.021750582, -0.06265193, 0.0013838343, 0.004718599, 0.0912046, -0.03162902, 0.0068474854, 0.03768067, 0.06455708, 0.05438624, 0.037669647, -0.029328724, 0.012631214, -0.0045509343, -0.06435496, 0.01702324, -0.03401611, -0.040206768, -0.070185296, -0.016096553, -0.061270416, 0.026345372, 0.011884865, -0.06484639, -0.02558234, 0.04587883, -0.11359529, 0.040146768, -0.055775646, 0.0061168983, -0.0048076385, 0.062883995, -0.039922416, 0.034403104, -0.054666605, -0.018775044, 0.03913615, 0.09883086, 0.004599052, -0.017615803, 0.049207404, -0.028853238, -0.03133672, -0.008951171, 0.0020586925, -0.02730508, -0.035795856, -0.016383464, -0.07661383, -0.008882272, -0.019656606, -0.03784541, -0.08635407, 0.13113299, -0.1259048, -0.009453137, -0.028576115, -0.024959145, -0.025366211

### Ενοποίηση embeddings ταινιών με και χωρίς overview

Σε αυτό το βήμα συνδυάζουμε τα δύο διαφορετικά είδη embeddings:  
- **`emb_text_df`**: ταινίες που έχουν overview → τα embeddings προέρχονται από το sentence-BERT.  
- **`emb_no_text_df`**: ταινίες χωρίς overview → τα embeddings προέρχονται από τα genre prototypes.  

Τα ενώνουμε σε ένα ενιαίο DataFrame `all_emb`, το οποίο περιέχει για κάθε ταινία το embedding της.

In [98]:
# Ξεκινάμε το DataFrame για τις ταινίες χωρίς overview, με τα πεδία που χρειαζόμαστε
emb_no_text_df = df_no_text[["movieId", "title", "genre_list", "embedding"]].copy()

# Αντίστοιχο DataFrame για τις ταινίες με overview (text embeddings)
emb_text_df = df_text[["movieId", "title", "genre_list", "embedding"]].copy()

# Ενώνουμε τα δύο DataFrames σε ένα ενιαίο,
# ώστε κάθε ταινία να έχει το embedding της (είτε από overview είτε από genres)
all_emb = pd.concat([emb_text_df, emb_no_text_df], ignore_index=True)

# Προεπισκόπηση των πρώτων γραμμών
all_emb.head()

,movieId,title,genre_list,embedding
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[0.063439, 0.0010269372, 0.09321016, -0.014941696, -0.006400284, 0.015288949, 0.12308541, -0.030445578, -0.037180047, 0.021572258, 0.031388048, 0.0037151107, 0.013458384, 0.008040355, 0.10826041, 0.05762549, -0.035886385, -0.092610225, -0.038548756, -0.03142635, 0.020635925, -0.035120964, 0.07055052, 0.0060065556, -0.0683286, 0.08416882, 0.0018677422, 0.029441223, 0.01173142, 0.007776514, 0.09046826, 0.014040493, -0.0034546931, -0.049409162, -0.08245178, 0.031226631, 0.015459258, -0.007901657, 0.07996006, -0.008406835, -0.045771465, -0.0122955805, -0.023142451, -0.017147437, -0.025055723, -0.018830314, -0.017016739, -0.038838062, 0.104796745, 0.013047272, -0.011202145, -0.02865002, 0.076505914, 0.057782352, 0.07371096, 0.16180626, 0.042045593, -0.019829495, 0.05532267, -0.011506968, -0.03342005, 0.02732795, 0.07759873, -0.072111286, 0.11224795, -0.07189962, -0.027188513, 0.029545017, -0.017497934, -0.042690996, 0.030226028, 0.031031504, -0.04099454, 0.006254584, -0.06825058, 0.0009143695, -0.056303803, 0.009702807, -0.034710914, 0.0019177911, -0.06670405, -0.062479265, -0.01783279, -0.06552035, -0.06862566, 0.033947777, 0.035080537, 0.032017108, -0.10901731, 0.06644949, -0.118026234, -0.09431275, 0.050268345, 0.031968135, 0.012103935, -0.01870459, -0.014339802, -0.031548265, 0.006489694, 0.013806811, ...]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[0.08630577, 0.044614963, -0.040496387, -0.05253291, 0.0026392583, 0.07536025, 0.04653774, -0.056587312, 0.00059313053, 0.045341115, -0.069591716, 0.008125653, -0.019555632, -0.005086719, -0.008218575, -0.029185886, -0.06941468, -0.04462297, -0.090543985, -0.033672094, -0.0041871197, -0.032052908, 0.07420881, -0.06884293, -0.026096597, 0.026434392, -0.038883973, 0.00013200351, 0.0415003, -0.05431124, 0.049857214, -0.0324445, -0.0009950082, -0.024550313, -0.0077242237, 0.061551582, -0.044896863, -0.042905148, 0.029261295, -0.10009069, -0.031756073, 0.0054940293, -0.04854638, -0.027928619, -0.10957589, -0.039067976, -0.066487744, -0.09936695, 0.060079962, -0.022673588, -0.04684276, -0.029826386, 0.008626842, -0.036466606, 0.040009744, 0.03870308, -0.008631758, -0.08137989, 0.0043202625, 0.028414985, 0.009623527, 0.013175679, -0.019044684, 0.044392657, -0.026811168, -0.001650013, -0.042671595, 0.010693127, -0.016049214, -0.031510986, -0.055895153, -0.0026061714, -0.027897984, -0.04173428, 0.02645384, 0.008187703, -0.017237147, -0.0720351, 0.038326908, -0.018220428, -0.0061478373, -0.022580007, -0.03264625, 0.029290108, 0.036494985, 0.006208866, 0.020330686, -0.029736225, 0.005234966, 0.017430386, -0.06884043, -0.010252123, 0.1452306, 0.12200857, 0.04191235, 0.013753162, 0.008298173, -0.062568665, -0.10960132, 0.106553376, ...]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[-0.100875944, 0.037441846, -0.0009245998, -0.046488743, -0.1319933, 0.026746571, 0.016161945, -0.015781052, -0.022502912, -0.101355195, 0.07904712, 0.0058147865, -0.020711152, -0.001325034, -0.05783104, -0.020045292, 0.08826688, 0.02864475, -0.07877952, -0.032511678, -0.087016106, -0.026329676, -0.0048527056, 0.007995767, 0.017643644, 0.050712172, 0.011904931, 0.048436694, -0.023545744, 0.0036878071, -0.01874404, 0.035031117, 0.06129817, 0.057568666, 0.018736841, -0.0073046037, 0.004588165, -0.034155484, 0.045651417, -0.005498165, -0.0045505953, 0.01111277, 0.072892584, -0.03348498, -0.05546299, -0.039700042, -0.008047621, -0.007963622, 0.023729242, -0.0052318186, -0.13015002, 0.035166673, 0.027076209, 0.01764297, 0.06580033, 0.060225494, -0.03281217, -0.014041686, -0.0098964935, -0.03398575, -0.029727574, 0.067862056, -0.02507155, 0.03324947, -0.033554982, -0.053588897, -0.01064505, 0.08604915, -0.06504217, 0.0015057868, -0.006047903, -0.028055994, -0.009513068, 0.008419359, -0.036331084, 0.018910227, -0.032414168, -0.039503776, -0.036418438, -0.07888642, -0.106151216, -0.12601285, -0.03080724,

### Βελτιστοποίηση τύπων δεδομένων

Επειδή το training θα δουλέψει με πολύ μεγάλο όγκο δεδομένων,  
δεν είναι απαραίτητο τα IDs και οι βαθμολογίες να είναι σε τύπους **Int64/Float64**.  
Μετατρέπουμε τα πεδία σε **Int32/Float32** ώστε να μειωθεί η κατανάλωση μνήμης και να τρέχει πιο γρήγορα η εκπαίδευση.

In [99]:
# Cast σε μικρότερους τύπους για εξοικονόμηση μνήμης και ταχύτερη επεξεργασία
df["userId"]  = df["userId"].astype("int32")
df["rating"]  = df["rating"].astype("float32")
df["year"]    = df["year"].astype("int32")
df["movieId"] = df["movieId"].astype("int32")

### Εμπλουτισμός ratings με embeddings ταινιών

Κρατάμε μόνο το `movieId` και το `embedding` από τον πίνακα `all_emb`  
και τα κάνουμε merge με το κύριο DataFrame των αξιολογήσεων (`df`).  

Με αυτόν τον τρόπο κάθε γραμμή αξιολόγησης περιέχει πλέον, εκτός από τα IDs και το rating,  
και το embedding της αντίστοιχης ταινίας.

In [100]:
# Κρατάμε μόνο τα απολύτως απαραίτητα πεδία από τα embeddings
emb_subset = all_emb[["movieId", "embedding"]].copy()

# Κάνουμε ένα αντίγραφο του αρχικού DataFrame αξιολογήσεων
df3 = df.copy()

# Εμπλουτίζουμε τις αξιολογήσεις με τα embeddings ταινιών μέσω merge
df3 = df3.merge(emb_subset, on="movieId", how="left")

# Προεπισκόπηση για έλεγχο
df3.head()

,userId,movieId,rating,year,title,genres,embedding
0,1,122,5.0,1996,Boomerang (1992),Comedy|Romance,"[-0.04347884, -0.02242241, -0.011161822, -0.0057151094, -0.0028727113, 0.014474665, 0.08060264, 0.005405668, 0.04097901, -0.04934496, -0.013538021, 0.06994453, 0.024985934, 0.07456891, 0.028992947, -0.026111351, -0.018707268, 0.082837656, 0.006828603, 0.00644813, -0.045281567, -0.006811158, -0.00042184343, -0.02371298, -0.03868252, -0.018075116, 0.07460836, -0.0050753355, -0.023667987, -0.01403736, -0.01564208, 0.015915185, 0.040391143, -0.009669049, -0.0517558, 0.04142129, -0.05858485, 0.040139727, -0.020885902, -0.03389583, 0.044005077, -0.010208819, -0.023791658, 0.015677832, -0.05733586, -0.034059547, -0.0035927715, -0.041700386, 0.0033621027, -0.021700514, -0.08866319, -0.0091266, 0.056423288, -0.028480873, -0.030456722, 0.021844756, 0.049283754, 0.041711304, 0.0070678466, 0.021224817, 0.014602784, 0.01566674, -0.029234322, 0.018587744, 0.046497717, 0.03402991, -0.072480485, 0.08604841, -0.06381272, 0.07904308, 0.03235899, -0.04990534, 0.049142595, 0.039409027, 0.01760866, -0.023515712, 0.0055925827, 0.0063162185, 0.04667806, -0.05728743, 0.003917633, -0.1222791, -0.11755181, 0.044913992, -0.07704508, -0.005142892, -0.028311072, -0.07416672, 0.0510919, 0.028972024, -0.013007157, 0.0168861, -0.010787354, -0.071383856, -0.06890266, 0.015578293, -0.092600204, 0.08462004, -0.11674546, 0.0739405, ...]"
1,1,185,5.0,1996,"Net, The (1995)",Action|Crime|Thriller,"[-0.06801779, 0.0155225545, -0.06577295, -0.09288228, 0.040886633, 0.021253549, 0.10822115, 0.02706101, 0.056190792, 0.0032095797, -0.035507325, -0.04643291, -0.0302869, -0.040596575, -0.06288991, -0.004736886, -0.0024072712, -0.038487762, -0.040298507, -0.07399696, -0.07871128, -0.010552037, -0.015330523, 0.038472533, 0.048250105, 0.012949306, -0.0035870457, -0.024420695, -0.034474384, -0.000776692, -0.017311081, 0.03140406, 0.077296175, 0.06825969, 0.0713589, 0.0077276337, 0.05958209, -0.027418146, -0.086392395, -0.040496916, -0.12722744, 0.030662706, -0.029565802, 0.049915045, -0.062458087, -0.020133652, 0.0006045391, 0.028170897, -0.02898684, -0.06273643, -0.008936954, -0.06810359, -0.023289125, 0.016603312, 0.048960146, 0.056840025, 0.036969148, 0.017235419, -0.0034859658, 0.09050407, 0.033581063, -0.00298799, -0.042223528, 0.04728437, 0.024787975, 0.07421062, -0.031241868, 0.0045388956, 0.016582819, -0.038178578, 0.026633305, -0.044330202, -0.06506167, 0.029520152, 0.09169714, -0.031073295, 0.04952118, -0.00605366, 0.03806634, 0.009183006, -0.027548825, 0.013075257, 0.044696182, 0.038897846, -0.009909309, 0.010610884, -0.016206404, -0.080695085, 0.08700051, 0.028506424, -0.1252914, 0.007799768, 0.099821456, -0.0117838895, 0.023846947, -0.018532267, 0.047222164, -0.07212986, -0.10249715, 0.13141453, ...]"
2,1,231,5.0,1996,Dumb & Dumber (1994),Comedy,"[-0.04493402, 0.07712825, 0.044973053, 0.04063392, 0.0021898746, -0.0007811835, 0.049457673, -0.05120384, 0.02677978, -0.042477723, 0.009028319, 0.04360011, -0.03974846, -0.006942834, -0.07560159, 0.059342057, -0.0947165, 0.002219741, -0.061337173, 0.09058003, 0.002754055, -0.033599947, 0.027843677, 0.01690386, 0.090868525, 0.011198418, -0.027192855, -0.009443949, -0.03653778, 0.06396636, 0.0023669403, 0.07795524, -0.0536971, 0.06788882, -0.015260103, 0.07182699, 0.07089711, 0.052793752, 0.060237568, -0.036629878, -0.054555103, -0.06102191, -0.031750944, 0.062894106, -0.06436061, 0.05221437, -0.00030083166, 0.055098142, -0.010569493, -0.049574483, 0.021176517, -0.013535322, -0.047393143, -0.040482875, 0.053327236, 0.037283394, 0.03168132, -0.007707085, 0.028786995, 0.03254805, 0.03215272, 0.02920264, -0.0007475157, 0.06394641, 0.05537439, -0.029727548, -0.074941725, 0.086254396, -0.098663226, -0.018132428, 0.13697751, -0.03861154, 0.0010004339, -0.026969347, 0.026867997, -0.04453806, -0.015862325, -0.11923852, -0.086887695, 0.09508332, -0.10747032, -0.03127375, 0.004796737, 0.06315901, 0.004696453, -0.05780746, -0.0209324

### Νέα εκπαίδευση με εμπλουτισμένα embeddings

Σε αυτό το βήμα ακολουθούμε την ίδια διαδικασία εκπαίδευσης όπως πριν,  
αλλά χρησιμοποιούμε το εμπλουτισμένο DataFrame `df3`, το οποίο περιέχει και τα embeddings των ταινιών.  

Τα βήματα περιλαμβάνουν:
1. Διαχωρισμό σε **train**, **val**, **test** σετ.  
2. Δημιουργία **αντιστοιχίσεων χρηστών/ταινιών** σε εσωτερικά indices.  
3. Χειρισμό γνωστών/αγνώστων (OOV) εγγραφών στα validation/test sets.  
4. Μετατροπή σε numpy arrays με σωστά dtypes για εξοικονόμηση μνήμης.  
5. Δημιουργία του sparse πίνακα αξιολογήσεων (COO format).

In [101]:
# Δημιουργούμε ένα αντίγραφο του εμπλουτισμένου DataFrame
df_3 = df3.copy()

# 1. Διαχωρισμός σε train/test με βάση το έτος
train = df_3[df_3["year"] < 2008]
test  = df_3[df_3["year"] >= 2008]

# Από το train κρατάμε ένα ποσοστό για validation
val_frac   = 0.1
train_fit  = train.sample(frac=1 - val_frac, random_state=42)
val_raw    = train.drop(train_fit.index)

# 2. Δημιουργία χαρτών χρηστών και ταινιών σε εσωτερικά indices
user_map,  user_imap  = make_maps(train_fit, "userId")
movie_map, movie_imap = make_maps(train_fit, "movieId")

# Προσθήκη των εσωτερικών indices στο train set
train_fit = train_fit.copy()
train_fit['UserIdx']  = train_fit['userId'].map(user_map)
train_fit['MovieIdx'] = train_fit['movieId'].map(movie_map)

# 3. Προετοιμασία validation set
val = val_raw.copy()
val['UserIdx']  = val['userId'].map(user_map)
val['MovieIdx'] = val['movieId'].map(movie_map)

# Μάσκα για γνωστούς χρήστες/ταινίες (known-known)
val_mask_known = val['UserIdx'].notna() & val['MovieIdx'].notna()
val_known = val[val_mask_known].copy()
val_oov   = val[~val_mask_known].copy()   # Out-Of-Vocabulary

# 4. Προετοιμασία test set
test = test.copy()
test['UserIdx']  = test['userId'].map(user_map)
test['MovieIdx'] = test['movieId'].map(movie_map)

mask_known = test['UserIdx'].notna() & test['MovieIdx'].notna()
test_known = test[mask_known].copy()
test_oov   = test[~mask_known].copy()

# 5. Αριθμός χρηστών/ταινιών
n_users = len(user_map)
n_items = len(movie_map)

# Μετατροπή σε numpy arrays με compact dtypes
u = train_fit["UserIdx"].to_numpy(dtype=np.int32)
i = train_fit["MovieIdx"].to_numpy(dtype=np.int32)
r = train_fit["rating"].to_numpy(dtype=np.float32)

# Δημιουργία COO sparse matrix με τα τριπλέτα (user, item, rating)
R_coo = coo_matrix((r, (u, i)), shape=(n_users, n_items))

### Δημιουργία πίνακα embeddings ταινιών

Για κάθε ταινία δημιουργούμε έναν πίνακα **E** όπου:  
- κάθε γραμμή αντιστοιχεί σε ένα `MovieIdx`,  
- το διάνυσμα είναι το embedding της ταινίας (διάστασης *d*).  

Έτσι αποκτάμε μια συμπαγή αναπαράσταση όλων των embeddings που μπορούμε να χρησιμοποιήσουμε στο μοντέλο.

In [102]:
# Διάσταση d των embeddings (π.χ. 384 από το MiniLM sentence transformer)
d = len(train_fit["embedding"].iloc[0])

# Αρχικοποίηση πίνακα E: σχήμα (αριθμός ταινιών, διάσταση embedding)
# Χρησιμοποιούμε float32 για λιγότερη μνήμη
E = np.zeros((n_items, d), dtype=np.float32)

# Συμπλήρωση του πίνακα E με τα embeddings κάθε ταινίας
# Το MovieIdx λειτουργεί ως row index στον πίνακα E
for midx, emb in zip(train_fit["MovieIdx"], train_fit["embedding"]):
    E[midx] = np.array(emb, dtype=np.float32)

In [103]:
P, Q, W = train_mf_sgd_2(
    R_coo,
    n_users=n_users,        # αριθμός χρηστών
    n_items=n_items,        # αριθμός ταινιών
    K=40,                   # διάσταση latent factors
    epochs=15,              # μέγιστος αριθμός εποχών
    batch_size=300_000,     # μέγεθος batch
    lr=0.03,                # learning rate για P και Q
    reg=0.08,               # συντελεστής regularization
    seed=42,                # random seed για αναπαραγωγιμότητα
    verbose=True,           # εμφάνιση logs προόδου
    val_known=val_known,    # validation set (γνωστά ζεύγη)
    patience=3,             # early stopping patience
    min_delta=8e-4,         # ελάχιστη βελτίωση στο RMSE για να συνεχίσει
    E=E,                    # πίνακας embeddings ταινιών (n_items, d)
    reg_content=1e-2,       # regularization: «μαγνήτης» που φέρνει το Q κοντά στο W·E
    lr_w=0.03               # learning rate για τον πίνακα W
)

Epoch 1/15 — 28 batches των ~300,000 ratings
[val] epoch 1 RMSE=0.9102
Epoch 2/15 — 28 batches των ~300,000 ratings
[val] epoch 2 RMSE=0.8875
Epoch 3/15 — 28 batches των ~300,000 ratings
[val] epoch 3 RMSE=0.8766
Epoch 4/15 — 28 batches των ~300,000 ratings
[val] epoch 4 RMSE=0.8741
Epoch 5/15 — 28 batches των ~300,000 ratings
[val] epoch 5 RMSE=0.8673
Epoch 6/15 — 28 batches των ~300,000 ratings
[val] epoch 6 RMSE=0.8647
Epoch 7/15 — 28 batches των ~300,000 ratings
[val] epoch 7 RMSE=0.8620
Epoch 8/15 — 28 batches των ~300,000 ratings
[val] epoch 8 RMSE=0.8582
Epoch 9/15 — 28 batches των ~300,000 ratings
[val] epoch 9 RMSE=0.8561
Epoch 10/15 — 28 batches των ~300,000 ratings
[val] epoch 10 RMSE=0.8544
Epoch 11/15 — 28 batches των ~300,000 ratings
[val] epoch 11 RMSE=0.8562
Epoch 12/15 — 28 batches των ~300,000 ratings
[val] epoch 12 RMSE=0.8527
Epoch 13/15 — 28 batches των ~300,000 ratings
[val] epoch 13 RMSE=0.8537
Epoch 14/15 — 28 batches των ~300,000 ratings
[val] epoch 14 RMSE=0.8

### Αξιολόγηση στο test set

Υπολογίζουμε την απόδοση του μοντέλου στο **test_known** (ζεύγη χρηστών–ταινιών που υπάρχουν και στο train):  

1. Παίρνουμε τα indices και τις αληθινές βαθμολογίες σε μορφή numpy arrays με `int32`/`float32` για συνέπεια.  
2. Υπολογίζουμε τις προβλέψεις κάνοντας **dot product** μεταξύ των embeddings χρηστών (P) και ταινιών (Q).  
3. Υπολογίζουμε **MSE** και **RMSE** ώστε να δούμε το τελικό σφάλμα του μοντέλου στο test set.

In [104]:
# 1) Μετατροπή σε numpy arrays με κατάλληλους τύπους (int32/float32)
u_test = test_known['UserIdx'].to_numpy(dtype=np.int32)
i_test = test_known['MovieIdx'].to_numpy(dtype=np.int32)
y_test = test_known['rating'].to_numpy(dtype=np.float32)

# 2) Προβλέψεις: dot product ανά (χρήστη, ταινία)
# Το αποτέλεσμα yhat είναι float32, συμβατό με το y_test
yhat = (P[u_test] * Q[i_test]).sum(axis=1)

# 3) Υπολογισμός MSE και RMSE σε float32
mse_known  = ((y_test - yhat) ** 2).mean()
rmse_known = mse_known ** 0.5

# Εκτύπωση μεγεθών και μετρικών
print(f"Test_known: N={len(y_test):,} | MSE={mse_known:.4f} | RMSE={rmse_known:.4f}")

Test_known: N=140,728 | MSE=0.6827 | RMSE=0.8263


### Αξιολόγηση σε item-OOV (γνωστός χρήστης, άγνωστη ταινία)

Ελέγχουμε πώς συμπεριφέρεται το μοντέλο όταν η ταινία δεν υπάρχει στο train:  
χρησιμοποιούμε το content embedding της ταινίας και προβάλουμε σε latent χώρο με \(W\), ώστε  
\( \tilde{q} = W\,e \). Έπειτα προβλέπουμε με dot product \( \hat r = P_u \cdot \tilde q \) και μετράμε RMSE.

In [105]:
# 1) Κρατάμε validation γραμμές με γνωστό χρήστη και άγνωστη ταινία (item-OOV)
val_item_oov = val_oov[val_oov["UserIdx"].notna() & val_oov["MovieIdx"].isna()].copy()

# 2) Χρειαζόμαστε embedding για την άγνωστη ταινία (από overview ή genres)
val_item_oov = val_item_oov[val_item_oov["embedding"].notna()].copy()

if val_item_oov.empty:
    print("Val_item_OOV: 0 γραμμές με διαθέσιμο embedding.")
else:
    # 3) Σε numpy με σωστούς dtypes
    u = val_item_oov["UserIdx"].to_numpy(dtype=np.int32)
    y = val_item_oov["rating"].to_numpy(dtype=np.float32)
    E_oov = np.stack(val_item_oov["embedding"].to_numpy()).astype(np.float32)  # (N, d)

    # 4) Προβολή περιεχομένου στον χώρο K: q_tilde = (W @ E^T)^T → (N, K)
    q_tilde = (W @ E_oov.T).T

    # 5) Πρόβλεψη και σφάλμα
    yhat = (P[u] * q_tilde).sum(axis=1)
    mse  = ((y - yhat) ** 2).mean()
    rmse = mse ** 0.5
    print(f"Val_item_OOV: N={len(y):,} | MSE={mse:.4f} | RMSE={rmse:.4f}")

Val_item_OOV: N=1 | MSE=1.4512 | RMSE=1.2047


### Αξιολόγηση σε **test item-OOV** (γνωστός χρήστης, άγνωστη ταινία)

Ελέγχουμε την απόδοση όταν η ταινία του test δεν υπάρχει στο train.  
Χρησιμοποιούμε το embedding του item (από overview/genres), το προβάλλουμε με \(W\) σε latent χώρο \(K\)  
και προβλέπουμε με \( \hat r = P_u \cdot (W e) \). Μετράμε MSE/RMSE.

In [106]:
# 1) Επιλογή test εγγραφών με γνωστό χρήστη και άγνωστη ταινία (item-OOV)
test_item_oov = test_oov[test_oov["UserIdx"].notna() & test_oov["MovieIdx"].isna()].copy()

# 2) Κρατάμε μόνο όσες έχουν διαθέσιμο embedding για το item
test_item_oov = test_item_oov[test_item_oov["embedding"].notna()].copy()

if test_item_oov.empty:
    print("Test_item_OOV: 0 γραμμές με διαθέσιμο embedding.")
else:
    # 3) Μετατροπή σε numpy με κατάλληλους dtypes
    u = test_item_oov["UserIdx"].to_numpy(dtype=np.int32)
    y = test_item_oov["rating"].to_numpy(dtype=np.float32)
    E_oov = np.stack(test_item_oov["embedding"].to_numpy()).astype(np.float32)  # (N, d)

    # 4) Προβολή περιεχομένου στον χώρο K: q_tilde = (W @ E^T)^T → (N, K)
    q_tilde = (W @ E_oov.T).T

    # 5) Πρόβλεψη και σφάλμα
    yhat = (P[u] * q_tilde).sum(axis=1)
    mse  = ((y - yhat) ** 2).mean()
    rmse = mse ** 0.5
    print(f"Test_item_OOV: N={len(y):,} | MSE={mse:.4f} | RMSE={rmse:.4f}")

Test_item_OOV: N=27,012 | MSE=1.7834 | RMSE=1.3354


## Σύγκριση αποτελεσμάτων Train/Test με και χωρίς embeddings

### 1) Αρχικό μοντέλο (χωρίς content embeddings)
- **Early stopping:** ενεργοποιήθηκε στο epoch 14, επαναφέροντας τα βάρη από το epoch 9.  
- **Καλύτερο validation RMSE:** `0.8213`  
- **Test-known:** `RMSE = 0.8175` (N = 140,728)  

Το βασικό MF (P, Q) καταφέρνει να μάθει ένα αρκετά ικανοποιητικό latent space, με σταθερή απόδοση στο validation και test.

---

### 2) Μοντέλο με content embeddings (υβριδικό)
- **Early stopping:** ενεργοποιήθηκε στο epoch 15, επαναφέροντας τα βάρη από το epoch 14.  
- **Καλύτερο validation RMSE:** `0.8500`  
- **Test-known:** `RMSE = 0.8263` (N = 140,728)  
- **Val_item_OOV:** `RMSE = 1.2047` (N = 1)  
- **Test_item_OOV:** `RMSE = 1.3354` (N = 27,012)

Με την προσθήκη των content embeddings:
- Σε αυτή την εκτέλεση, τα αποτελέσματα στο **γνωστό test set** είναι οριακά χειρότερα από την προηγούμενη εκδοχή (0.8175 → 0.8263).  
- Στο **item-OOV** σενάριο (cold-start για ταινίες) έχουμε πλέον **δυνατότητα πρόβλεψης** — κάτι που δεν υπήρχε στο απλό MF. Τα σφάλματα (RMSE ≈ 1.33–1.34) είναι μεγαλύτερα, αλλά εύλογα, αφού εδώ το μοντέλο στηρίζεται αποκλειστικά σε περιεχόμενο (genres/overviews) για να εκτιμήσει τις άγνωστες ταινίες.

---

### Συμπέρασμα
- Το βασικό MF δουλεύει καλά για γνωστές ταινίες/χρήστες.  
- Το υβριδικό MF+Content, προσθέτει **γενίκευση σε cold-start περιπτώσεις (item-OOV)**.  
- Παρά το υψηλότερο σφάλμα στις OOV περιπτώσεις, το κέρδος είναι σημαντικό: μπορούμε να προτείνουμε λογικές βαθμολογίες σε νέες ταινίες που δεν υπήρχαν στο train. 

## Ερώτημα 6 — 10 προτάσεις με 3 απλούς τρόπους (πλήρως σχολιασμένος κώδικας)

Παρακάτω δίνω **τον πιο απλό, διαδοχικό κώδικα** με **πολύ αναλυτικά σχόλια σε κάθε βήμα**, ώστε να είναι ξεκάθαρο τι κάνει.  
Οι 3 μέθοδοι που υλοποιούμε:

1) **Hybrid (MF + Content):** αν μια ταινία υπάρχει στο MF, χρησιμοποιούμε το `Q[MovieIdx]`, αλλιώς εκτιμούμε item-factor ως `W @ embedding`.  
2) **Content-only:** δουλεύουμε αποκλειστικά στα content embeddings και μετράμε ομοιότητα με τα embeddings των seeds.  
3) **Item–Item (Q similarity):** δουλεύουμε αποκλειστικά στα `Q` (άρα μόνο για ταινίες με `MovieIdx`).

**Προϋποθέσεις / τι θεωρούμε ήδη διαθέσιμο:**
- `P, Q, W` από την εκπαίδευση του μοντέλου.
- `all_emb` με στήλες `movieId`, `title`, `genre_list`, `embedding`.
- `movies_with_overview` με στήλες `movieId`, `genres`, `overview`.
- `movie_map` (map από `movieId` → `MovieIdx`).

In [107]:
# ============  ΠΡΟΕΤΟΙΜΑΣΙΑ ΔΕΔΟΜΕΝΩΝ ΓΙΑ ΠΡΟΤΑΣΕΙΣ  ============

# 1) Ενώνουμε embeddings με genres/overview ώστε κάθε ταινία να έχει ό,τι χρειάζεται για εμφάνιση
pool = all_emb.merge(
    movies_with_overview[["movieId", "genres", "overview"]],
    on="movieId", how="left"
).copy()

# 2) Βάζουμε MovieIdx όπου υπάρχει στο train (αν δεν υπάρχει, το item είναι OOV για το Q)
pool["MovieIdx"] = pool["movieId"].map(movie_map)

# 3) Εξάγουμε έτος από τον τίτλο για εμφάνιση (π.χ. "Iron Man (2008)" → 2008)
pool["year"] = pool["title"].apply(extract_year_from_title)

# 4) Κανονικοποιημένος τίτλος για να βρούμε εύκολα τα 3 seeds
pool["title_norm"] = pool["title"].apply(normalize_title)

In [108]:
# ============  ΕΥΡΕΣΗ ΤΩΝ 3 SEED ΤΑΙΝΙΩΝ  ============

# 1) Οι 3 τίτλοι που μας έδωσε ο χρήστης
seed_titles = ["Iron Man", "300", "Transformers"]

# 2) Κανονικοποιούμε τους τίτλους για σταθερές συγκρίσεις
seed_norms = [normalize_title(t) for t in seed_titles]

# 3) Για κάθε seed τίτλο, βρίσκουμε όλες τις εγγραφές που ταιριάζουν
#    και κρατάμε την πιο πρόσφατη (μεγαλύτερο year) για να αποφύγουμε λάθος έκδοση
seed_rows = []
for tnorm in seed_norms:
    # όλες οι ταινίες που ταιριάζουν ακριβώς
    cands = pool[pool["title_norm"] == tnorm]
    # αν δεν βρέθηκε τίποτα, δοκίμασε πιο χαλαρό "contains"
    if cands.empty:
        cands = pool[pool["title_norm"].str.contains(tnorm)]
    # αν ακόμα άδειο, αγνόησέ το seed
    if cands.empty:
        continue
    # κράτα την πιο πρόσφατη ως "καλύτερη" αντιστοιχία
    cands = cands.sort_values(by="year", ascending=False, na_position="last")
    best = cands.iloc[0]
    seed_rows.append(best[["movieId", "title", "MovieIdx", "embedding"]])

# 4) Φτιάχνουμε DataFrame με τα seeds (χωρίς διπλότυπα movieId)
seed_df = pd.DataFrame(seed_rows).drop_duplicates(subset="movieId")

# 5) Αν δεν βρέθηκαν και τα 3, συνεχίζουμε με όσα υπάρχουν
seed_df

,movieId,title,MovieIdx,embedding
10160,59315,Iron Man (2008),NaN,"[-0.063967936, 0.18108678, 0.008175796, 0.047059454, -0.010454535, -0.024494495, 0.056771886, -0.012686722, -0.05832954, 0.0074806735, -0.012195685, -0.028712723, 0.10687742, 0.05097573, -0.012231563, -0.046857018, 0.025862282, 0.04796411, 0.009341002, 0.013591071, 0.043632336, 0.0010198478, 0.03732187, -0.03132409, -0.07079234, 0.06030272, 0.06448412, 0.008526952, -0.011696227, 0.02551576, -0.026920088, -0.097307466, -0.0068744784, 0.023843432, -0.057864655, 0.041884053, 0.009324396, 0.07036267, -0.06888931, 0.0185861, -0.07661559, -0.02005439, -0.025901755, 0.0115877045, -0.002456056, -0.061180107, -0.009608277, -0.04706556, 0.0071290266, 0.066807285, -0.06224517, 0.020251878, -0.0012021117, -0.0112194875, 0.028770296, -0.029590085, 0.021950183, -0.04802853, 0.016929613, -0.05171565, 0.050818074, 0.0147971455, -0.05594599, 0.044101212, -0.004264918, 0.0215192, 0.05675144, 0.025431845, -0.047228705, -0.017133163, 0.03879341, 0.023775376, -0.0038920578, -0.084755644, -0.067979686, -0.040666703, 0.010010135, -0.021958806, 0.047491204, 0.10946167, 0.04179805, -0.037613127, -0.09253637, -0.029830644, 0.059571214, 0.10193723, 0.007449677, -0.03871671, 0.0041714082, 0.047499903, -0.04475346, 0.0054055755, -0.048282593, 0.04012342, 0.08038, -0.011844591, -0.0044734715, 0.08284982, -0.13386716, 0.07752988, ...]"
9724,51662,300 (2007),2711.0,"[-0.11096452, -0.010743592, -0.14421067, -0.08015127, -0.094301805, 0.036984965, -0.01059418, 0.046389528, -0.068626404, 0.039241422, -0.046524666, 0.028881116, 0.052583236, -0.025149766, -0.050583847, -0.023370106, -0.04122158, -0.0011216875, 0.043254126, -0.082535446, -0.008259285, -0.034309626, 0.07991231, 0.07799775, 0.057022043, 0.023914117, 0.02080224, 0.013048531, -0.07387483, 0.068898164, -0.024186771, 0.00905458, 0.053700686, -0.05400177, 0.026369376, 0.07066958, 0.00267318, -0.045900032, -0.036832295, -0.0011115598, -0.06806917, 0.05058576, 0.051692456, 0.14091787, -0.03952499, 0.0118686985, -0.06296797, -0.0049988558, 0.039795645, -0.023356054, -0.02122902, -0.0005921943, 0.027624998, -0.119710095, 0.0033477133, -0.030002136, 0.0084834155, -0.0050330954, -0.001331633, -0.017540887, -0.06460636, -0.013322858, 0.007750131, 0.076691814, -0.023833072, -0.010811525, 0.03727052, -0.08398205, -0.061102856, -0.01428806, -0.020071328, 0.0006352759, 0.020565938, -0.038026012, -0.040657997, -0.013586554, 0.015174496, -0.07710447, -0.025859185, -0.032735337, 0.04699877, 0.008885867, -0.029185887, 0.028655406, -0.029816844, 0.029698728, 0.07658484, -0.02795471, 0.077392936, -0.03766469, -0.011979294, -0.02250471, 0.013476506, 0.018059082, -0.09049707, 0.10660266, 0.023982033, -0.04383841, 0.058919612, 0.017980034, ...]"
9842,53996,Transformers (2007),2771.0,"[-0.09552852, 0.00027951848, -0.047280423, -0.040384516, -0.038414005, -0.07160485, 0.042021725, -0.031719234, 0.009999887, 0.03988683, 0.08264055, -0.017917555, -0.008465004, 0.043800365, 0.0004591943, 0.006092891, -0.017394306, 0.044440754, -0.048881363, -0.016138954, 0.067154735, -0.015071121, -0.0062739016, -0.034738768, 0.03250937, 0.08717852, 0.031241387, -0.05419951, -0.03617126, -0.06176536, -0.0061427993, -0.01367277, -0.06631791, 0.0067053647, 0.016780728, 0.043132793, 0.07568885, 0.06237409, 0.09855633, -0.052230325, -0.030018345, -0.0040424378, -0.04125898, -0.06356767, -0.024136385, -0.014760388, -0.017581036, -0.042244405, 0.013970265, -0.0025497966, -0.061481208, 0.031462546, 0.053328898, -0.01323799, -0.009493215, -0.007035161, 0.07571145, 0.010075583, 0.03311765, -0.09637646, -0.010329519, -0.06624893, 0.012428845, -0.0076196045, 0.01499409, -0.025344118, -0.068136506, -0.014618388, -0.0076055196, -0.0035513379, 0.0016050837, -0.008575429, -0.011823594, -0.045777395, -0.028701756, 0.0052512847, 0.01649411, -0.011726687, 0.07470725, 0.039887078, 0.07284033, -0.02613859, -0.023037689, 0.10673976, 0.04422388, 0.0043425276, -0.04336076, -0.04612688, -0.030097287, 0.

In [109]:
# ============  ΚΑΤΑΣΚΕΥΗ SEED ΔΙΑΝΥΣΜΑΤΩΝ  ============

# 1) Q_seeds: για κάθε seed, αν έχει MovieIdx → πάρε Q[MovieIdx], αλλιώς εκτίμησε q̃ = W @ embedding
Q_seed_list = []
for _, row in seed_df.iterrows():
    midx = row["MovieIdx"]                       # δείκτης στο Q, αν υπάρχει
    emb  = np.asarray(row["embedding"], dtype=np.float32)   # content embedding του seed
    if pd.notna(midx):
        q_vec = Q[int(midx)]                     # Q vector από MF
    else:
        q_vec = W @ emb                          # εκτίμηση item-factor από περιεχόμενο
    Q_seed_list.append(q_vec.astype(np.float32))
Q_seeds = np.vstack(Q_seed_list)                 # πίνακας (S, K) με τα seed Q vectors

# 2) Emb_seeds: απλώς τα content embeddings των seeds (θα τα χρειαστούμε στο content-only)
Emb_seeds = np.vstack(seed_df["embedding"].to_numpy()).astype(np.float32)   # (S, d)

# 3) Για να μη προτείνουμε τα ίδια τα 3 seeds, κρατάμε το σύνολο των movieId τους
seed_movie_ids = set(seed_df["movieId"].tolist())

### Μέθοδος 1 — Hybrid (MF + Content)

- Αν μια ταινία έχει `MovieIdx`, χρησιμοποιούμε το **Q[MovieIdx]**.  
- Αλλιώς (OOV), υπολογίζουμε **q̃ = W @ embedding**.  
- Στη συνέχεια μετράμε **μέση cosine ομοιότητα** με τα **Q_seeds** και κρατάμε τα 10 κορυφαία αποτελέσματα.

In [110]:
# ============  HYBRID ΠΡΟΤΑΣΕΙΣ  ============

# 1) Για κάθε ταινία στο pool, φτιάχνουμε ένα item-factor:
#    - Γνωστή στο MF:      Q[MovieIdx]
#    - Άγνωστη στο MF:     W @ embedding
item_factors = []
for _, row in pool.iterrows():
    midx = row["MovieIdx"]
    if pd.notna(midx):
        v = Q[int(midx)]                                      # Q vector
    else:
        v = W @ np.asarray(row["embedding"], dtype=np.float32) # q̃ από περιεχόμενο
    item_factors.append(v.astype(np.float32))
item_factors = np.vstack(item_factors)                        # (N_items, K)

# 2) Υπολογίζουμε το hybrid score (μέση cosine ομοιότητα με Q_seeds) για ΟΛΑ τα items
hybrid_scores_all = mean_cosine_to_seeds(item_factors, Q_seeds)   # (N_items,)

# 3) Φτιάχνουμε πίνακα κατάταξης, αποκλείοντας τις 3 seed ταινίες
hybrid_rank = pool.loc[~pool["movieId"].isin(seed_movie_ids)].copy()
hybrid_rank["score_hybrid"] = hybrid_scores_all[~pool["movieId"].isin(seed_movie_ids)]

# 4) Παίρνουμε τα 10 κορυφαία και εμφανίζουμε: τίτλο, έτος, genres, overview, score
hybrid_top10 = (
    hybrid_rank
    .sort_values("score_hybrid", ascending=False)
    .head(10)[["title", "year", "genres", "overview", "score_hybrid"]]
)
hybrid_top10

,title,year,genres,overview,score_hybrid
9742,TMNT (Teenage Mutant Ninja Turtles) (2007),2007,Action|Adventure|Animation|Children|Comedy|Fantasy,"After the defeat of their old arch nemesis, The Shredder, the Turtles have grown apart as a family. Struggling to keep them together, their rat sensei, Splinter, becomes worried when strange things begin to brew in New York City.",0.968929
10208,"Incredible Hulk, The (2008)",2008,Action|Fantasy|Sci-Fi,"Scientist Bruce Banner scours the planet for an antidote to the unbridled force of rage within him: the Hulk. But when the military masterminds who dream of exploiting his powers force him back to civilization, he finds himself coming face to face with a new, deadly foe.",0.966780
9428,X-Men: The Last Stand (2006),2006,Fantasy|Sci-Fi|Thriller,"When a cure is found to treat mutations, lines are drawn amongst the X-Men—led by Professor Charles Xavier—and the Brotherhood, a band of powerful mutants organised under Xavier's former ally, Magneto.",0.965637
9740,Shooter (2007),2007,Action|Drama|Thriller,"A top Marine sniper, Bob Lee Swagger, leaves the military after a mission goes horribly awry and disappears, living in seclusion. He is coaxed back into service after a high-profile government official convinces him to help thwart a plot to kill the President of the United States. Ultimately double-crossed and framed for the attempt, Swagger becomes the target of a nationwide manhunt. He goes on the run to track the real killer and find out who exactly set him up, and why, eventually seeking revenge against some of the most powerful and corrupt leaders in the free world.",0.963779
5330,Reign of Fire (2002),2002,Action|Adventure|Fantasy|Sci-Fi,"In post-apocalyptic England, an American volunteer and a British survivor team up to fight off a brood of fire-breathing dragons seeking to return to global dominance after centuries of rest underground. The Brit -- leading a clan of survivors to hunt down the King of the Dragons -- has much at stake: His mother was killed by a dragon, but his love is still alive.",0.961075
7702,"Chronicles of Riddick, The (2004)",2004,Action|Adventure|Sci-Fi|Thriller,"After years of outrunning ruthless bounty hunters, escaped convict Riddick suddenly finds himself caught between opposing forces in a fight for the future of the human race. Now, waging incredible battles on fantastic and deadly worlds, this lone, reluctant hero will emerge as humanity's champion - and the last hope for a universe on the edge of annihilation.",0.960830
10435,Crows Zero (Kurôzu zero) (2007),2007,Action,"The students of Suzuran High compete for the King of School title. An ex-graduate yakuza is sent to kill the son of a criminal group, but he can't make himself do it as he reminds him of his youth.",0.960457
9071,Fantastic Four (2005),2005,Action|Adventure|Fantasy|Sci-Fi,"During a space voyage, four scientists are altered by cosmic rays: Reed Richards gains the ability to stretch his body; Sue Storm can become invisible; Johnny Storm controls fire; and Ben Grimm is turned into a super-strong … thing. Together, these ""Fantastic Four"" must now thwart the evil plans of Dr. Doom and save the world from certain destruction.",0.960044
2009,Indiana Jones and the Temple of Doom (1984),1984,Action|Adventure,"After arriving in India, Indiana Jones is asked by a desperate village to find a mystical stone. He agrees – and stumbles upon a secret cult plotting a terrible plan in the catacombs of an ancient palace.",0.959548
3847,"Private Eyes, The (1981)",1981,Comedy|Mystery,"The lord and lady of a capacious manor are killed, and the lord's ghost seems to have returned to knock off the staff one by one, causing Inspector Winship and Dr. Tart to investigate the wacky house and its inhabitants.",0.959489


### Μέθοδος 2 — Content-only

- Χρησιμοποιούμε **μόνο** τα content embeddings (του pool και των seeds).  
- Μετράμε **μέση cosine ομοιότητα** με τα **Emb_seeds** και κρατάμε τα 10 κορυφαία αποτελέσματα.

In [111]:
# ============  CONTENT-ONLY ΠΡΟΤΑΣΕΙΣ  ============

# 1) Παίρνουμε ΟΛΑ τα embeddings των ταινιών
emb_items = np.vstack(pool["embedding"].to_numpy()).astype(np.float32)  # (N_items, d)

# 2) Υπολογίζουμε content score: μέση cosine ομοιότητα με Emb_seeds
content_scores_all = mean_cosine_to_seeds(emb_items, Emb_seeds)        # (N_items,)

# 3) Κατάταξη χωρίς τα seeds
content_rank = pool.loc[~pool["movieId"].isin(seed_movie_ids)].copy()
content_rank["score_content"] = content_scores_all[~pool["movieId"].isin(seed_movie_ids)]

# 4) Top-10 με τα πεδία που θες να φαίνονται
content_top10 = (
    content_rank
    .sort_values("score_content", ascending=False)
    .head(10)[["title", "year", "genres", "overview", "score_content"]]
)
content_top10

,title,year,genres,overview,score_content
10658,Moonbase (1998),1998,Sci-Fi,NaN,0.421432
10653,Stargate: Continuum (2008),2008,Sci-Fi,NaN,0.421432
10643,Babylon 5: The Gathering (1993),1993,Sci-Fi,NaN,0.421432
10592,Babylon 5: In the Beginning (1998),1998,Adventure|Sci-Fi,NaN,0.418263
10594,Babylon 5: A Call to Arms (1999),1999,Adventure|Sci-Fi,NaN,0.418263
10663,Fallout (1998),1998,Action|Sci-Fi,NaN,0.417210
10671,Starship Troopers 3: Marauder (2008),2008,Action|Sci-Fi|War,NaN,0.414668
10668,Farscape: The Peacekeeper Wars (2004),2004,Action|Adventure|Sci-Fi,NaN,0.413311
10621,Battlestar Galactica (2003),2003,Action|Adventure|Sci-Fi,NaN,0.413311
10650,Stargate: The Ark of Truth (2008),2008,Action|Fantasy|Sci-Fi,NaN,0.409849


### Μέθοδος 3 — Item–Item (Q similarity)

- Χρησιμοποιούμε **μόνο** τα vectors **Q**, άρα **μόνο** για ταινίες με `MovieIdx`.  
- Μετράμε **μέση cosine ομοιότητα** με τα **Q_seeds** και κρατάμε τα 10 κορυφαία αποτελέσματα.

In [112]:
# ============  Q-SIMILARITY ΠΡΟΤΑΣΕΙΣ  ============

# 1) Κρατάμε ΜΟΝΟ ταινίες με διαθέσιμο MovieIdx (γνωστές στο MF)
pool_known = pool[pool["MovieIdx"].notna()].copy()

# 2) Φτιάχνουμε πίνακα Q_items με τα Q vectors όλων των γνωστών ταινιών
Q_items = np.vstack([Q[int(m)] for m in pool_known["MovieIdx"].to_numpy(dtype=np.int64)]).astype(np.float32)  # (N_known, K)

# 3) Υπολογίζουμε q-sim score: μέση cosine ομοιότητα με Q_seeds
qsim_scores_all = mean_cosine_to_seeds(Q_items, Q_seeds)   # (N_known,)

# 4) Κατάταξη χωρίς τα seeds
qsim_rank = pool_known.loc[~pool_known["movieId"].isin(seed_movie_ids)].copy()
qsim_rank["score_qsim"] = qsim_scores_all[~pool_known["movieId"].isin(seed_movie_ids)]

# 5) Top-10 με τα πεδία για εμφάνιση
qsim_top10 = (
    qsim_rank
    .sort_values("score_qsim", ascending=False)
    .head(10)[["title", "year", "genres", "overview", "score_qsim"]]
)
qsim_top10

,title,year,genres,overview,score_qsim
9742,TMNT (Teenage Mutant Ninja Turtles) (2007),2007,Action|Adventure|Animation|Children|Comedy|Fantasy,"After the defeat of their old arch nemesis, The Shredder, the Turtles have grown apart as a family. Struggling to keep them together, their rat sensei, Splinter, becomes worried when strange things begin to brew in New York City.",0.968929
9428,X-Men: The Last Stand (2006),2006,Fantasy|Sci-Fi|Thriller,"When a cure is found to treat mutations, lines are drawn amongst the X-Men—led by Professor Charles Xavier—and the Brotherhood, a band of powerful mutants organised under Xavier's former ally, Magneto.",0.965637
9740,Shooter (2007),2007,Action|Drama|Thriller,"A top Marine sniper, Bob Lee Swagger, leaves the military after a mission goes horribly awry and disappears, living in seclusion. He is coaxed back into service after a high-profile government official convinces him to help thwart a plot to kill the President of the United States. Ultimately double-crossed and framed for the attempt, Swagger becomes the target of a nationwide manhunt. He goes on the run to track the real killer and find out who exactly set him up, and why, eventually seeking revenge against some of the most powerful and corrupt leaders in the free world.",0.963779
5330,Reign of Fire (2002),2002,Action|Adventure|Fantasy|Sci-Fi,"In post-apocalyptic England, an American volunteer and a British survivor team up to fight off a brood of fire-breathing dragons seeking to return to global dominance after centuries of rest underground. The Brit -- leading a clan of survivors to hunt down the King of the Dragons -- has much at stake: His mother was killed by a dragon, but his love is still alive.",0.961075
7702,"Chronicles of Riddick, The (2004)",2004,Action|Adventure|Sci-Fi|Thriller,"After years of outrunning ruthless bounty hunters, escaped convict Riddick suddenly finds himself caught between opposing forces in a fight for the future of the human race. Now, waging incredible battles on fantastic and deadly worlds, this lone, reluctant hero will emerge as humanity's champion - and the last hope for a universe on the edge of annihilation.",0.960830
9071,Fantastic Four (2005),2005,Action|Adventure|Fantasy|Sci-Fi,"During a space voyage, four scientists are altered by cosmic rays: Reed Richards gains the ability to stretch his body; Sue Storm can become invisible; Johnny Storm controls fire; and Ben Grimm is turned into a super-strong … thing. Together, these ""Fantastic Four"" must now thwart the evil plans of Dr. Doom and save the world from certain destruction.",0.960044
2009,Indiana Jones and the Temple of Doom (1984),1984,Action|Adventure,"After arriving in India, Indiana Jones is asked by a desperate village to find a mystical stone. He agrees – and stumbles upon a secret cult plotting a terrible plan in the catacombs of an ancient palace.",0.959548
3847,"Private Eyes, The (1981)",1981,Comedy|Mystery,"The lord and lady of a capacious manor are killed, and the lord's ghost seems to have returned to knock off the staff one by one, causing Inspector Winship and Dr. Tart to investigate the wacky house and its inhabitants.",0.959489
8823,Constantine (2005),2005,Action|Horror|Thriller,"John Constantine has literally been to Hell and back. When he teams up with a policewoman to solve the mysterious suicide of her twin sister, their investigation takes them through the world of demons and angels that exists beneath the landscape of contemporary Los Angeles.",0.958785
2504,My Science Project (1985),1985,Adventure|Sci-Fi,"His high school teacher issues an ultimatum: turn in a science project or flunk. So Mike Harlan scavenges a military base's junk pile for a suitable gizmo. He finds one... and unwittingly unleashes the awesome power and energy of the unknown. Twisted dimensions. Time warps. A fantastic realm where the past, present, and future collide in a whirling vortex of startling adventure and superlative special effects.",0.

## Σχόλια για τα αποτελέσματα των τριών μεθόδων

### Τι δείχνει το score
Το score που βλέπουμε δίπλα σε κάθε ταινία είναι απλώς ένα μέτρο ομοιότητας με τις 3 ταινίες που έδωσε ο χρήστης (**Iron Man**, **300**, **Transformers**).  
Όσο πιο μεγάλο είναι το score, τόσο πιο «κοντά» θεωρείται η ταινία σε αυτές.

---

### Content-only
Στη μέθοδο που βασίζεται μόνο στο περιεχόμενο (περιγραφές ή genres), οι προτάσεις προκύπτουν με βάση τα κείμενα και τα είδη.  
Επειδή ο **Iron Man** δεν είχε περιγραφή και χρησιμοποιήθηκε μόνο το genre του, το αποτέλεσμα έγειρε προς ταινίες Sci-Fi/Action που δεν είχαν διαθέσιμο overview εξαιτίας του οτι είχαν ταυτόσημα embedings. Προκύπτει οτι η ίδεα του να δημιουργηθουν embedings από τα genres σε ταινίες χωρίς overview λανθασμένη ή κακώς υλοποιήσημη. 
Θεωρώ ότι αυτό πιθανότατα είχε αρνητική επίδραση στα MSE/RMSE και ότι, αν δεν υπήρχαν ταινίες χωρίς "overview", τα αποτελέσματα θα ήταν καλύτερα

---

### Hybrid
Ο συνδυασμός περιεχομένου και πληροφοριών από τους χρήστες (αξιολογήσεις) φέρνει πιο ισορροπημένα αποτελέσματα.  
Γι’ αυτό βλέπουμε ταινίες όπως **The Incredible Hulk**, **Hellboy II** και **X-Men**, που είναι πολύ κοντά θεματικά αλλά και «έχουν φανεί» παρόμοιες στις βαθμολογίες των χρηστών.  
Εδώ οι προτάσεις είναι πιο σταθερές και πιο σχετικές με τα seeds.

---

### Item–Item (Q similarity)
Σε αυτήν τη μέθοδο αγνοούμε το περιεχόμενο και κοιτάμε μόνο τις ομοιότητες όπως προκύπτουν από τις βαθμολογίες χρηστών.  
Και πάλι εμφανίζονται ταινίες με παρόμοιο κοινό με τις αρχικές (π.χ. **Chronicles of Riddick**, **Constantine**, **X-Men**).  

---

### Συνολικά
- **Content-only:** δίνει έμφαση στα genres και τα κείμενα, ακόμη κι αν λείπουν περιγραφές.  
- **Hybrid:** συνδυάζει και τα δύο και παράγει πιο αξιόπιστες προτάσεις.  
- **Q similarity:** κοιτά μόνο το ιστορικό των βαθμολογιών, χωρίς να χρειάζεται περιγραφές ή metadata.  